#*Importing libraries*


In [ ]:
!pip install -Uq pymcel geopandas
!pip install -Uq sgp4

import spiceypy as spy
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import sys
import pandas as pd
import numpy as np
import requests
import plotly.graph_objects as go
import numpy as np
from sgp4.io import twoline2rv
from sgp4.earth_gravity import wgs84
from scipy.spatial.distance import euclidean

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.2/910.2 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
# Archivos:
!gdown 1jiovEI3tCqbi1LX4-en9psHHPaVdZYc9
!gdown 1br7KIg5LABQQTw_ZNJ-DeRN8Io9oi8uN

Downloading...
From: https://drive.google.com/uc?id=1jiovEI3tCqbi1LX4-en9psHHPaVdZYc9
To: /content/ne_110m_admin_0_countries.shx
100% 1.52k/1.52k [00:00<00:00, 6.70MB/s]
Downloading...
From: https://drive.google.com/uc?id=1br7KIg5LABQQTw_ZNJ-DeRN8Io9oi8uN
To: /content/ne_110m_admin_0_countries.shp
100% 181k/181k [00:00<00:00, 83.2MB/s]


In [ ]:
#@title Code for Graph the earth
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import sys
import pandas as pd
import numpy as np
import requests
import plotly.graph_objects as go
import numpy as np

def dibuja_tierra():
  def plot_back(fig):
      """back half of sphere"""
      clor=f'rgb(220, 220, 220)'
      R = np.sqrt(6368.134)
      u_angle = np.linspace(0, np.pi, 25)
      v_angle = np.linspace(0, np.pi, 25)
      x_dir = np.outer(R*np.cos(u_angle), R*np.sin(v_angle))
      y_dir = np.outer(R*np.sin(u_angle), R*np.sin(v_angle))
      z_dir = np.outer(R*np.ones(u_angle.shape[0]), R*np.cos(v_angle))
      fig.add_surface(z=z_dir, x=x_dir, y=y_dir, colorscale=[[0, clor], [1, clor]], opacity=1.0, showlegend=False, lighting=dict(diffuse=0.1)) # opacity=fig.sphere_alpha, colorscale=[[0, fig.sphere_color], [1, fig.sphere_color]])

  def plot_front(fig):
      """front half of sphere"""
      clor=f'rgb(220, 220, 220)'
      R = np.sqrt(6368.134)
      u_angle = np.linspace(-np.pi, 0, 25)
      v_angle = np.linspace(0, np.pi, 25)
      x_dir = np.outer(R*np.cos(u_angle), R*np.sin(v_angle))
      y_dir = np.outer(R*np.sin(u_angle), R*np.sin(v_angle))
      z_dir = np.outer(R*np.ones(u_angle.shape[0]), R*np.cos(v_angle))
      fig.add_surface(z=z_dir, x=x_dir, y=y_dir, colorscale=[[0, clor], [1, clor]], opacity=1.0, showlegend=False, lighting=dict(diffuse=0.1)) # opacity=fig.sphere_alpha, colorscale=[[0, fig.sphere_color], [1, fig.sphere_color]])

  def plot_polygon(poly):
      xy_coords = poly.exterior.coords.xy
      lon = np.array(xy_coords[0])
      lat = np.array(xy_coords[1])

      lon = lon * np.pi/180
      lat = lat * np.pi/180

      R = 6378.134
      x = R * np.cos(lat) * np.cos(lon)
      y = R * np.cos(lat) * np.sin(lon)
      z = R * np.sin(lat)
      return x, y, z

  # Read the shapefile.  Creates a DataFrame object
  gdf = gpd.read_file("ne_110m_admin_0_countries.shp")
  fig = go.Figure()
  plot_front(fig)
  plot_back(fig)

  marker = dict(color=[f'rgb({np.random.randint(0,256)}, {np.random.randint(0,256)}, {np.random.randint(0,256)})' for _ in range(25)],
            size=10)

  for i in gdf.index :
      # print(gdf.loc[i].NAME)            # Call a specific attribute

      polys = gdf.loc[i].geometry         # Polygons or MultiPolygons

      if polys.geom_type == 'Polygon':
          x, y, z = plot_polygon(polys)
          fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color=f'rgb(0, 0,0)'), showlegend=False) )

      elif polys.geom_type == 'MultiPolygon':

          for poly in polys.geoms:
              x, y, z = plot_polygon(poly)
              fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color=f'rgb(0, 0,0)'), showlegend=False) )
  fig.show()
  return fig

In [ ]:
fig = dibuja_tierra()

# *Creating  .txt  with the TLE information*

In [ ]:
#@title Geosynchronous (Last 30 days)

%%file TLE_GEO.txt
0 SYNCOM 2 (A 26)
1   634U 63031A   24094.62906801 -.00000012  00000-0  00000+0 0  9990
2   634  31.6809 311.7212 0008388 183.7806 350.0102  1.00262555222293
0 SYNCOM 3
1   858U 64047A   24100.55129917  .00000128  00000-0  00000+0 0  9998
2   858   5.0050  72.1227 0001555 130.5681  36.4557  1.00351824180726
0 INTELSAT 1-F1
1  1317U 65028A   24100.48773028  .00000119  00000-0  00000+0 0  9999
2  1317   5.2038  71.3007 0003606 106.0521  48.4834  1.00323769119171
0 ATS 1
1 02608U 66110A   24099.92873402 -.00000290  00000-0  00000-0 0  9991
2 02608   2.9654  76.3391 0004200 165.2416 227.2971  1.00370681126784
0 INTELSAT 2-F2
1 02639U 67001A   24099.89969831 -.00000207  00000-0  00000-0 0  9991
2 02639   1.1949  83.7947 0005459 193.2104  35.1786  1.00246878111231
0 INTELSAT 2-F3
1 02717U 67026A   24099.94492020  .00000099  00000-0  00000-0 0  9990
2 02717   2.1145  69.0471 0016950 214.1426 272.8976  1.00211433115889
0 INTELSAT 2-F4
1  2969U 67094A   24100.50164780 -.00000284  00000-0  00000+0 0  9994
2  2969   2.0351  81.1238 0009572 117.5450 117.5880  1.00193015108782
0 ATS 3
1 03029U 67111A   24099.92040969 -.00000095  00000-0  00000-0 0  9998
2 03029   1.1148  86.5217 0017907  11.3160 326.0252  1.00271912206614
0 LES 6
1 03431U 68081D   24099.50049517 -.00000265  00000-0  00000-0 0  9993
2 03431   1.3806 324.0918 0008714  15.6130 322.1437  1.00331384112766
0 OPS 0757 (TACSAT)
1 03691U 69013A   24099.85418553 -.00000260  00000-0  00000-0 0  9990
2 03691   0.8357  70.5036 0017316 169.7102 186.6999  1.00439716106072
0 ATS 5
1 04068U 69069A   24099.77103216  .00000097  00000-0  00000-0 0  9995
2 04068   2.0118 269.8120 0003301   1.2062  68.7818  0.99490161 38376
0 SKYNET 1
1  4250U 69101A   24100.13523850 -.00000100  00000-0  00000+0 0  9993
2  4250   1.6329 277.1982 0021852 308.7517 275.8018  1.00267747126496
0 INTELSAT 3-F6
1 04297U 70003A   24099.82964517 -.00000275  00000-0  00000-0 0  9993
2 04297   1.0268 268.7068 0011095  81.2018 304.7356  0.99161304168932
0 NATO 2A
1 04353U 70021A   24099.54975017 -.00000126  00000-0  00000-0 0  9997
2 04353   2.3922 284.7270 0001316 223.5891 145.5556  1.00277866117755
0 INTELSAT 3-F7
1 04376U 70032A   24099.43412828  .00000032  00000-0  00000-0 0  9993
2 04376   0.4267 131.9005 0002701 214.1023  13.9836  1.00238232 39343
0 NATO 2B
1 04902U 71009A   24099.63414563 -.00000102  00000-0  00000-0 0  9992
2 04902   3.1872 270.9520 0004399 186.1448 223.7369  1.00271453110625
0 OPS 9431 (DSCS 2-1)
1 05587U 71095A   24099.63449372 -.00000112  00000-0  00000-0 0  9994
2 05587   3.6472 273.1658 0002407 336.3928  73.1612  1.00260020114005
0 OPS 9432 (DSCS 2-2)
1 05588U 71095B   24099.94135227 -.00000007  00000-0  00000-0 0  9993
2 05588   3.6115 273.9020 0003639  86.2237 176.6505  1.00310839 39340
0 INTELSAT 4-F3
1  5709U 71116A   24100.54713508  .00000033  00000-0  00000+0 0  9992
2  5709   8.3221 298.4681 0012245 106.4790 189.2461  0.99624340110941
0 INTELSAT 4-F4
1 05775U 72003A   24099.86715189 -.00000197  00000-0  00000-0 0  9993
2 05775   8.6219 301.5337 0004110 128.5431  66.6482  0.99838258129415
0 OPS 1570
1 05851U 72010A   24099.90082961 -.00000246  00000-0  00000-0 0  9991
2 05851   1.6883 272.7467 0006937   8.0315  37.2977  0.99784809 17955
0 INTELSAT 4-F5
1 06052U 72041A   24099.48825025 -.00000242  00000-0  00000-0 0  9994
2 06052   7.6947 294.9921 0002399 226.9666 132.8061  1.00097204 38650
0 OPS 6063
1 06380U 73013A   24099.95658122 -.00000112  00000-0  00000-0 0  9998
2 06380   2.3955 272.8902 0017792 228.1018 117.4375  1.00273189 18403
0 ANIK A2 (TELESAT 2)
1 06437U 73023A   24099.92908209 -.00000276  00000-0  00000-0 0  9994
2 06437   8.7854 302.8546 0011278  66.1284 101.3382  0.99797456 39172
0 OPS 6157
1 06691U 73040A   24099.77085812  .00000109  00000-0  00000-0 0  9998
2 06691   2.8237 264.3787 0005232  29.2450  44.0462  0.99817101 17966
0 INTELSAT 4-F7
1 06796U 73058A   24099.56471948 -.00000045  00000-0  00000-0 0  9999
2 06796   9.0889 303.7153 0006010 175.5714 187.5800  0.99143411127723
0 COSMOS 637
1 07229U 74017A   24099.93943778  .00000002  00000-0  00000-0 0  9995
2 07229   2.8777 273.6940 0038496  45.3875 207.5925  1.00773021114148
0 WESTAR 1
1  7250U 74022A   24100.36663154 -.00000236  00000-0  00000+0 0  9991
2  7250   8.9260 304.6724 0004501  49.7364 314.5602  0.99907301114342
0 MOLNIYA 1-S
1  7392U 74060A   24100.03437937 -.00000354  00000-0  00000+0 0  9993
2  7392   3.4264 272.0293 0009198 232.3414 173.5752  1.00272841126142
0 WESTAR 2
1 07466U 74075A   24099.93282406 -.00000304  00000-0  00000-0 0  9992
2 07466   9.0507 306.1665 0004793 174.3209  10.0920  0.99857993129678
0 INTELSAT 4-F8
1  7544U 74093A   24100.41787505  .00000005  00000-0  00000+0 0  9991
2  7544   9.6698 310.6537 0004877 156.2063  71.8926  0.99792338117286
0 SKYNET 2B
1 07547U 74094A   24099.66686611  .00000049  00000-0  00000-0 0  9994
2 07547   5.5871 288.6244 0003843 163.5161  45.1682  1.00156418116747
0 SYMPHONIE 1
1  7578U 74101A   24099.96519646 -.00000367  00000-0  00000+0 0  9996
2  7578   6.6181 287.0395 0001588  85.9710 292.3540  0.99954238109641
0 SMS 2
1 07648U 75011A   24099.56402330 -.00000054  00000-0  00000-0 0  9996
2 07648   7.4800 292.0775 0011843 244.1653 126.7992  0.99507008127723
0 ANIK A3 (TELESAT-3)
1  7790U 75038A   24099.56550269 -.00000180  00000-0  00000+0 0  9997
2  7790   9.4300 309.0441 0002307 113.2493 248.8026  1.00037103121950
0 INTELSAT 4-F1
1 07815U 75042A   24099.16830847  .00000068  00000-0  00000-0 0  9994
2 07815   9.9343 312.0871 0010778 243.0517 116.7019  0.99257832114015
0 SYMPHONIE 2
1 08132U 75077A   24099.48996705 -.00000033  00000-0  00000-0 0  9999
2 08132   6.0939 285.0049 0004080  57.3308 281.4748  0.99960615134455
0 INTELSAT 4A-F1
1 08330U 75091A   24099.70237135  .00000025  00000-0  00000-0 0  9995
2 08330   9.5740 310.4030 0004664 208.0314 170.1497  0.99921046113486
0 COSMOS 775
1 08357U 75097A   24099.84355128  .00000141  00000-0  00000-0 0  9997
2 08357   4.4280 276.0570 0012966 158.8419 101.3637  1.00274495123740
0 GOES 1 (SMS-C)
1 08366U 75100A   24099.64627660 -.00000009  00000-0  00000-0 0  9997
2 08366   6.6420 287.4154 0001749 336.8325  44.2365  1.00304785177057
0 SATCOM 1
1 08476U 75117A   24099.49692887 -.00000201  00000-0  00000-0 0  9997
2 08476   9.6961 310.6732 0013184 247.5452 102.7745  0.99600429 38394
0 OPS 3165
1 08482U 75118A   24099.76920469  .00000098  00000-0  00000-0 0  9998
2 08482   4.3532 274.5885 0031265   1.7530  52.1449  1.00479141 18420
0 RADUGA 1
1 08513U 75123A   24099.95623312 -.00000092  00000-0  00000-0 0  9992
2 08513   4.8700 279.2148 0005572 241.0014  96.2677  1.00246597186794
0 CTS
1 08585U 76004A   24099.69810723  .00000097  00000-0  00000-0 0  9999
2 08585   5.9876 283.2947 0012192 240.6326 141.8924  1.00286059114282
0 INTELSAT 4A-F2
1 08620U 76010A   24099.66608292  .00000090  00000-0  00000-0 0  9992
2 08620   9.6162 310.0333 0003829 265.1589 277.8484  0.99683093612649
0 MARISAT 1
1 08697U 76017A   24099.76659403 -.00000025  00000-0  00000-0 0  9993
2 08697   7.7106 300.9173 0013056 175.3067 198.6413  0.99208028174170
0 LES 8
1 08746U 76023A   24096.41180943 -.00000088  00000-0  00000-0 0  9991
2 08746  20.8779  47.2907 0032313 131.1479  56.7199  1.00269055121635
0 LES 9
1 08747U 76023B   24099.38316557 -.00000112  00000-0  00000-0 0  9991
2 08747  20.8445  47.2605 0023632 155.3012  28.3864  1.00268737121669
0 NATO 3A
1 08808U 76035A   24099.63529253  .00000052  00000-0  00000-0 0  9991
2 08808   6.8087 302.4855 0022576 309.3232  50.0797  0.99838134739288
0 COMSTAR 1
1  8838U 76042A   24100.30214781 -.00000007  00000-0  00000+0 0  9996
2  8838   9.6681 310.7376 0002076   2.6102   8.1641  0.99827607112655
0 OPS 2112
1  8916U 76059A   24099.96510944 -.00000298  00000-0  00000+0 0  9992
2  8916   5.4316 280.7930 0005035 195.3971 188.7205  0.99473779 17534
0 PALAPA 1
1 09009U 76066A   24099.76702913  .00000094  00000-0  00000-0 0  9995
2 09009   9.5786 311.0635 0004512 184.1416 181.7437  1.00094846119948
0 COMSTAR 2
1  9047U 76073A   24100.10338826 -.00000238  00000-0  00000+0 0  9994
2  9047   9.6064 311.1393 0002503 133.5191 245.5915  0.99968731175535
0 RADUGA 2
1 09416U 76092A   24099.95397056 -.00000001  00000-0  00000-0 0  9994
2 09416   5.5402 281.4347 0027704  18.0091 305.4945  1.00278703121323
0 EKRAN 1
1  9503U 76107A   24100.10025544 -.00000138  00000-0  00000+0 0  9998
2  9503   5.6289 281.8388 0070013 164.7200 227.0618  1.00221478190548
0 KIKU 2 (ETS 2)
1 09852U 77014A   24099.96702392 -.00000372  00000-0  00000-0 0  9998
2 09852   7.2730 291.9589 0002403 147.3300 236.0165  1.00015609119236
0 PALAPA 2
1  9862U 77018A   24100.00287720 -.00000314  00000-0  00000+0 0  9999
2  9862  10.2350 316.0586 0005696 114.8308  77.7067  1.00045519135095
0 INTELSAT 4A-F4
1 10024U 77041A   24099.93360727 -.00000294  00000-0  00000+0 0  9992
2 10024  10.4628 316.8023 0011259 242.2845 295.7863  0.99445653113323
0 GOES 2
1 10061U 77048A   24099.89787084 -.00000341  00000-0  00000-0 0  9998
2 10061   8.3202 300.3812 0012411  41.5893 320.2271  0.99401397 38337
0 HIMAWARI 1 (GMS 1)
1 10143U 77065A   24099.83269097 -.00000202  00000-0  00000-0 0  9997
2 10143   8.4317 299.7255 0016912 216.1219 155.2453  0.99253706113296
0 RADUGA 3
1 10159U 77071A   24099.95814764 -.00000215  00000-0  00000-0 0  9995
2 10159   6.1693 284.3629 0008037 259.7889  83.0033  1.00353805169023
0 SIRIO
1 10294U 77080A   24099.90083227 -.00000121  00000-0  00000-0 0  9998
2 10294   9.0155 305.7577 0008872 142.2916 149.4547  1.00270897109909
0 EKRAN 2
1 10365U 77092A   24099.96032319 -.00000284  00000-0  00000-0 0  9992
2 10365   6.2836 285.2324 0035458 359.6294 353.8676  1.00241955126836
0 METEOSAT 1
1 10489U 77108A   24099.96693691 -.00000357  00000-0  00000+0 0  9993
2 10489   7.0861 290.3773 0011124  83.6971 300.6729  1.00308560129248
0 INTELSAT 4A-F3
1 10557U 78002A   24099.76955278  .00000089  00000-0  00000-0 0  9995
2 10557  10.5102 318.2998 0001906 234.5595 137.7782  0.99907217 40764
0 OPS 6391 (FLTSATCOM 1)
1 10669U 78016A   24100.51555851 -.00000274  00000-0  00000+0 0  9999
2 10669   9.0417 299.3267 0006685 219.2310  27.7132  0.99007372746732
0 INTELSAT 4A-F6
1 10778U 78035A   24100.37054756 -.00000052  00000-0  00000+0 0  9997
2 10778  10.3055 317.3895 0004918 286.5993  85.7630  1.00265837114242
0 OPS 8790
1 10787U 78038A   24099.69706296  .00000096  00000-0  00000-0 0  9993
2 10787   4.9477 317.1607 0011512 285.2985  57.4579  1.00484137 17227
0 YURI (BSE)
1 10792U 78039A   24100.16482618  .00000119  00000-0  00000+0 0  9999
2 10792   7.4588 292.6804 0013865 301.1094  67.1524  1.00243560110733
0 OTS 2
1 10855U 78044A   24099.54916852 -.00000320  00000-0  00000-0 0  9992
2 10855   9.7331 310.0544 0007302 350.9411 242.3762  0.99142276196064
0 GOES 3
1 10953U 78062A   24099.56672101 -.00000119  00000-0  00000-0 0  9995
2 10953   8.7132 305.4331 0001259 134.0034 238.2070  0.99358468120593
0 COMSTAR 3
1 10975U 78068A   24099.87966034 -.00000225  00000-0  00000+0 0  9997
2 10975  10.9800 320.6624 0019864 302.0065 244.9827  0.99201223128082
0 GEOS 2
1 10981U 78071A   24099.63905212  .00000055  00000-0  00000-0 0  9994
2 10981   7.5536 294.4743 0002894   7.3599   5.7747  0.99375187166053
0 RADUGA 4
1 10987U 78073A   24099.95710337 -.00000129  00000-0  00000-0 0  9992
2 10987   6.8126 288.3218 0009204  21.4947 311.4929  1.00193246111585
0 ANIK B1 (TELESAT-4)
1 11153U 78116A   24099.96841628 -.00000368  00000-0  00000+0 0  9999
2 11153  10.9335 321.5721 0017778 315.3099  45.5157  0.99819724128068
0 EKRAN 3
1 11273U 79015A   24100.10086461 -.00000187  00000-0  00000+0 0  9995
2 11273   7.1265 290.7086 0039372 349.6756  35.7126  1.00326408126987
0 RADUGA 5
1 11343U 79035A   24100.52993014 -.00000159  00000-0  00000+0 0  9991
2 11343   7.2917 291.4049 0005659 185.5659 351.8411  1.00294131121315
0 GORIZONT 2
1 11440U 79062A   24099.59717890 -.00000051  00000-0  00000-0 0  9991
2 11440   7.7061 294.3101 0002431 344.4894 202.9284  1.00239280130005
0 WESTAR 3
1 11484U 79072A   24099.73700633  .00000098  00000-0  00000-0 0  9993
2 11484  11.5443 326.6027 0000599  44.6287 144.1053  0.99938667132526
0 EKRAN 4
1 11561U 79087A   24099.85023986 -.00000307  00000-0  00000-0 0  9991
2 11561   7.5742 293.3732 0010471 141.8799 165.4968  1.00274323 39353
0 GORIZONT 3
1 11648U 79105A   24099.66451650  .00000101  00000-0  00000-0 0  9991
2 11648   7.8783 295.9077 0011867 233.1112 315.8452  1.00224426114465
0 OPS 6393 (FLTSATCOM 3)
1 11669U 80004A   24099.56654696 -.00000247  00000-0  00000-0 0  9990
2 11669   7.8373 310.5913 0022966 219.4890 146.4461  1.00378709161844
0 RADUGA 6
1 11708U 80016A   24100.16543535  .00000106  00000-0  00000+0 0  9997
2 11708   7.6772 294.8419 0001365 215.8771 153.9214  1.00233794114135
0 GOES 4
1 11964U 80074A   24099.73300329 -.00000001  00000-0  00000-0 0  9993
2 11964   9.7278 309.2629 0033232  90.5343  93.6484  0.99230532119507
0 RADUGA 7
1 12003U 80081A   24099.69915150  .00000081  00000-0  00000-0 0  9992
2 12003   7.8810 297.0165 0008583 285.4155  88.7095  1.00178879115292
0 SBS 1
1 12065U 80091A   24100.16795899  .00000067  00000-0  00000+0 0  9995
2 12065  11.3763 323.4373 0008817 151.0295 203.5541  0.99814798121750
0 EKRAN 6
1 12120U 80104A   24099.66529972  .00000075  00000-0  00000-0 0  9991
2 12120   8.1616 298.3167 0006557  73.9248 116.5579  1.00278571119390
0 ESIAFI 1 (COMSTAR 4)
1 12309U 81018A   24099.13887903 -.00000072  00000-0  00000-0 0  9996
2 12309  10.2495 316.8764 0004298  94.0697 265.8583  1.00266126159423
0 RADUGA 8
1 12351U 81027A   24099.89667674 -.00000260  00000-0  00000-0 0  9996
2 12351   8.2197 297.4790 0079900 248.0942 298.6734  1.00367379114521
0 GOES 5
1 12472U 81049A   24099.62037427  .00000076  00000-0  00000-0 0  9996
2 12472  10.7948 321.2369 0000424 317.1066   0.3392  1.00269086114822
0 APPLE
1 12545U 81057B   24099.90178685 -.00000171  00000-0  00000+0 0  9995
2 12545   8.8684 303.7373 0025826 200.9158 178.9319  1.00057552114387
0 EKRAN 7
1 12564U 81061A   24099.95997510 -.00000274  00000-0  00000-0 0  9997
2 12564   8.4139 300.3996 0004541  76.0706 260.1450  1.00233361121499
0 RADUGA 9
1 12618U 81069A   24099.66503865  .00000089  00000-0  00000-0 0  9992
2 12618   8.4707 300.8462 0003481 151.8321  34.8251  1.00207317 39327
0 HIMAWARI 2 (GMS 2)
1 12677U 81076A   24100.50494053  .00000069  00000-0  00000+0 0  9996
2 12677   9.6956 310.2823 0007948 308.3128 352.4122  0.99550065124276
0 SBS 2
1 12855U 81096A   24100.57695822 -.00000298  00000-0  00000+0 0  9995
2 12855  11.5350 326.9243 0005032 193.2981 345.5530  1.00116490110560
0 RADUGA 10
1 12897U 81102A   24099.59735295 -.00000073  00000-0  00000-0 0  9999
2 12897   8.5703 301.9406 0004590 198.1037 342.4494  1.00293636109327
0 SATCOM 3R
1 12967U 81114A   24099.18007314 -.00000309  00000-0  00000-0 0  9991
2 12967  13.0261 337.5913 0002244 329.0924 265.5545  1.00118386120050
0 INTELSAT 503
1 12994U 81119A   24099.34437410  .00000018  00000-0  00000-0 0  9994
2 12994  12.3678 332.4879 0037713 241.8823 309.0806  0.99467627118746
0 SATCOM 4
1 13035U 82004A   24099.27072671  .00000091  00000-0  00000-0 0  9992
2 13035  13.5955 341.2851 0040769 178.4218 181.5449  0.99587961117244
0 EKRAN 8
1 13056U 82009A   24100.51566338 -.00000160  00000-0  00000+0 0  9995
2 13056   8.9043 303.3997 0026483 322.7518 282.5766  0.99951789110629
0 WESTAR 4
1 13069U 82014A   24099.97024376 -.00000330  00000-0  00000+0 0  9992
2 13069  13.4678 340.6642 0000974 268.2855  83.1622  0.99778263117823
0 INTELSAT 504
1 13083U 82017A   24099.17300156 -.00000106  00000-0  00000-0 0  9998
2 13083  12.4661 332.9263 0011602 209.5161 353.6415  0.99223527133811
0 INSAT 1A
1 13129U 82031A   24099.96502241 -.00000387  00000-0  00000-0 0  9992
2 13129   8.9369 305.1152 0014782  67.6146 291.7339  1.00254052158414
0 COSMOS 1366
1 13177U 82044A   24099.90133948 -.00000158  00000-0  00000-0 0  9993
2 13177  10.1187 305.0840 0009388 112.8781 183.0562  1.00286712125599
0 WESTAR 5
1 13269U 82058A   24100.53851477 -.00000325  00000-0  00000+0 0  9991
2 13269  13.8811 342.9057 0014479 252.7123 301.7868  0.99218752111607
0 ANIK D1 (TELESAT 6)
1 13431U 82082A   24099.90439753 -.00000046  00000-0  00000-0 0  9997
2 13431  13.1093 338.6097 0004327 348.2278  10.5669  1.00141209 38597
0 EKRAN 9
1 13554U 82093A   24100.10216994 -.00000231  00000-0  00000+0 0  9995
2 13554   8.8705 306.0524 0025260 352.9189  24.3438  1.00213385111976
0 GORIZONT 6
1 13624U 82103A   24099.93491260 -.00000254  00000-0  00000-0 0  9990
2 13624   9.0852 308.0818 0000190 350.0521 203.8845  1.00287987112934
0 SATCOM C5
1 13631U 82105A   24099.65142755 -.00000121  00000-0  00000-0 0  9994
2 13631  13.0117 338.1550 0005495  65.7823 283.0929  1.00274054118352
0 SBS 3
1 13651U 82110B   24099.93995991 -.00000144  00000-0  00000-0 0  9996
2 13651  13.3525 340.0833 0009244 101.4397  87.7810  0.99938796 38554
0 ANIK C3 (TELESAT-5)
1 13652U 82110C   24099.12342883 -.00000288  00000-0  00000-0 0  9993
2 13652  13.3354 339.8943 0002934  98.3697 101.7352  0.99905258 38510
0 SAKURA 2A (CS-2A)
1 13782U 83006A   24099.18000253 -.00000270  00000-0  00000-0 0  9991
2 13782  11.6075 325.8969 0016671 312.0660 294.1865  0.99415174119897
0 RADUGA 12
1 13974U 83028A   24100.09816690 -.00000054  00000-0  00000+0 0  9990
2 13974   9.8525 312.3621 0002040 100.5483 248.8471  1.00254453161365
0 SATCOM 1R
1 13984U 83030A   24099.89697404 -.00000030  00000-0  00000-0 0  9994
2 13984  13.9045 344.3482 0009291 106.9556 143.7645  0.99844088122498
0 GOES 6
1 14050U 83041A   24099.70028281  .00000034  00000-0  00000-0 0  9995
2 14050  11.4720 326.7004 0002461 160.9531 189.3997  1.00292079207990
0 INTELSAT 506
1 14077U 83047A   24100.36497681 -.00000012  00000-0  00000+0 0  9995
2 14077  13.3442 339.1493 0037146 336.4785 220.6021  0.99161806116154
0 NAHUEL I2 (ANIK C2)
1 14133U 83059B   24099.94690649 -.00000270  00000-0  00000+0 0  9997
2 14133  13.6991 341.8340 0039239 102.2574 258.1335  0.99126582118091
0 PALAPA B1
1 14134U 83059C   24099.56776528 -.00000249  00000-0  00000+0 0  9993
2 14134  12.6049 335.0862 0004751 103.2710 244.4725  1.00150132112820
0 GALAXY 1
1 14158U 83065A   24099.00421907 -.00000269  00000-0  00000-0 0  9996
2 14158  14.1755 347.6433 0000755  92.4191  85.6679  1.00143682 38611
0 ARABSAT 1DR (TELSTAR 3A)
1 14234U 83077A   24098.53395303 -.00000293  00000-0  00000-0 0  9998
2 14234  14.3542 347.8740 0026799 146.9316 213.2039  0.99526844 38379
0 INSAT 1B
1 14318U 83089B   24099.86033080  .00000070  00000-0  00000-0 0  9994
2 14318  12.2679 332.5747 0008626 210.9372 339.4266  1.00279011117196
0 SATCOM 2R
1 14328U 83094A   24100.55392275 -.00000344  00000-0  00000+0 0  9998
2 14328  14.7291 351.8953 0017160  61.5052 123.8689  0.99413320147697
0 GALAXY 2
1 14365U 83098A   24099.84015509 -.00000173  00000-0  00000-0 0  9995
2 14365  14.1745 347.9493 0006830  89.9366 145.9576  1.00142522124225
0 EKRAN 11
1 14377U 83100A   24099.03279546 -.00000343  00000-0  00000-0 0  9995
2 14377   9.4653 310.5680 0000293 323.7850  36.0704  1.00270077 39350
0 INTELSAT 507
1 14421U 83105A   24099.23003365 -.00000097  00000-0  00000-0 0  9992
2 14421  12.9380 336.7016 0006239  89.0842 118.8184  0.99689928129254
0 YURI 2A (BS-2A)
1 14659U 84005A   24099.56392275 -.00000307  00000-0  00000-0 0  9996
2 14659  12.2497 330.0496 0017200 191.5879  20.1786  0.99053413110399
0 RADUGA 14
1 14725U 84016A   24100.09973331 -.00000140  00000-0  00000+0 0  9995
2 14725  10.1268 316.4274 0004361 143.6601 210.1531  1.00299161110072
0 COSMOS 1540
1 14783U 84022A   24100.09929819 -.00000115  00000-0  00000+0 0  9990
2 14783  11.0065 311.6914 0006667 227.4815 128.5997  1.00284797113001
0 COSMOS 1546
1 14867U 84031A   24099.89877251 -.00000104  00000-0  00000-0 0  9999
2 14867  10.2187 317.0631 0022317   0.6497 277.1852  1.00257988 39351
0 STTW-T2
1 14899U 84035A   24099.30035699  .00000064  00000-0  00000-0 0  9990
2 14899  10.8319 323.2567 0009467 163.1100 195.5268  1.00234308 39347
0 GORIZONT 9
1 14940U 84041A   24099.84733433 -.00000299  00000-0  00000-0 0  9995
2 14940  10.2686 317.8669 0004370 239.0348  40.0970  1.00295223113926
0 CHINASAT 5 (SPACENET 1)
1 14985U 84049A   24100.14193924 -.00000266  00000-0  00000+0 0  9996
2 14985  14.6687 354.3798 0008229 213.7184 329.6702  0.99982533123303
0 RADUGA 15
1 15057U 84063A   24100.30258292  .00000072  00000-0  00000+0 0  9995
2 15057  10.2649 317.4319 0004784 175.6189 190.7924  1.00268380133074
0 GORIZONT 10
1 15144U 84078A   24099.49997465 -.00000317  00000-0  00000-0 0  9991
2 15144  10.3847 318.8975 0000389 310.6032  48.1572  1.00345713146558
0 HIMAWARI 3 (GMS 3)
1 15152U 84080A   24099.80507983  .00000097  00000-0  00000-0 0  9994
2 15152  11.8399 329.0728 0008323 121.6155  71.6041  0.99831623144666
0 TELSTAR 302
1 15237U 84093D   24099.07418524 -.00000302  00000-0  00000-0 0  9993
2 15237  14.5671 351.3650 0004819 129.3937  62.5886  0.99782513 38452
0 GALAXY 3
1 15308U 84101A   24099.36438289  .00000083  00000-0  00000-0 0  9999
2 15308  14.5429 351.1137 0006068  42.7419 144.7950  0.99837950 38496
0 SPACENET 2
1 15385U 84114A   24099.28383785  .00000006  00000-0  00000-0 0  9994
2 15385  14.8336 358.0937 0010100  71.7483 113.8242  0.99753121118992
0 GORIZONT 11
1 15484U 85007A   24099.90284745 -.00000193  00000-0  00000-0 0  9999
2 15484  10.5965 320.9657 0002608 175.3468 111.6786  1.00152039144619
0 ARABSAT 1A
1 15560U 85015A   24099.80699432  .00000093  00000-0  00000+0 0  9992
2 15560  12.8003 336.8935 0028817 212.0335 344.0552  1.00378538118088
0 BRAZILSAT 1
1 15561U 85015B   24099.90207722 -.00000129  00000-0  00000-0 0  9993
2 15561  14.5673 350.8361 0002680   5.3698 258.7006  0.99544897 38384
0 COSMOS 1629
1 15574U 85016A   24099.93647902 -.00000217  00000-0  00000+0 0  9996
2 15574  10.5944 320.0432 0005944 216.5799 333.8809  1.00262366142436
0 NAHUEL I1 (ANIK C1)
1 15642U 85028B   24099.78285097 -.00000214  00000-0  00000-0 0  9999
2 15642  14.9278 358.2250 0016245 125.1070  96.1917  0.99659110 38420
0 GSTAR 1
1 15677U 85035A   24099.62799865 -.00000121  00000-0  00000+0 0  9997
2 15677  14.6209 354.8111 0006588  87.2673 236.0026  1.00272517117220
0 TELECOM 1B
1 15678U 85035B   24099.70045685  .00000042  00000-0  00000-0 0  9993
2 15678  11.4232 326.4167 0001488  80.3460 271.3987  1.00169183128998
0 MORELOS 1
1 15824U 85048B   24099.86765177 -.00000311  00000-0  00000-0 0  9992
2 15824  14.4772 348.8834 0005978 118.3484 163.2916  0.99469516109291
0 ARABSAT 1B
1 15825U 85048C   24099.73909487 -.00000041  00000-0  00000-0 0  9993
2 15825  13.2327 339.8938 0015311  15.2192 171.3624  1.00439955127409
0 TELSTAR 303
1 15826U 85048D   24099.90475235 -.00000258  00000-0  00000-0 0  9992
2 15826  14.5970 352.2993 0001829 171.4797  99.8783  0.99870322 38510
0 INTELSAT 511
1 15873U 85055A   24100.14037282 -.00000242  00000-0  00000+0 0  9999
2 15873  13.8830 344.6272 0015094  79.5106 105.0636  1.00082174116534
0 RADUGA 16
1 15946U 85070A   24100.43629385 -.00000253  00000-0  00000+0 0  9992
2 15946  10.8192 322.2566 0001410 184.1104 177.6902  1.00285999114213
0 OPTUS A1 (AUSSAT 1)
1 15993U 85076B   24099.48243727 -.00000192  00000-0  00000-0 0  9997
2 15993  13.9785 344.6902 0002803  44.2383 147.5957  0.99770531111616
0 ASC 1
1 15994U 85076C   24100.19517730 -.00000246  00000-0  00000+0 0  9996
2 15994  14.3114 349.3477 0005255  69.7265 125.5720  1.00288291109994
0 INTELSAT 512
1 16101U 85087A   24098.23243562 -.00000152  00000-0  00000+0 0  9995
2 16101  14.6015 349.6225 0008610 316.8756 260.8997  0.99113354122371
0 USA 11
1 16116U 85092B   24100.00775046 -.00000278  00000-0  00000+0 0  9992
2 16116  14.7265 353.3589 0003397 144.1074  37.0472  0.99328113 17963
0 COSMOS 1700
1 16199U 85102A   24099.89957227 -.00000169  00000-0  00000-0 0  9992
2 16199  11.0605 324.2266 0005435 186.3888  89.6726  1.00316476143650
0 RADUGA 17
1 16250U 85107A   24099.73396054  .00000114  00000-0  00000-0 0  9992
2 16250  10.9705 323.7385 0013023 202.7694 332.2971  1.00144426143667
0 MORELOS 2
1 16274U 85109B   24100.55163110 -.00000354  00000-0  00000+0 0  9994
2 16274  14.7440   2.6888 0006371 117.9935  53.7748  0.99567832129359
0 OPTUS A2 (AUSSAT 2)
1 16275U 85109C   24099.21391543 -.00000308  00000-0  00000-0 0  9998
2 16275  13.9982 345.9901 0001785 321.4041 278.0229  0.99947216 38538
0 SATCOM K2
1 16276U 85109D   24099.87376559 -.00000015  00000-0  00000-0 0  9996
2 16276  14.8671 358.4380 0007150 257.2544 276.4079  0.99610798117636
0 SATCOM K1
1 16482U 86003B   24099.35882870  .00000056  00000-0  00000-0 0  9994
2 16482  14.8909 357.1458 0002376 133.3033  46.2699  0.99556343116481
0 STTW-1
1 16526U 86010A   24099.67104319 -.00000156  00000-0  00000-0 0  9993
2 16526  11.8170 329.0400 0001334 197.0347 353.7792  1.00210930147442
0 BS-2B (YURI 2B)
1 16597U 86016A   24099.93865458 -.00000243  00000-0  00000+0 0  9990
2 16597  13.2004 337.8423 0013523 193.2563 350.9720  0.99294077120267
0 GSTAR 2
1 16649U 86026A   24099.87856054 -.00000300  00000-0  00000-0 0  9999
2 16649  14.6139 351.8720 0004254  64.9553 205.2925  0.99793700108665
0 BRAZILSAT 2
1 16650U 86026B   24099.47688086 -.00000103  00000-0  00000-0 0  9996
2 16650  14.7662 356.4241 0002283  20.8492 171.2048  0.99646342 38413
0 COSMOS 1738
1 16667U 86027A   24099.48863353 -.00000275  00000-0  00000-0 0  9998
2 16667  11.6337 323.7525 0014750 123.2936  69.7686  1.00277485139334
0 GORIZONT 12
1 16769U 86044A   24100.03464045 -.00000379  00000-0  00000+0 0  9993
2 16769  11.3317 326.4920 0001488 277.5994  75.3119  1.00282526139260
0 GOES 7
1 17561U 87022A   24099.89848041 -.00000079  00000-0  00000-0 0  9992
2 17561  13.6572 343.3687 0004137  65.3288 191.8405  0.99889119118975
0 AGILA 1 (PALAPA B2P)
1 17706U 87029A   24099.07183562 -.00000305  00000-0  00000-0 0  9992
2 17706  14.6929 354.2454 0004183  68.0129 107.8424  0.99913219 34537
0 KIKU 5 (ETS 5)
1 18316U 87070A   24099.85077742 -.00000185  00000-0  00000-0 0  9990
2 18316  13.6101 340.9534 0007159  58.5418 209.0181  0.99331071120898
0 COSMOS 1888
1 18384U 87084A   24100.30362719  .00000104  00000-0  00000+0 0  9993
2 18384  11.9303 331.4752 0002588 167.5176 189.9908  1.00338975133674
0 COSMOS 1894
1 18443U 87091A   24100.42570362  .00000099  00000-0  00000+0 0  9998
2 18443  11.9012 330.9662 0003419  44.5337 173.5230  1.00329768112807
0 COSMOS 1897
1 18575U 87096A   24099.90186221 -.00000233  00000-0  00000-0 0  9998
2 18575  12.1299 332.3260 0008922 107.0143 169.5672  1.00316279133178
0 RADUGA 21
1 18631U 87100A   24100.00466014 -.00000310  00000-0  00000+0 0  9997
2 18631  12.4555 331.0920 0002657 149.7232  25.0046  1.00245521133069
0 STTW-2
1 18922U 88014A   24099.67025999 -.00000140  00000-0  00000-0 0  9990
2 18922  13.9137 345.5570 0007209 140.2986  29.4243  1.00304904132095
0 SPACENET 3R
1 18951U 88018A   24099.74833816 -.00000359  00000-0  00000-0 0  9991
2 18951  14.5620   5.6527 0000952 284.1260 305.5661  0.99664378116644
0 COSMOS 1940
1 19073U 88034A   24100.10225697 -.00000312  00000-0  00000+0 0  9998
2 19073  12.0752 332.1315 0018603 272.4722  79.0014  1.00674206113328
0 PAS 1
1 19217U 88051C   24099.17229993 -.00000112  00000-0  00000-0 0  9993
2 19217  14.2364  10.5059 0002342  27.9765 134.4466  0.99389038140225
0 INSAT 1C
1 19330U 88063A   24099.89858794 -.00000128  00000-0  00000-0 0  9995
2 19330  12.3973 333.3248 0009418 120.9458 144.1879  1.00229022112321
0 COSMOS 1961
1 19344U 88066A   24099.89901872 -.00000164  00000-0  00000-0 0  9999
2 19344  12.4760 335.1714 0006277 132.6091 133.6882  1.00269699130996
0 GORIZONT 16
1 19397U 88071A   24099.86299661  .00000001  00000-0  00000-0 0  9995
2 19397  12.5436 335.2733 0013735  75.2735 109.6886  0.99984027130242
0 GSTAR 3
1 19483U 88081A   24099.63405861 -.00000115  00000-0  00000-0 0  9995
2 19483  12.4438 324.7895 0007489 173.4813 181.9902  1.00272458126572
0 SBS 5
1 19484U 88081B   24100.55333909 -.00000340  00000-0  00000+0 0  9992
2 19484  14.4200   6.8743 0004714 325.7686 207.4537  0.99797947121418
0 CS 3B
1 19508U 88086A   24099.87628925 -.00000000  00000-0  00000+0 0  9994
2 19508  14.9966 359.7125 0003188 347.3859 199.2338  0.99223998129302
0 TDRS 3
1 19548U 88091B   24100.07271642 -.00000309  00000-0  00000+0 0  9991
2 19548  13.2166 346.4090 0038414 343.3765 204.9739  1.00263192117350
0 RADUGA 22
1 19596U 88095A   24100.30232185  .00000068  00000-0  00000+0 0  9994
2 19596  12.5473 335.6551 0007591 161.5305 184.4352  1.00306080113690
0 TDF 1
1 19621U 88098A   24099.28268517  .00000031  00000-0  00000-0 0  9999
2 19621  14.9631 356.3322 0009016 233.4400 313.1164  0.99191950125911
0 SKYNET 4B
1 19687U 88109A   24099.78901381 -.00000281  00000-0  00000-0 0  9990
2 19687  13.8602 342.7271 0002209 306.5859 301.2445  0.99670170110198
0 STTW-3
1 19710U 88111A   24099.80577601  .00000114  00000-0  00000+0 0  9990
2 19710  14.4748 351.6151 0006727 141.9395  32.1056  1.00256375122836
0 GORIZONT 17
1 19765U 89004A   24100.48487844 -.00000131  00000-0  00000+0 0  9998
2 19765  13.1658 338.7167 0022237 147.6597 349.2376  0.99064833127651
0 INTELSAT 515
1 19772U 89006A   24100.47656144 -.00000048  00000-0  00000+0 0  9995
2 19772  14.9862   0.1611 0023604  97.9083 113.8541  0.99104173116698
0 JCSAT 1
1 19874U 89020A   24099.80908286  .00000082  00000-0  00000-0 0  9993
2 19874  14.9594 359.1999 0004654 115.0352  69.5539  0.99531059120836
0 TELE-X
1 19919U 89027A   24099.47688086 -.00000187  00000-0  00000-0 0  9993
2 19919  15.0447 358.3654 0004067 130.3252  58.1992  0.99177734109279
0 RADUGA 23
1 19928U 89030A   24099.73996510 -.00000030  00000-0  00000-0 0  9997
2 19928  12.7357 337.4936 0020884 179.5913  14.4899  1.00180930126835
0 SUPERBIRD A
1 20040U 89041A   24099.22819671 -.00000076  00000-0  00000-0 0  9992
2 20040  13.1038 337.9225 0006379  73.4467 127.7631  0.99742810126489
0 DFS 1
1 20041U 89041B   24099.55411727 -.00000130  00000-0  00000+0 0  9999
2 20041  14.3720 352.6688 0011394 295.3759 261.3091  1.00651141116438
0 GORIZONT 18
1 20107U 89052A   24099.86754399 -.00000303  00000-0  00000-0 0  9996
2 20107  13.2442 339.9445 0034191 330.9207 320.5901  0.99401915126464
0 TVSAT 2
1 20168U 89062A   24100.00743990 -.00000190  00000-0  00000+0 0  9991
2 20168  14.7115   1.8280 0003398 142.4704  44.6046  0.99856853126514
0 SIRIUS W (MARCOPOLO 1)
1 20193U 89067A   24099.87742053  .00000019  00000-0  00000+0 0  9994
2 20193  14.6669   6.2720 0007747  39.0968 146.7359  0.99185153109237
0 USA 44
1 20203U 89069B   24099.79077088  .00000051  00000-0  00000+0 0  9994
2 20203  14.4086   9.9190 0011081  13.3261 164.1899  0.99115768 17054
0 FLTSATCOM 8 (USA 46)
1 20253U 89077A   24099.74844550 -.00000375  00000-0  00000-0 0  9990
2 20253  12.6566 355.9819 0006462  23.5788 212.8829  1.00282772252607
0 GORIZONT 19
1 20263U 89081A   24099.73683229  .00000078  00000-0  00000-0 0  9998
2 20263  12.9559 339.0593 0003665 225.1798 310.2327  1.00207267 39322
0 INTELSAT 602
1 20315U 89087A   24099.56067955 -.00000304  00000-0  00000-0 0  9998
2 20315  13.8770  13.1823 0004051 172.7993 356.7858  0.99009353123405
0 RADUGA 24
1 20367U 89098A   24099.85405117 -.00000353  00000-0  00000-0 0  9999
2 20367  13.3695 339.6018 0001711 191.1543  79.2152  1.00264166124954
0 COSMOS 2054
1 20391U 89101A   24100.43864346 -.00000180  00000-0  00000+0 0  9998
2 20391  13.0920 339.8733 0003675 135.2026 221.5743  1.00270086125857
0 LEASAT 5
1 20410U 90002B   24099.94283167 -.00000103  00000-0  00000+0 0  9996
2 20410  11.2830 356.2297 0005907  64.0689 124.3375  0.99599119110450
0 STTW-4
1 20473U 90011A   24099.90261594 -.00000312  00000-0  00000-0 0  9994
2 20473  14.5464 352.4799 0005708 126.5896 140.5225  1.00287845125022
0 RADUGA 25
1 20499U 90016A   24099.35005052  .00000098  00000-0  00000-0 0  9991
2 20499  13.1606 340.3666 0004984 165.6809  24.2752  1.00339820124882
0 INTELSAT 603
1 20523U 90021A   24099.89077192 -.00000008  00000-0  00000-0 0  9990
2 20523  13.5060  14.9495 0007017  55.5881 111.3156  0.99481800117453
0 ASIASAT 1
1 20558U 90030A   24099.19095906 -.00000257  00000-0  00000-0 0  9990
2 20558  14.8147   5.0154 0001874  15.5751 195.5389  0.99244086 38278
0 NEWSAT-1 (PALAPA B2R)
1 20570U 90034A   24098.94218631 -.00000053  00000-0  00000-0 0  9996
2 20570  14.4836   8.8799 0011261 119.0820  63.4706  0.99530170 38321
0 INSAT 1D
1 20643U 90051A   24099.89848000 -.00000180  00000-0  00000-0 0  9991
2 20643  14.6995   0.0606 0017368 164.5780  77.8889  1.00275151110730
0 GORIZONT 20
1 20659U 90054A   24099.89869537 -.00000127  00000-0  00000-0 0  9990
2 20659  13.4670 341.8139 0006602 193.3385  60.8388  1.00322526123673
0 COSMOS 2085
1 20693U 90061A   24099.72965900 -.00000084  00000-0  00000-0 0  9991
2 20693  13.4414 342.2004 0009498  39.0542 150.5856  1.00263949123489
0 DFS 2
1 20706U 90063B   24100.51625459 -.00000235  00000-0  00000+0 0  9997
2 20706  14.2810   9.5152 0003798 253.8124 285.0270  0.99554132114960
0 THOR 1 (MARCOPOLO 2)
1 20762U 90074A   24100.47876087 -.00000152  00000-0  00000+0 0  9999
2 20762  14.1592  12.4622 0000509 221.8948 318.6444  0.99211439122593
0 SKYNET 4C
1 20776U 90079A   24099.87628925  .00000118  00000-0  00000+0 0  9997
2 20776  13.5984 355.2793 0002860  18.5540 172.6161  1.00276280122923
0 EUTE 2-F1
1 20777U 90079B   24099.83993980 -.00000060  00000-0  00000-0 0  9998
2 20777  14.8269   3.9511 0002271 212.1251  11.3951  0.99342608102824
0 GALAXY 6
1 20873U 90091B   24099.90083252 -.00000034  00000-0  00000-0 0  9990
2 20873  13.4891  15.7364 0002388 189.1203  34.3937  0.99754122 38479
0 GORIZONT 21
1 20923U 90094A   24099.95882788  .00000016  00000-0  00000+0 0  9993
2 20923  13.4982 342.5570 0005190 220.0409 346.6648  1.00314514122420
0 SATCOM C1
1 20945U 90100A   24099.59209448 -.00000118  00000-0  00000+0 0  9995
2 20945  12.7168  20.7424 0015506  14.1802 202.8953  0.99187336119319
0 GSTAR 4
1 20946U 90100B   24099.60724006 -.00000293  00000-0  00000-0 0  9995
2 20946  13.7332  15.8099 0006094 341.8516 187.9227  0.99146349 38889
0 GORIZONT 22
1 20953U 90102A   24099.87071980 -.00000007  00000-0  00000-0 0  9993
2 20953  13.4673 342.9869 0002145  14.8351 157.8892  1.00251360122351
0 RADUGA 26
1 21016U 90112A   24099.73761549  .00000051  00000-0  00000-0 0  9995
2 21016  13.5545 343.6296 0006359 109.9758  64.5659  1.00251036122244
0 RADUGA 1-2
1 21038U 90116A   24099.84047778 -.00000144  00000-0  00000-0 0  9990
2 21038  13.5942 343.6800 0007537 121.5826 114.0415  1.00215307 39323
0 ITALSAT 1
1 21055U 91003A   24099.87385262  .00000046  00000-0  00000-0 0  9991
2 21055  14.7395 357.5800 0014653  44.2328 130.7211  0.99969854109097
0 COSMOS 2133
1 21111U 91010A   24099.89901876 -.00000169  00000-0  00000+0 0  9995
2 21111  13.7818 348.4445 0006306 146.8457 106.4155  1.00270461121410
0 RADUGA 27
1 21132U 91014A   24099.86754397 -.00000383  00000-0  00000-0 0  9992
2 21132  14.0881 342.9693 0002650 111.9400 177.4559  1.00229326121296
0 ANIK E2
1 21222U 91026A   24097.00789296 -.00000142  00000-0  00000-0 0  9998
2 21222  13.4428  18.0663 0007149  17.5037 170.7595  0.99130444109111
0 SPACENET 4 (ASC 2)
1 21227U 91028A   24099.90083222 -.00000001  00000-0  00000+0 0  9995
2 21227  13.0738  19.4844 0005140 326.5047 252.4920  0.99562513118425
0 AURORA 2
1 21392U 91037A   24099.40835203 -.00000072  00000-0  00000-0 0  9996
2 21392  14.3425  11.0915 0003281 257.3366 275.1968  0.99018478118814
0 TDRS 5
1 21639U 91054B   24100.14141709 -.00000302  00000-0  00000+0 0  9999
2 21639  14.1732 359.6168 0003513 354.8278 180.2070  1.00628367119707
0 INTELSAT 605
1 21653U 91055A   24099.16924359 -.00000255  00000-0  00000-0 0  9993
2 21653  12.7514  23.1552 0001191 115.2834  67.9448  0.99326548118984
0 COSMOS 2155
1 21702U 91064A   24099.87507093  .00000104  00000-0  00000-0 0  9990
2 21702  13.7875 345.1104 0000314  49.3445 145.4527  1.00190869119328
0 ANIK E1
1 21726U 91067A   24100.56812875 -.00000184  00000-0  00000+0 0  9993
2 21726  13.4796  18.2669 0009805 318.8762 243.4704  0.99178938109355
0 INTELSAT 601
1 21765U 91075A   24099.94805301  .00000066  00000-0  00000-0 0  9999
2 21765  12.5962  22.4714 0001910   7.2731 183.4318  0.99590086118418
0 COSMOS 2172
1 21789U 91079A   24099.84015503 -.00000173  00000-0  00000-0 0  9995
2 21789  13.8715 346.2156 0003349  39.3671 198.0785  1.00152513 39344
0 EUTE 2-F3
1 21803U 91083A   24099.74907587 -.00000321  00000-0  00000-0 0  9993
2 21803  14.5799   7.5587 0001572  63.4235 167.1937  0.99300065113620
0 USA 75
1 21805U 91080B   24099.22404439  .00000001  00000-0  00000-0 0  9997
2 21805  14.4385 351.2623 0006578  90.9367  89.2669  0.99183293 17771
0 RADUGA 28
1 21821U 91087A   24099.89978703 -.00000245  00000-0  00000-0 0  9996
2 21821  13.9640 346.9237 0006588 144.2742 117.0513  1.00359237118305
0 USA 78
1 21873U 92006A   24100.00743990 -.00000245  00000-0  00000+0 0  9998
2 21873  14.5703   8.5683 0003865 335.8644 203.1699  0.99042343 17268
0 SUPERBIRD B1
1 21893U 92010A   24100.51870266 -.00000255  00000-0  00000+0 0  9999
2 21893  14.1754  12.0905 0014618 105.5454  74.6358  0.99058167116419
0 GALAXY 5
1 21906U 92013A   24099.42688289  .00000063  00000-0  00000-0 0  9993
2 21906  12.6292  23.9294 0016136  72.7751 119.6384  0.99286153 38306
0 GORIZONT 25
1 21922U 92017A   24099.40791692  .00000055  00000-0  00000-0 0  9997
2 21922  14.0692 348.4001 0027859  97.4416  97.0933  0.99854355116953
0 TELECOM 2B
1 21939U 92021A   24099.87341087  .00000078  00000-0  00000+0 0  9994
2 21939  14.2530   9.5801 0016967  67.9238 133.4119  0.99509810109427
0 PALAPA B4
1 21964U 92027A   24099.31091042 -.00000164  00000-0  00000-0 0  9998
2 21964  12.4356  22.8557 0003933  28.6380 168.6171  0.99931996 39196
0 INSAT 2A
1 22027U 92041A   24099.87524498  .00000102  00000-0  00000-0 0  9993
2 22027  14.6662 357.9445 0005834 263.1348 279.2456  1.00180289116040
0 EUTE 2-F4
1 22028U 92041B   24100.14280946 -.00000172  00000-0  00000+0 0  9991
2 22028  14.3751  11.2925 0005045 235.6110 295.3705  0.99076181108314
0 GORIZONT 26
1 22041U 92043A   24099.30568262 -.00000002  00000-0  00000-0 0  9996
2 22041  14.2942 350.4965 0031872 250.6386 272.6395  0.99244078115538
0 OPTUS B1 (AUSSAT B1)
1 22087U 92054A   24099.58523000 -.00000155  00000-0  00000-0 0  9998
2 22087  11.8899  29.6871 0009262  86.1811 112.9553  0.99207788126773
0 SATCOM C4
1 22096U 92057A   24099.90164737  .00000000  00000-0  00000-0 0  9993
2 22096  12.9784  21.7898 0002689 359.1503 223.6659  0.99014129116992
0 COSMOS 2209
1 22112U 92059A   24100.34270031  .00000067  00000-0  00000+0 0  9992
2 22112  14.1255 348.5885 0006687 326.7655 224.7002  1.00362795115647
0 HISPASAT 1A
1 22116U 92060A   24099.41267396  .00000058  00000-0  00000-0 0  9994
2 22116  13.6445  15.9787 0004291 337.9451 213.6863  0.99315180111550
0 HELLAS-SAT 1 (DFS 3)
1 22175U 92066A   24099.85405101 -.00000275  00000-0  00000-0 0  9993
2 22175  13.8192  12.8462 0002723  91.1883 147.7042  0.99728118112576
0 GALAXY 7
1 22205U 92072A   24099.07296692 -.00000297  00000-0  00000-0 0  9995
2 22205  14.2358   9.1744 0008437  38.1072 128.5409  0.99705382108904
0 EKRAN 20
1 22210U 92074A   24099.87837753  .00000026  00000-0  00000+0 0  9998
2 22210  14.1632 350.1855 0005292 157.8286  63.2120  1.00310671115154
0 GORIZONT 27
1 22245U 92082A   24100.27517080 -.00000008  00000-0  00000+0 0  9991
2 22245  14.2003 349.9212 0011529  21.5784 164.6216  1.00148027119147
0 SUPERBIRD A1
1 22253U 92084A   24100.08024024 -.00000258  00000-0  00000+0 0  9999
2 22253  11.5963  27.5843 0012352 146.0163  31.5476  0.99121802113883
0 COSMOS 2224
1 22269U 92088A   24099.94309272  .00000021  00000-0  00000-0 0  9991
2 22269  13.9749 353.7218 0003803 233.4979 319.2952  1.00229867114075
0 TDRS 6
1 22314U 93003B   24099.79529750 -.00000303  00000-0  00000-0 0  9992
2 22314  14.1620   2.9226 0008282 143.6766 291.3943  1.00270896114394
0 RADUGA 29
1 22557U 93013A   24100.58029433 -.00000319  00000-0  00000+0 0  9991
2 22557  14.2763 351.0701 0006552 141.0638  15.8695  1.00174817113686
0 UFO 1 (USA 98)
1 22563U 93015A   24099.55749660  .00000100  00000-0  00000-0 0  9993
2 22563  23.8666  84.9186 0006020 265.8478 277.6141  0.99250227116760
0 HGS 4 (GALAXY 4)
1 22694U 93039A   24099.89869556 -.00000116  00000-0  00000+0 0  9995
2 22694  14.6120   1.1948 0013791  36.4005 198.5458  1.00264561110985
0 USA 93
1 22719U 93046A   24099.90498291 -.00000071  00000-0  00000-0 0  9998
2 22719  13.2105  19.7665 0003595  68.0831 170.0845  0.99047460 17278
0 HISPASAT 1B
1 22723U 93048A   24099.90083249 -.00000029  00000-0  00000-0 0  9991
2 22723  13.0720  18.7254 0007709   6.2853 214.4937  0.99716121111868
0 INSAT 2B
1 22724U 93048B   24099.80882179  .00000088  00000-0  00000+0 0  9993
2 22724  14.2004   8.5158 0014501 196.9276 336.5753  0.99966088112593
0 UFO 2 (USA 95)
1 22787U 93056A   24099.97222951  .00000118  00000-0  00000+0 0  9999
2 22787  11.6380   7.7239 0005470  22.0853 186.5701  1.00271916110648
0 ACTS
1 22796U 93058B   24099.65103972 -.00000117  00000-0  00000-0 0  9998
2 22796  14.6009   1.6538 0050200  63.3275 260.8887  1.00273513118881
0 RADUGA 30
1 22836U 93062A   24099.89848007 -.00000176  00000-0  00000-0 0  9994
2 22836  14.3787 353.0948 0001421 319.0743 289.9840  1.00270759 37988
0 INTELSAT 701
1 22871U 93066A   24099.65311656 -.00000208  00000-0  00000-0 0  9992
2 22871   9.1455  52.9405 0003984  12.3430 357.9966  0.99323007111166
0 GORIZONT 28
1 22880U 93069A   24100.01018709 -.00000223  00000-0  00000+0 0  9997
2 22880  14.4491 353.8880 0033488 148.8517  45.2206  0.99658515111178
0 GORIZONT 29
1 22907U 93072A   24099.83993973 -.00000192  00000-0  00000-0 0  9999
2 22907  14.3450 353.2724 0009752 356.0620 237.2347  1.00111739111161
0 SOLIDARIDAD 1
1 22911U 93073A   24099.33737682 -.00000141  00000-0  00000-0 0  9997
2 22911  14.1386   8.3244 0003328 226.3093 342.1397  1.00275822109840
0 TELSTAR 401
1 22927U 93077A   24100.25674361 -.00000146  00000-0  00000+0 0  9990
2 22927  14.6660 356.9605 0010825 113.8543  78.0004  1.00285733110552
0 THAICOM 1
1 22931U 93078B   24099.78295427 -.00000128  00000-0  00000-0 0  9994
2 22931  10.5316  40.6438 0002460 179.8157 182.9121  0.99181412 38246
0 GALS 1
1 22963U 94002A   24099.84037035 -.00000203  00000-0  00000-0 0  9997
2 22963  14.6258 359.9245 0009776 201.1925  21.0500  1.00357199110696
0 RADUGA 1-3
1 22981U 94008A   24099.84015509 -.00000217  00000-0  00000-0 0  9990
2 22981  14.4909 354.2237 0002864 124.6474 105.3936  1.00326900110465
0 USA 99
1 22988U 94009A   24100.37479662 -.00000202  00000-0  00000+0 0  9996
2 22988  16.4047  36.1703 0002574 325.3870 241.4779  1.00271040 23632
0 RADUGA 31
1 23010U 94012A   24099.80847372  .00000070  00000-0  00000-0 0  9992
2 23010  14.4556 354.2293 0004407 119.6782  66.7621  1.00228727110335
0 GALAXY 1R
1 23016U 94013A   24099.90387721 -.00000012  00000-0  00000-0 0  9994
2 23016  11.9350  27.8867 0001510 354.2786 224.3915  0.99217899 38251
0 GORIZONT 30
1 23108U 94030A   24099.91229951 -.00000001  00000-0  00000-0 0  9994
2 23108  14.3834 354.1755 0001843 220.2871 317.7455  1.00235684109446
0 UFO 3 (USA 104)
1 23132U 94035A   24100.55152339 -.00000301  00000-0  00000+0 0  9996
2 23132  12.0244  10.2948 0000370  55.7413 111.6770  1.00248124123680
0 COSMOS 2282
1 23168U 94038A   24100.03590980 -.00000229  00000-0  00000+0 0  9998
2 23168  14.0171 358.4384 0008974 104.4121  80.5830  1.00284588108988
0 INTELSAT 2 (PAS 2)
1 23175U 94040A   24099.68392255 -.00000313  00000-0  00000-0 0  9997
2 23175  10.8388  40.6416 0005792  14.7568 173.9985  0.99196307107715
0 BS-3N
1 23176U 94040B   24100.28300282 -.00000102  00000-0  00000+0 0  9991
2 23176  12.4718  25.5261 0005455  25.4014 167.3106  0.99149210108266
0 APSTAR 1
1 23185U 94043A   24099.01074578 -.00000166  00000-0  00000-0 0  9999
2 23185  12.4977  23.3849 0005663  32.2468 145.0570  0.99380482 37123
0 BRAZILSAT B1
1 23199U 94049A   24099.14711017 -.00000270  00000-0  00000-0 0  9996
2 23199  11.5700  32.2935 0002692 136.1351  47.4663  0.99251163 38268
0 TURKSAT 1B
1 23200U 94049B   24099.07679591 -.00000262  00000-0  00000-0 0  9992
2 23200  12.9199  22.1033 0017130  85.0969  89.5265  0.99049853107370
0 COSMOS 2291
1 23267U 94060A   24099.47362786  .00000083  00000-0  00000-0 0  9997
2 23267  14.4168 355.6410 0006004 183.7703  41.8119  1.00359408108170
0 NSS 703 (INTELSAT 703)
1 23305U 94064A   24099.85793924 -.00000182  00000-0  00000-0 0  9999
2 23305  10.2809  42.8223 0005241 265.9446 304.2720  0.99302200107791
0 THAICOM 2
1 23314U 94065B   24100.52679472 -.00000007  00000-0  00000+0 0  9993
2 23314   9.9904  44.6948 0000793  89.1946 147.4448  0.99586017107686
0 EXPRESS 1
1 23319U 94067A   24099.55240950 -.00000298  00000-0  00000-0 0  9998
2 23319  14.2916   7.9299 0003251 209.4631 328.5805  0.99834279107599
0 ELEKTRO (GOMS)
1 23327U 94069A   24099.72410483 -.00000145  00000-0  00000-0 0  9999
2 23327  14.7531 355.4334 0006058 225.7410 314.3951  1.00288285107858
0 DFH-3 1
1 23415U 94080A   24100.42538669  .00000077  00000-0  00000+0 0  9999
2 23415  14.1355 350.6475 0095133   3.2308 195.4250  1.00961612127634
0 LUCH
1 23426U 94082A   24099.49123504 -.00000138  00000-0  00000-0 0  9995
2 23426  14.8120   0.8129 0005797 161.3974 198.5891  1.00274153107242
0 USA 107
1 23435U 94084A   24100.07475782 -.00000243  00000-0  00000+0 0  9998
2 23435  14.5506   0.7770 0003805 260.4947 274.4301  0.99077740 23637
0 RADUGA 32
1 23448U 94087A   24099.89848034 -.00000168  00000-0  00000-0 0  9993
2 23448  14.5188 356.9736 0005769 224.4147  19.8798  1.00272471124867
0 INTELSAT 704
1 23461U 95001A   24099.14798039 -.00000264  00000-0  00000+0 0  9998
2 23461  10.4479  41.5923 0008686 125.2424  53.5100  0.99183715106537
0 UFO 4 (USA 108)
1 23467U 95003A   24099.57149594 -.00000029  00000-0  00000+0 0  9998
2 23467   9.7433  15.6877 0002291  32.8638 166.8130  1.00270730106921
0 HIMAWARI 5 (GMS 5)
1 23522U 95011B   24099.94378890 -.00000094  00000-0  00000-0 0  9996
2 23522  13.9782  10.1173 0008428 120.7440  58.6589  0.99407739105655
0 BRAZILSAT B2
1 23536U 95016A   24099.58186293 -.00000210  00000-0  00000-0 0  9990
2 23536  10.8282  37.2669 0000166 152.9147 209.2958  0.99241120106164
0 HOT BIRD 1
1 23537U 95016B   24099.14545674 -.00000270  00000-0  00000-0 0  9991
2 23537  12.0894  28.3056 0005157  53.4741 125.1668  0.99317477105767
0 AMSC 1
1 23553U 95019A   24100.41967234 -.00000101  00000-0  00000+0 0  9997
2 23553  12.5683  21.8214 0005849 345.4250 234.5183  1.00272839106153
0 INTELSAT 706
1 23571U 95023A   24099.92006159  .00000027  00000-0  00000+0 0  9991
2 23571   9.3082  51.3311 0002563 319.2758  51.4371  0.99048532105374
0 DIRECTV 3 (DBS/NIMIQ 3)
1 23598U 95029A   24100.14759571 -.00000238  00000-0  00000+0 0  9991
2 23598  10.5995  41.0985 0002523 313.4131 213.7692  0.99003699104845
0 TDRS 7
1 23613U 95035B   24099.84026267 -.00000211  00000-0  00000-0 0  9991
2 23613  13.7115 353.7195 0027713  62.4905 168.7514  1.00274522105230
0 KOREASAT 1
1 23639U 95041A   24099.39456366 -.00000090  00000-0  00000-0 0  9998
2 23639  14.6233   4.6050 0001252 359.5176 238.7644  0.99664206104556
0 JCSAT 3
1 23649U 95043A   24099.01292133 -.00000123  00000-0  00000-0 0  9997
2 23649  12.1278  22.6725 0016640  52.7663 137.4731  0.99149294 61726
0 NSTAR A
1 23651U 95044A   24100.34940105  .00000016  00000-0  00000+0 0  9998
2 23651  12.2550  26.8603 0020095   0.4262 189.1401  0.99125675104046
0 COSMOS 2319
1 23653U 95045A   24099.94352784  .00000047  00000-0  00000+0 0  9998
2 23653  14.4947 358.7588 0004354 204.8243 345.2763  1.00320013104761
0 TELSTAR 4 (TELSTAR 402R)
1 23670U 95049A   24099.33083722 -.00000190  00000-0  00000-0 0  9998
2 23670  12.9511  18.3508 0003732 347.9179 217.5389  1.00254380104484
0 LUCH 1
1 23680U 95054A   24099.89869565 -.00000114  00000-0  00000-0 0  9999
2 23680  14.9110   5.5222 0008718 114.8529 115.4914  1.00266908104335
0 USA 115
1 23712U 95060A   24100.27551888 -.00000027  00000-0  00000+0 0  9995
2 23712  13.7770   9.5207 0002506 353.4061 174.2194  1.00270497 23632
0 GALS 2
1 23717U 95063A   24099.93590944  .00000098  00000-0  00000+0 0  9996
2 23717  14.2323   8.5125 0005854 334.6935 226.5680  0.99885071103687
0 AMOS 5I (ASIASAT 2)
1 23723U 95064A   24099.85235806 -.00000078  00000-0  00000-0 0  9998
2 23723   9.7224  47.9817 0010231 321.1370  47.7846  0.99303735103394
0 INSAT 2C
1 23731U 95067B   24097.03851578 -.00000109  00000-0  00000-0 0  9993
2 23731  13.8323  14.1465 0005019  46.8398 148.5128  0.99789609103406
0 GALAXY 3R
1 23741U 95069A   24099.28590939 -.00000257  00000-0  00000-0 0  9990
2 23741  12.5250  21.8571 0005150  56.9560 141.4955  1.00303770103790
0 INTELSAT 3R (PAS 3R)
1 23764U 96002A   24100.49660848  .00000044  00000-0  00000+0 0  9998
2 23764  10.3099  42.7500 0012052 272.4019 283.7096  0.99087499102888
0 AFRICASAT 1 (MEASAT 1)
1 23765U 96002B   24099.33452382 -.00000106  00000-0  00000-0 0  9992
2 23765  11.2100  34.9197 0004424 336.3990 228.0026  0.99061061102926
0 ABS 1A (KOREASAT 2)
1 23768U 96003A   24099.27999368 -.00000159  00000-0  00000-0 0  9997
2 23768  11.5150  31.1717 0003618  64.3138 116.7444  0.99789099103195
0 GORIZONT 31
1 23775U 96005A   24100.50710524 -.00000257  00000-0  00000+0 0  9999
2 23775  14.4739   0.5040 0019956 218.8570 316.1678  0.99901932103068
0 PAKSAT 1 (ANATOLIA 1)
1 23779U 96006A   24099.85166187 -.00000091  00000-0  00000-0 0  9996
2 23779   9.1912  52.5911 0002764 150.1714 210.3667  0.99588946102896
0 NSTAR B
1 23781U 96007A   24099.94492020 -.00000069  00000-0  00000-0 0  9993
2 23781  11.6741  31.4318 0009132 297.5442 226.6554  0.99262191102519
0 INMARSAT 3-F1
1 23839U 96020A   24099.65416083 -.00000095  00000-0  00000-0 0  9994
2 23839   8.9220  53.4308 0004032 342.6869  32.9868  1.00001354102269
0 ASTRA 1F
1 23842U 96021A   24100.52380993 -.00000200  00000-0  00000+0 0  9995
2 23842   3.3549  83.3008 0001774 332.8136 143.5909  0.99121262102176
0 PALAPA C2
1 23864U 96030A   24100.14968425 -.00000252  00000-0  00000+0 0  9996
2 23864  10.1724  44.6450 0005801  67.9416 106.9381  0.99003758101821
0 GALAXY 9
1 23877U 96033A   24100.35314302 -.00000110  00000-0  00000+0 0  9998
2 23877  10.7674  38.2537 0007556 136.9540  62.0638  0.99536743101747
0 GORIZONT 32
1 23880U 96034A   24098.87326442  .00000106  00000-0  00000-0 0  9999
2 23880  14.4366   0.9835 0007313   0.6947 178.1955  1.00172137102014
0 INTELSAT 709
1 23915U 96035A   24099.59501177 -.00000080  00000-0  00000+0 0  9990
2 23915   9.0192  54.4804 0013340 220.9467 327.1414  0.99182750101453
0 APSTAR 1A
1 23943U 96039A   24100.08215475 -.00000233  00000-0  00000+0 0  9995
2 23943  11.7647  25.0818 0021219 336.6262 214.1697  0.99194511101298
0 ARABSAT 2A
1 23948U 96040A   24099.22814878 -.00000260  00000-0  00000-0 0  9998
2 23948  13.7543  14.7320 0003929  85.9292 342.9611  0.99232433100929
0 TURKSAT 1C
1 23949U 96040B   24099.86610039 -.00000377  00000-0  00000-0 0  9991
2 23949  10.8241  35.9310 0002675 120.3274 113.4790  1.00219809101664
0 UFO 7 (USA 127)
1 23967U 96042A   24099.44864000  .00000090  00000-0  00000-0 0  9997
2 23967  10.3644  20.9532 0007439  77.0244 116.1010  1.00358856101463
0 ITALSAT 2
1 24208U 96044A   24100.39737531 -.00000188  00000-0  00000+0 0  9991
2 24208  13.3875  12.6295 0025315 125.1105  98.1992  1.00790179101599
0 INMARSAT 3-F2
1 24307U 96053A   24099.91710282  .00000044  00000-0  00000-0 0  9991
2 24307   8.0299  61.5562 0007064 341.3932   2.7250  0.99980856100925
0 ECHOSTAR 2
1 24313U 96055A   24099.71385833 -.00000294  00000-0  00000-0 0  9996
2 24313  10.6884  37.4052 0014868  62.8808 290.0702  1.00241303100995
0 AMC-1 (GE-1)
1 24315U 96054A   24099.89029987 -.00000189  00000-0  00000+0 0  9993
2 24315   7.2019  67.2035 0006571 236.7489 317.8137  0.99503222100987
0 EXPRESS 2
1 24435U 96058A   24099.90476777 -.00000327  00000-0  00000-0 0  9990
2 24435  14.6430   0.4306 0009143  10.7012 251.1735  1.00304109100798
0 ARABSAT 2B
1 24652U 96063A   24099.51833642  .00000050  00000-0  00000-0 0  9994
2 24652   8.8484  55.7969 0001002  17.1693 343.8379  0.99412234100512
0 MEASAT 2
1 24653U 96063B   24099.58533940 -.00000076  00000-0  00000-0 0  9999
2 24653  11.0558  34.6114 0003424 148.4034  46.1514  0.99661085100318
0 INMARSAT 3-F3
1 24674U 96070A   24100.50303723 -.00000363  00000-0  00000+0 0  9997
2 24674   8.5757  56.7699 0004450 320.3119 130.4570  0.99999171 99913
0 AMC-2 (GE-2)
1 24713U 97002A   24099.65433486 -.00000164  00000-0  00000-0 0  9990
2 24713   9.1603  51.7796 0003114  62.5817 315.6750  0.99473377 99476
0 NAHUEL 1A
1 24714U 97002B   24099.90676659 -.00000075  00000-0  00000-0 0  9991
2 24714  11.3034  34.1002 0005017 161.8544  58.5671  0.99382299 99106
0 USA 130
1 24737U 97008A   24099.86765156 -.00000232  00000-0  00000+0 0  9992
2 24737  14.1400   9.4574 0008634 308.9316 311.3217  0.99029392 23450
0 DIRECTV 6 (TEMPO 2)
1 24748U 97011A   24100.34896594  .00000024  00000-0  00000+0 0  9993
2 24748  12.1257  28.2929 0026647 306.0936 239.6960  0.99028533 98493
0 BSAT-1A
1 24769U 97016B   24099.92014861  .00000023  00000-0  00000-0 0  9991
2 24769   9.6038  46.9860 0003107 305.0411  70.5669  0.99118023 38887
0 GOES 10
1 24786U 97019A   24099.90395417 -.00000023  00000-0  00000-0 0  9991
2 24786  12.1830  26.5487 0027993 217.4982   5.3356  0.99120512 98142
0 DFH-3 2
1 24798U 97021A   24099.88382674  .00000079  00000-0  00000+0 0  9994
2 24798  12.8294  19.4780 0009330  80.0835 108.9122  1.00181448 98575
0 GALAXY 25 (TELSTAR 5)
1 24812U 97026A   24099.95597206  .00000057  00000-0  00000-0 0  9990
2 24812   5.3642  76.8318 0011641 316.2487 222.6639  0.99089619 98307
0 INMARSAT 3-F4
1 24819U 97027A   24100.48615442 -.00000215  00000-0  00000+0 0  9998
2 24819  10.0622  43.7368 0012192  27.8663  47.8041  0.99496026 98124
0 SUPERBIRD C
1 24880U 97036A   24099.87093075  .00000035  00000-0  00000+0 0  9997
2 24880  10.7107  39.6619 0007452 350.5253   9.4171  0.99068720 97363
0 ABS 3 (AGILA 2)
1 24901U 97042A   24100.53565288 -.00000111  00000-0  00000+0 0  9990
2 24901   9.5959  49.1747 0001545 342.4722 184.4443  0.99263627 97284
0 INTELSAT 5 (PAS 5)
1 24916U 97046A   24099.68801260 -.00000068  00000-0  00000-0 0  9998
2 24916   8.6530  56.0362 0001644 286.2106 269.0034  1.00272405 97605
0 AMC-3 (GE-3)
1 24936U 97050A   24100.42811372 -.00000265  00000-0  00000+0 0  9993
2 24936   6.1776  72.8802 0002950 315.1329 252.2570  1.00271494 97434
0 NSS 5 (INTELSAT 803)
1 24957U 97053A   24100.12410531 -.00000072  00000-0  00000+0 0  9994
2 24957   8.8051  54.6343 0002187 241.4791 325.5585  0.99104432 97126
0 APSTAR 2R (TELSTAR 10)
1 25010U 97062A   24099.78784937 -.00000230  00000-0  00000+0 0  9994
2 25010   9.0041  54.6070 0010510 286.5254  88.1640  0.99211428 96542
0 USA 134
1 25019U 97065A   24099.60311490 -.00000067  00000-0  00000-0 0  9996
2 25019  11.8283  27.3048 0003970 332.9609 301.9005  1.00275105 24934
0 KUPON
1 25045U 97070A   24099.19936785 -.00000269  00000-0  00000-0 0  9999
2 25045  14.6154   0.6933 0001503 282.3007  77.6491  1.00262594 96627
0 SIRIUS 2 (GE-1E)
1 25049U 97071A   24100.53931358 -.00000057  00000-0  00000+0 0  9990
2 25049  10.6519  39.8151 0004135  18.5878 163.9682  0.99407534 96260
0 INDOSTAR 1
1 25050U 97071B   24099.88246785 -.00000031  00000-0  00000+0 0  9992
2 25050  12.0582  25.6300 0002319 352.5844 201.5320  1.00359292 96744
0 ASTRA 1G
1 25071U 97076A   24100.02723024 -.00000245  00000-0  00000+0 0  9992
2 25071   7.6498  62.0709 0004629 121.9269 149.1731  0.99038060 96484
0 GALAXY 8
1 25086U 97078A   24099.89891108 -.00000030  00000-0  00000-0 0  9994
2 25086  13.5168  15.6205 0009465  38.1180 185.4249  0.99694037 95971
0 INTELSAT 804
1 25110U 97083A   24099.07871042 -.00000169  00000-0  00000-0 0  9996
2 25110  12.3362  23.2483 0007544  22.8842 161.2452  1.00231449 96310
0 HGS 1 (ASIASAT 3)
1 25126U 97086A   24100.55924483  .00000107  00000-0  00000+0 0  9990
2 25126  11.8669  38.5514 0052335  28.0573 184.2053  1.00201517 97138
0 SKYNET 4D
1 25134U 98002A   24099.37477442  .00000035  00000-0  00000-0 0  9994
2 25134  12.2652  15.5033 0004646 118.8062  83.2991  0.99150005 95441
0 BRAZILSAT B3
1 25152U 98006A   24100.15534072 -.00000188  00000-0  00000+0 0  9999
2 25152   9.0872  54.4883 0003027  12.4344 183.2481  0.99220646 95699
0 INMARSAT 3-F5
1 25153U 98006B   24100.36165299 -.00000295  00000-0  00000+0 0  9993
2 25153   7.8913  60.6635 0004758 317.5832 256.0046  1.00267962 95870
0 NSS 806 (INTELSAT 806)
1 25239U 98014A   24099.92803785 -.00000149  00000-0  00000-0 0  9994
2 25239   5.0781  79.1980 0005160  29.8335 356.1123  0.99076967 39051
0 UFO 8 (USA 138)
1 25258U 98016A   24100.08093642 -.00000155  00000-0  00000+0 0  9996
2 25258  10.1705  27.0491 0002731 251.9747 290.1250  1.00308378 95563
0 BSAT-1B
1 25312U 98024B   24099.92410598  .00000075  00000-0  00000-0 0  9990
2 25312   9.8353  51.4640 0002548 276.3950 262.7053  0.99116740 38232
0 COSMOS 2350
1 25315U 98025A   24099.89880333 -.00000118  00000-0  00000-0 0  9997
2 25315  13.6695   9.5222 0006974 134.7437  91.7962  1.00286708 94990
0 INTELSAT 805
1 25371U 98037A   24099.59143542  .00000096  00000-0  00000+0 0  9996
2 25371   5.8819  74.6993 0011138 130.3820 245.7144  0.99474063 94428
0 CHINASAT 5B (SINOSAT 1)
1 25404U 98044A   24100.53482446 -.00000022  00000-0  00000+0 0  9997
2 25404   9.3019  51.8751 0004812 257.9245 344.9135  0.99194694 93973
0 ASTRA 2A
1 25462U 98050A   24100.52540498  .00000063  00000-0  00000+0 0  9993
2 25462   5.0739  78.2259 0003242 293.4138  72.9369  1.00272768 93730
0 EUTE W2
1 25491U 98056A   24100.52035767  .00000018  00000-0  00000+0 0  9999
2 25491  10.1743  44.3819 0001981 145.3368 228.0671  0.99261298 92943
0 SIRIUS 3
1 25492U 98056B   24099.94883622  .00000010  00000-0  00000-0 0  9997
2 25492  10.4137  40.0035 0004133  32.1210 144.6861  0.99034331 93208
0 AFRISTAR
1 25515U 98063A   24099.58856368 -.00000043  00000-0  00000-0 0  9995
2 25515   8.4655  56.5326 0002730  14.3683   4.5373  0.99007973 92649
0 AMC-5 (GE-5)
1 25516U 98063B   24099.85288020 -.00000098  00000-0  00000-0 0  9990
2 25516   9.9003  46.4322 0004975   4.1948   9.3907  0.99330689 92897
0 BONUM 1
1 25546U 98068A   24099.58612704 -.00000110  00000-0  00000-0 0  9993
2 25546   9.1980  53.1130 0005536 355.3663  13.6883  0.99039728 92629
0 INTELSAT 6B (PAS 6B)
1 25585U 98075A   24099.88386021  .00000051  00000-0  00000-0 0  9990
2 25585  11.0681  36.5093 0014516 142.8414  47.5198  0.99150274 92069
0 GALAXY 26 (TELSTAR 6)
1 25626U 99005A   24100.13521682 -.00000191  00000-0  00000+0 0  9990
2 25626   9.9318  54.2185 0002424  33.3289 157.3025  0.99000228 91663
0 ARABSAT 3A
1 25638U 99009A   24100.53307600  .00000029  00000-0  00000+0 0  9992
2 25638  10.6314  38.3837 0006983 344.0533 190.0010  1.00171517 91993
0 SKYNET 4E
1 25639U 99009B   24097.03313501 -.00000038  00000-0  00000-0 0  9992
2 25639  11.5945  13.4563 0003262 357.5125 194.5095  1.00272843 43349
0 RADUGA 1-4
1 25642U 99010A   24099.89858770 -.00000124  00000-0  00000-0 0  9992
2 25642  15.4115   8.2357 0002043 254.8987 335.5007  1.00189744 91934
0 INSAT 2E (APR-1)
1 25666U 99016A   24099.65268144 -.00000197  00000-0  00000-0 0  9998
2 25666   9.8178  46.5850 0008378 291.3192  82.8565  0.99661580 43477
0 TELKOM 1
1 25880U 99042A   24099.95684229 -.00000118  00000-0  00000-0 0  9997
2 25880   5.6884  75.3400 0015313 239.4615 305.5487  1.00241560 90378
0 ABS 7 (KOREASAT 3)
1 25894U 99046A   24099.91919137 -.00000009  00000-0  00000-0 0  9990
2 25894   4.4436  80.2644 0003031 327.5967   9.4339  0.99954923 89084
0 YAMAL 101
1 25896U 99047A   24099.29709616 -.00000133  00000-0  00000-0 0  9991
2 25896  14.4862   5.4838 0088276  44.9918 160.1151  0.99858622 89811
0 YAMAL 102
1 25897U 99047B   24099.01126792 -.00000059  00000-0  00000-0 0  9997
2 25897  13.0556  18.3759 0004637 358.9462 186.4618  0.99997924 89990
0 GALAXY 27 (TELSTAR 7)
1 25922U 99052A   24099.69354841 -.00000247  00000-0  00000-0 0  9993
2 25922   8.4593  58.4625 0015896 322.8480  33.5411  0.99021031 43332
0 ABS 6 (LMI 1)
1 25924U 99053A   24100.51120076 -.00000127  00000-0  00000+0 0  9997
2 25924   0.0586  81.3460 0002755 311.2052 148.7096  1.00271646 89827
0 AMC-4 (GE-4)
1 25954U 99060A   24099.92864699 -.00000170  00000-0  00000-0 0  9995
2 25954   3.1845  83.0557 0001244 276.5484 108.8166  0.99132590 35492
0 UFO 10 (USA 146)
1 25967U 99063A   24099.65041885 -.00000193  00000-0  00000-0 0  9991
2 25967   9.4459  30.9391 0002135 310.8814  66.8935  1.00270575 89330
0 GALAXY 11
1 26038U 99071A   24100.11757293 -.00000060  00000-0  00000+0 0  9993
2 26038   4.6812  80.1120 0000964 227.3333  98.9959  1.00271751 89062
0 USA 148
1 26052U 00001A   24099.58544753  .00000031  00000-0  00000+0 0  9994
2 26052  10.5996  38.2024 0001751 301.8572 248.1242  1.00271835 23607
0 GALAXY 10R
1 26056U 00002A   24099.90112456  .00000015  00000-0  00000-0 0  9999
2 26056  10.9014  36.9789 0003372 106.5230  92.5738  0.99634666 88368
0 HISPASAT 1C
1 26071U 00007A   24099.72612851 -.00000175  00000-0  00000-0 0  9993
2 26071   6.4063  72.2311 0002645 233.6218 150.9736  0.99205648 39360
0 GARUDA 1
1 26089U 00011A   24099.95736442 -.00000069  00000-0  00000-0 0  9997
2 26089   7.6734  73.6823 0003074 319.5825 230.0058  0.99901869 33678
0 ASIASTAR
1 26107U 00016A   24100.52435023 -.00000338  00000-0  00000+0 0  9998
2 26107   7.2454  66.1969 0004333 318.2210 107.5849  1.00268785 88149
0 INSAT 3B
1 26108U 00016B   24099.68801260 -.00000186  00000-0  00000-0 0  9995
2 26108   9.1974  52.2069 0006327 238.8754 320.4758  0.99666326 43461
0 GALAXY 4R
1 26298U 00020A   24100.14655144 -.00000297  00000-0  00000+0 0  9994
2 26298  11.7048  29.6702 0002006 196.4730 336.2008  0.99838129 87684
0 GOES 11
1 26352U 00022A   24099.86364207 -.00000215  00000-0  00000-0 0  9991
2 26352  10.0979  53.8207 0002544 260.1533 313.6638  0.99042609 44553
0 USA 149
1 26356U 00024A   24098.49077362  .00000094  00000-0  00000-0 0  9992
2 26356  12.5668  17.2564 0001132 270.8030 280.2807  1.00288915 24887
0 GORIZONT 33
1 26372U 00029A   24098.91276990  .00000109  00000-0  00000-0 0  9997
2 26372  13.4717  12.5273 0001325 166.1750  27.1267  1.00370830 87408
0 FENGYUN 2B
1 26382U 00032A   24099.62202319 -.00000289  00000-0  00000-0 0  9995
2 26382  12.4826  22.5074 0001255 337.1713 206.8760  1.00105288 86923
0 TDRS 8
1 26388U 00034A   24100.05117472  .00000091  00000-0  00000+0 0  9997
2 26388  11.4333  37.7963 0003664 287.2752  83.6190  1.00278715 87161
0 COSMOS 2371
1 26394U 00036A   24098.84285541 -.00000073  00000-0  00000-0 0  9994
2 26394  13.5534  13.2675 0003718 117.2691  80.1368  1.00267807 87010
0 INTELSAT 9 (PAS 9)
1 26451U 00043A   24100.22374045 -.00000147  00000-0  00000+0 0  9991
2 26451   8.6389  56.2275 0003085 297.2858 265.4322  1.00373864 86839
0 BRAZILSAT B4
1 26469U 00046A   24099.82442382 -.00000289  00000-0  00000-0 0  9994
2 26469   7.3409  66.9924 0003835 228.1695 331.5804  0.99166972 86838
0 RADUGA 1-5
1 26477U 00049A   24099.89891097 -.00000131  00000-0  00000-0 0  9990
2 26477  13.5069  13.8270 0004242  73.9314 151.6860  1.00204780 86409
0 AMC-7 (GE-7)
1 26495U 00054B   24099.45174713  .00000087  00000-0  00000-0 0  9996
2 26495   4.8653  79.7440 0001240 122.8662 215.3415  0.99130866 86039
0 NSS 11 (AAP-1)
1 26554U 00059A   24099.83312608  .00000017  00000-0  00000-0 0  9999
2 26554   0.0605  70.8221 0002796 308.7568 293.9022  1.00271446 33273
0 NSAT 110
1 26559U 00060A   24099.82703449 -.00000302  00000-0  00000+0 0  9999
2 26559   4.7485  79.5380 0007084   0.6227 200.5251  0.99183908 85886
0 USA 153
1 26575U 00065A   24099.90917632 -.00000331  00000-0  00000-0 0  9995
2 26575  10.2366  41.4833 0001538 331.8971 285.9025  1.00266661 24912
0 AMC-6 (GE-6)
1 26580U 00067A   24100.12845074  .00000097  00000-0  00000+0 0  9997
2 26580   0.0641  73.9493 0002980 312.7667  78.3698  1.00272304 85979
0 INTELSAT 12 (PAS 12)
1 26590U 00068A   24099.72412699 -.00000220  00000-0  00000-0 0  9995
2 26590   6.3390  73.1479 0010586 202.5690 170.3843  0.99030607 43580
0 INTELSAT 1R (PAS 1R)
1 26608U 00072A   24099.75780473 -.00000143  00000-0  00000+0 0  9994
2 26608   6.0879  73.3861 0001461 326.3108 227.7204  1.00272508 85638
0 ANIK F1
1 26624U 00076A   24099.91962647 -.00000067  00000-0  00000-0 0  9995
2 26624   3.1258  84.5431 0006689 325.6865   9.2417  1.00271608 35129
0 ASTRA 2D
1 26638U 00081A   24099.86467169 -.00000223  00000-0  00000-0 0  9993
2 26638   8.7208  55.7762 0000029 268.3691 306.5862  0.99002085 44832
0 AMC-8 (GE-8)
1 26639U 00081B   24100.15803841 -.00000114  00000-0  00000+0 0  9998
2 26639   4.0581  82.2130 0005517 253.3345 289.1028  0.99152546 85201
0 CHINASAT 31 (BEIDOU 1B)
1 26643U 00082A   24099.94988049  .00000055  00000-0  00000-0 0  9996
2 26643  11.4385  32.9117 0081807  67.0031 122.3101  0.99250341 44145
0 SKYNET 4F
1 26695U 01005B   24099.86155200 -.00000215  00000-0  00000-0 0  9994
2 26695  11.2953  22.9849 0003671 177.0568  62.0770  0.99147128 84685
0 USA 157
1 26715U 01009A   24099.91658068  .00000007  00000-0  00000+0 0  9994
2 26715  10.6757  24.1971 0002742 356.3909  22.7196  1.00271235 23635
0 BSAT-2A
1 26720U 01011B   24100.12070574 -.00000141  00000-0  00000+0 0  9999
2 26720   9.0210  54.2950 0001345 222.5391 146.5824  0.99181091 84047
0 XM-2
1 26724U 01012A   24099.85383744 -.00000088  00000-0  00000-0 0  9998
2 26724   7.7006  64.2138 0004489  49.3629 311.6472  0.99080833 34321
0 EKRAN 21
1 26736U 01014A   24099.31091042 -.00000127  00000-0  00000-0 0  9995
2 26736  13.9689  19.2987 0031903  55.1992 147.0934  0.99584425 83965
0 XM-1
1 26761U 01018A   24100.51090663 -.00000287  00000-0  00000+0 0  9996
2 26761   6.8780  70.0971 0009244 307.0583 151.5786  0.99008796 83644
0 INTELSAT 10 (PAS 10)
1 26766U 01019A   24099.73868823 -.00000250  00000-0  00000-0 0  9995
2 26766   7.1591  67.3080 0003024 308.7313 234.8985  1.00157831 37601
0 INTELSAT 901
1 26824U 01024A   24100.15064149 -.00000210  00000-0  00000+0 0  9999
2 26824   0.0174 132.2939 0000684 255.7310 196.5767  1.00271092 83572
0 ASTRA 2C
1 26853U 01025A   24100.42802670 -.00000264  00000-0  00000+0 0  9990
2 26853   6.1988  72.7090 0002592 302.1716 264.8731  1.00269724 83565
0 ARTEMIS
1 26863U 01029A   24099.86805112 -.00000287  00000-0  00000-0 0  9990
2 26863  14.3590  12.4145 0013144 146.0724 113.9358  0.99291518 35915
0 GOES 12
1 26871U 01031A   24099.84939929  .00000005  00000-0  00000-0 0  9991
2 26871  10.7209  43.6108 0007337 309.1146  48.3168  0.99134891 82836
0 USA 159
1 26880U 01033A   24100.14629036 -.00000304  00000-0  00000+0 0  9996
2 26880  12.0324  21.7269 0001358 241.8720 297.5726  1.00280535 23609
0 COSMOS 2379
1 26892U 01037A   24099.88020525  .00000081  00000-0  00000-0 0  9991
2 26892  12.5711  19.8078 0004420 191.8040 355.5687  1.00171729 82841
0 INTELSAT 902
1 26900U 01039A   24099.93134469 -.00000288  00000-0  00000+0 0  9991
2 26900   4.0847  81.4219 0003277 332.8579  68.5800  1.00270927 82750
0 DIRECTV 4S
1 26985U 01052A   24099.83208181 -.00000200  00000-0  00000-0 0  9996
2 26985   4.2071  81.1633 0003676   8.0591 218.8190  0.99267013 43130
0 INSAT 3C
1 27298U 02002A   24099.75919709 -.00000240  00000-0  00000-0 0  9990
2 27298   6.4682  72.1250 0008533 123.8789  78.8570  0.99304507 81157
0 ECHOSTAR 7
1 27378U 02006A   24099.96972163 -.00000319  00000-0  00000+0 0  9995
2 27378   1.9433  87.2053 0004386 302.0548 300.3469  0.99848169 81027
0 INTELSAT 904
1 27380U 02007A   24100.15029340 -.00000219  00000-0  00000+0 0  9992
2 27380   4.9061  78.6059 0002994 298.2641 205.6151  1.00271916 81076
0 JCSAT 2A
1 27399U 02015A   24100.42654733 -.00000080  00000-0  00000+0 0  9997
2 27399   6.3122  72.8274 0005896  32.1816 166.8362  0.99025166 80463
0 ASTRA 3A
1 27400U 02015B   24099.92215013 -.00000012  00000-0  00000+0 0  9991
2 27400   8.8157  54.5927 0001637 311.9152  66.8066  0.99083592 80640
0 INTELSAT 903
1 27403U 02016A   24099.92803785 -.00000141  00000-0  00000-0 0  9998
2 27403   5.6950  75.5104 0006055 331.2430  58.2106  0.99019240 37485
0 NSS 7
1 27414U 02019A   24100.15629796 -.00000170  00000-0  00000+0 0  9990
2 27414   7.4138  65.4583 0003333 355.4578 194.2524  0.99084385 80532
0 DIRECTV 5 (TEMPO 1)
1 27426U 02023A   24100.52226590 -.00000062  00000-0  00000+0 0  9994
2 27426   2.8246  85.2409 0003366 301.5015 249.3924  1.00271792 80297
0 INTELSAT 905
1 27438U 02027A   24100.15125065 -.00000190  00000-0  00000+0 0  9997
2 27438   4.7311  79.2690 0002717 297.4206 211.1491  1.00270821 79952
0 EXPRESS 4A
1 27441U 02029A   24099.88255487  .00000073  00000-0  00000+0 0  9991
2 27441  10.2403  42.3269 0005512  70.0803 107.0661  0.99200959 79793
0 GALAXY 3C
1 27445U 02030A   24099.92249823 -.00000158  00000-0  00000-0 0  9995
2 27445   1.6642  87.7590 0002065   9.2507 337.8897  1.00271043 38470
0 EUTE 8 WEST C (HB 6)
1 27499U 02038A   24099.91736389  .00000074  00000-0  00000-0 0  9995
2 27499   6.5965  71.1218 0003351 299.7806  36.2978  0.99319677 42997
0 INTELSAT 906
1 27513U 02041A   24100.52679734  .00000008  00000-0  00000+0 0  9992
2 27513   3.7608  82.6779 0002930 299.1467  70.2223  1.00273396 78962
0 DRTS
1 27516U 02042B   24099.71768733 -.00000257  00000-0  00000-0 0  9990
2 27516   9.6746  48.2005 0015509 214.8987 148.2098  0.99274300 37650
0 EUTE 25C (EUTE W5)
1 27554U 02051A   24099.59134839  .00000137  00000-0  00000-0 0  9998
2 27554   7.0629  68.2518 0014708 316.9143  65.2610  1.00348499 40783
0 TDRS 10
1 27566U 02055A   24100.53797514  .00000082  00000-0  00000+0 0  9995
2 27566   9.4357  45.8702 0007285 325.1432 209.7564  1.00269625 78216
0 NSS 6
1 27603U 02057A   24100.49955926 -.00000125  00000-0  00000+0 0  9996
2 27603   3.3070  84.1136 0005313 249.0574 332.2734  0.99123032 78007
0 NIMIQ 2
1 27632U 02062A   24099.91962647 -.00000071  00000-0  00000-0 0  9997
2 27632   7.5701  64.8611 0005104 314.4618  40.1900  1.00270093 36167
0 INTELSAT 907
1 27683U 03007A   24099.95170795  .00000085  00000-0  00000-0 0  9996
2 27683   3.6778  81.7695 0007060  25.8592 124.8918  0.99156104 77269
0 USA 167
1 27691U 03008A   24099.98703911  .00000047  00000-0  00000+0 0  9998
2 27691   9.3600  50.0192 0001189 276.9246  96.0712  1.00272927 23614
0 INSAT 3A
1 27714U 03013A   24099.62684023  .00000101  00000-0  00000+0 0  9994
2 27714   6.8891  69.4110 0011818 307.1291 273.5259  0.99800155 76809
0 ASIASAT 4
1 27718U 03014A   24100.51319690 -.00000228  00000-0  00000+0 0  9996
2 27718   2.6507  85.7675 0002610 294.9840 149.7893  1.00270746 76911
0 COSMOS 2397
1 27775U 03015A   24100.48624660 -.00000364  00000-0  00000+0 0  9993
2 27775  11.8080  25.5561 0042015 352.6025  99.0575  1.00483277 76917
0 GSAT 2
1 27807U 03018A   24099.88394723  .00000014  00000-0  00000-0 0  9999
2 27807   9.6113  48.0943 0010872 330.5575 208.4452  0.99840259 76453
0 HELLAS-SAT 2
1 27811U 03020A   24100.29727454 -.00000087  00000-0  00000+0 0  9996
2 27811   5.3490  77.2598 0005500 286.1210 291.6632  1.00271433 76328
0 BD 1 03
1 27813U 03021A   24099.91962647 -.00000010  00000-0  00000+0 0  9996
2 27813   9.8222  47.6826 0013641 286.2353  85.7035  0.99767726 76298
0 AMC-9 (GE-12)
1 27820U 03024A   24099.75971922 -.00000236  00000-0  00000-0 0  9996
2 27820   5.9091  74.5076 0018204 327.2598 236.2528  0.99034225 76077
0 THURAYA 2
1 27825U 03026A   24100.45117472  .00000120  00000-0  00000+0 0  9995
2 27825   7.6262  34.3115 0002757 320.1490  50.1436  1.00274284 76332
0 BSAT-2C
1 27830U 03028A   24099.75684748 -.00000295  00000-0  00000-0 0  9993
2 27830   8.3777  59.2743 0006198 279.3465 283.8254  0.99204762 38725
0 OPTUS C1
1 27831U 03028B   24099.82912303 -.00000155  00000-0  00000+0 0  9997
2 27831   4.8709  78.9494 0005970 297.2486 275.5848  1.00272080 76306
0 GALAXY 23 (TELSTAR 13)
1 27854U 03034A   24100.13210569  .00000005  00000-0  00000+0 0  9994
2 27854   1.1267  89.8739 0002777 291.2560 103.4039  1.00271998 80530
0 USA 170
1 27875U 03040A   24100.12965948  .00000090  00000-0  00000+0 0  9995
2 27875   8.7905  54.9729 0003168   2.0490 203.7323  1.00269555 23614
0 INSAT 3E
1 27951U 03043E   24100.12235917  .00000117  00000-0  00000+0 0  9999
2 27951   7.8902  61.8038 0012879 239.9472 130.4307  1.00797533 75400
0 HORIZONS 1 (GALAXY 13)
1 27954U 03044A   24100.12627519  .00000125  00000-0  00000+0 0  9995
2 27954   0.0194 243.5060 0000535 171.9981  37.7912  1.00271635 75143
0 CHINASAT 20
1 28082U 03052A   24099.73313088 -.00000301  00000-0  00000-0 0  9992
2 28082   8.7024  56.9225 0003416  75.5168 113.4834  0.99113012 74469
0 YAMAL 202
1 28089U 03053A   24099.91296674 -.00000084  00000-0  00000+0 0  9998
2 28089   4.3130  80.5165 0003326 302.3352 306.9656  1.00272211 74661
0 YAMAL 201
1 28094U 03053B   24099.95605910 -.00000098  00000-0  00000-0 0  9991
2 28094   7.7633  63.3449 0003653  50.1669 142.5620  1.00312065 76869
0 UFO 11 (USA 174)
1 28117U 03057A   24100.38586354 -.00000108  00000-0  00000+0 0  9992
2 28117   7.9611  36.9259 0003075 356.9031  18.5719  1.00272442 74410
0 AMOS 2
1 28132U 03059A   24100.12453473  .00000026  00000-0  00000+0 0  9996
2 28132   6.5479  71.5541 0001648  28.5613 343.9147  0.99397709 74157
0 SESAT 2 (EXPRESS AM-22)
1 28134U 03060A   24099.65929515 -.00000014  00000-0  00000-0 0  9999
2 28134   6.2876  72.7728 0007403 109.7758 274.1208  0.99356099 74097
0 AMC-10 (GE-10)
1 28154U 04003A   24100.29892796 -.00000179  00000-0  00000+0 0  9993
2 28154   4.6534  77.7189 0002709   0.3845 225.7612  0.99123208 73707
0 USA 176
1 28158U 04004A   24099.90398481 -.00000222  00000-0  00000+0 0  9994
2 28158  10.9702  31.4473 0000630 273.9845 304.8775  1.00267461 23594
0 ABS 4 (MBSAT)
1 28184U 04007A   24100.52618817  .00000036  00000-0  00000+0 0  9997
2 28184   4.0070  81.6015 0005291 298.0644  68.9989  1.00269369 73574
0 EUTE 7A (EUTE W3A)
1 28187U 04008A   24100.12845074  .00000099  00000-0  00000+0 0  9993
2 28187   3.7908  81.3677 0002652 299.9887  83.5511  1.00272798 73564
0 RADUGA 1-7
1 28194U 04010A   24099.94683469  .00000121  00000-0  00000-0 0  9996
2 28194  12.7455  26.9025 0002689 213.4155 325.8073  1.00324833 73406
0 SUPERBIRD 6
1 28218U 04011A   24099.88133655  .00000084  00000-0  00000-0 0  9992
2 28218  12.6327  23.1888 0030731  25.6432 164.7325  0.99457531 72493
0 EXPRESS AM-11
1 28234U 04015A   24099.87341087  .00000074  00000-0  00000-0 0  9992
2 28234  11.9701  28.8674 0006300 310.5630 231.7589  0.99232988 72488
0 AMC-11 (GE-11)
1 28252U 04017A   24099.91927840 -.00000057  00000-0  00000-0 0  9998
2 28252   0.0586  63.1524 0001819 273.8538  80.4845  1.00271743 72852
0 INTELSAT 1002
1 28358U 04022A   24100.33382937 -.00000019  00000-0  00000+0 0  9999
2 28358   0.0076 258.6904 0000850 118.4416 300.1346  1.00270453 72618
0 APSTAR 5 (TELSTAR 18)
1 28364U 04024A   24099.89743573 -.00000283  00000-0  00000-0 0  9997
2 28364   4.9263  79.0547 0007052 312.1652 269.1540  0.99015950 72380
0 ANIK F2
1 28378U 04027A   24100.52237660 -.00000056  00000-0  00000+0 0  9990
2 28378   1.2155  89.5101 0002598 315.1535 230.5353  1.00271208 72343
0 GSAT 3 (EDUSAT)
1 28417U 04036A   24100.54405549 -.00000044  00000-0  00000+0 0  9996
2 28417   9.7954  47.6815 0002760 102.0388  79.3688  0.99268942 71172
0 AMC-15
1 28446U 04041A   24099.63405861 -.00000097  00000-0  00000-0 0  9997
2 28446   0.0577  63.5439 0002451 306.0835 310.9547  1.00271429 71296
0 EXPRESS AM-1
1 28463U 04043A   24099.90622852  .00000006  00000-0  00000+0 0  9997
2 28463   9.9300  46.1401 0008481 234.0459 325.8124  0.99189999 70779
0 AMC-16
1 28472U 04048A   24099.96197662 -.00000120  00000-0  00000-0 0  9993
2 28472   2.5202  86.3237 0002901 229.0337 332.5151  0.99103154 70563
0 NSS 10 (AMC-12)
1 28526U 05003A   24099.86227862 -.00000256  00000-0  00000+0 0  9999
2 28526   4.4403  79.9703 0003068 295.4166  95.1708  1.00270848 70305
0 XTAR-EUR
1 28542U 05005A   24099.61322674  .00000145  00000-0  00000-0 0  9996
2 28542   2.2696  86.6251 0003294 265.9179  94.5549  1.00272115 70173
0 XM-3
1 28626U 05008A   24100.42550306 -.00000213  00000-0  00000+0 0  9995
2 28626   0.0012 188.7244 0000293  17.8497  59.6311  1.00267825 70051
0 INMARSAT 4-F1
1 28628U 05009A   24100.12341015  .00000027  00000-0  00000+0 0  9992
2 28628   4.2066  40.6661 0001995   2.5429  17.1542  1.00272224 69651
0 EXPRESS AM-2
1 28629U 05010A   24099.95692931  .00000036  00000-0  00000-0 0  9990
2 28629   7.8443  62.7934 0019481  76.0932 121.5719  0.99037128 32646
0 DIRECTV 8
1 28659U 05019A   24099.91762495 -.00000007  00000-0  00000-0 0  9990
2 28659   0.0138 249.7488 0002716 132.9931  26.2060  1.00272779 69051
0 GALAXY 28 (IA 8)
1 28702U 05022A   24100.13837131 -.00000192  00000-0  00000+0 0  9994
2 28702   1.2312  88.6715 0003180 233.3922 196.6132  1.00272298 68813
0 IPSTAR 1
1 28786U 05028A   24099.86785720 -.00000371  00000-0  00000-0 0  9995
2 28786   0.0138 247.6500 0001645 130.3873 251.5045  1.00269727 44239
0 GALAXY 14
1 28790U 05030A   24100.16265062  .00000149  00000-0  00000+0 0  9992
2 28790   2.9532  84.6189 0001529 279.3349 285.3753  1.00269917 68281
0 ANIK F1-R
1 28868U 05036A   24099.91997456 -.00000080  00000-0  00000-0 0  9999
2 28868   2.2533  86.4307 0002933 308.6933  26.4174  1.00271366 38434
0 SYRACUSE 3A
1 28885U 05041B   24100.16526130  .00000126  00000-0  00000+0 0  9990
2 28885   2.5971  85.6896 0003033 291.4084 286.4943  1.00274093 67720
0 INMARSAT 4-F2
1 28899U 05044A   24100.18521494 -.00000266  00000-0  00000+0 0  9994
2 28899   4.1175  42.1402 0002123 351.0981  14.8435  1.00272867 67498
0 TELKOM 2
1 28902U 05046A   24099.92215013 -.00000007  00000-0  00000-0 0  9991
2 28902   2.6987  85.7270 0003182 198.7260 148.6133  0.99144623 67301
0 SPACEWAY 2
1 28903U 05046B   24100.12845074  .00000097  00000-0  00000+0 0  9990
2 28903   3.4424  81.0719 0000367 184.7470 199.4102  1.00270580 67488
0 INSAT 4A
1 28911U 05049A   24099.56367521  .00000002  00000-0  00000-0 0  9990
2 28911   4.3798  81.4028 0002270  75.9866 143.6278  0.99196470 37498
0 METEOSAT 9 (MSG 2)
1 28912U 05049B   24100.52314240  .00000122  00000-0  00000+0 0  9992
2 28912   7.5306  61.4261 0001506 316.4846  54.3550  1.00270794 67071
0 EUTE 172A (GE 23)
1 28924U 05052A   24099.90431051  .00000004  00000-0  00000-0 0  9998
2 28924   2.1832  86.2705 0006199 292.8083 318.1121  1.00272402 44315
0 ECHOSTAR 10
1 28935U 06003A   24100.52237716 -.00000063  00000-0  00000+0 0  9995
2 28935   0.0553  70.2867 0001888 269.5596 296.2391  1.00270727 66504
0 HIMAWARI 7
1 28937U 06004A   24099.89830595 -.00000294  00000-0  00000-0 0  9996
2 28937   3.7139  82.5184 0004587 150.8214  71.5279  0.99079374 66301
0 SPAINSAT
1 28945U 06007A   24099.53528620 -.00000224  00000-0  00000+0 0  9994
2 28945   0.0310 240.2089 0002315 116.2266   3.5563  1.00270436 66191
0 EUTELSAT HOTBIRD 13E
1 28946U 06007B   24100.30179971  .00000078  00000-0  00000+0 0  9990
2 28946   0.0751 159.6611 0004295 222.2361 297.5394  1.00270482 66286
0 JCSAT 9
1 29045U 06010A   24100.50904656 -.00000335  00000-0  00000+0 0  9990
2 29045   0.0169 192.7615 0001720 117.3623 203.4063  1.00272415 65935
0 ASTRA 1KR
1 29055U 06012A   24100.15986589  .00000112  00000-0  00000+0 0  9991
2 29055   0.0397  60.0329 0001783 256.1762 318.3799  1.00271132 65738
0 GOES 13
1 29155U 06018A   24100.50969101 -.00000273  00000-0  00000+0 0  9997
2 29155   2.4468  86.0486 0087585 252.8481 200.9902  0.99158612 65489
0 THAICOM 5
1 29163U 06020B   24099.95336139  .00000092  00000-0  00000-0 0  9993
2 29163   3.9519  82.3673 0000772  97.7444  61.4659  0.99201310 65322
0 KAZSAT 1
1 29230U 06022A   24099.91658068  .00000064  00000-0  00000-0 0  9993
2 29230  10.4453  41.9487 0002553 156.6163 205.4596  0.99231232 64711
0 GALAXY 16
1 29236U 06023A   24100.53286455 -.00000134  00000-0  00000+0 0  9996
2 29236   0.0208 236.0343 0002560 140.5750 274.4584  1.00269797 65253
0 EUTE HOT BIRD 13B (HB 8)
1 29270U 06032A   24099.94787896  .00000148  00000-0  00000-0 0  9996
2 29270   0.0610  42.2371 0003723 340.0244 189.7272  1.00271587 39290
0 JCSAT 10
1 29272U 06033A   24100.54755777 -.00000352  00000-0  00000+0 0  9998
2 29272   0.0269 105.7500 0002193 282.2837 135.3472  1.00272593 64752
0 SYRACUSE 3B
1 29273U 06033B   24099.94030800 -.00000051  00000-0  00000-0 0  9990
2 29273   0.8432  90.2539 0004459 280.5608 160.1418  1.00271167 64692
0 KOREASAT 5
1 29349U 06034A   24099.96380410 -.00000366  00000-0  00000-0 0  9991
2 29349   0.0113 219.4380 0001077 161.5251 276.7579  1.00271996 33304
0 CHINASAT 22A
1 29398U 06038A   24100.15386133 -.00000136  00000-0  00000+0 0  9995
2 29398  10.0481  43.4706 0003346  46.9295 151.8453  1.00102742 64354
0 DIRECTV 9S
1 29494U 06043A   24099.92119288 -.00000120  00000-0  00000-0 0  9999
2 29494   0.3507  89.4737 0003044 295.7873  42.8131  1.00271538 43367
0 OPTUS D1
1 29495U 06043B   24099.89995939 -.00000190  00000-0  00000-0 0  9998
2 29495   0.9655  89.5036 0003756 283.8765 300.2401  1.00272685 64109
0 ARABSAT 4B
1 29526U 06051A   24100.51922637  .00000138  00000-0  00000+0 0  9999
2 29526   0.0508 322.5077 0004944  33.1728  55.7211  1.00274433 63880
0 WILDBLUE 1
1 29643U 06054A   24099.91919137 -.00000056  00000-0  00000-0 0  9995
2 29643   0.7323  91.1436 0001745 295.3374  30.8914  1.00271657 45772
0 MEASAT 3
1 29648U 06056A   24100.16665366  .00000125  00000-0  00000+0 0  9999
2 29648   2.6607  86.2496 0003072 250.0023 334.7893  0.99740580 63481
0 ETS 8
1 29656U 06059A   24100.52144594  .00000037  00000-0  00000+0 0  9998
2 29656  10.0478  45.3643 0002253 315.4531 273.0563  0.99004552 63097
0 BEIDOU 1D
1 30323U 07003A   24099.85357639 -.00000085  00000-0  00000+0 0  9994
2 30323   7.9966  65.0279 0055364 252.9420 105.0411  0.99153522 62776
0 INSAT 4B
1 30793U 07007A   24099.69636678  .00000009  00000-0  00000-0 0  9990
2 30793   4.0253  83.1983 0012069 145.6486  67.3752  0.99077012 34093
0 SKYNET 5A
1 30794U 07007B   24099.93591245 -.00000277  00000-0  00000-0 0  9996
2 30794   3.2536  83.2677 0003841 305.8446 240.6900  1.00270284 62680
0 ANIK F3
1 31102U 07009A   24099.91771198 -.00000008  00000-0  00000-0 0  9999
2 31102   0.0205 213.4376 0002224 163.9246  31.9473  1.00269448 38544
0 ASTRA 1L
1 31306U 07016A   24100.23148549  .00000112  00000-0  00000+0 0  9993
2 31306   0.0249  13.8957 0004741 320.9564 325.6462  1.00268703 62023
0 GALAXY 17
1 31307U 07016B   24100.54445876 -.00000213  00000-0  00000+0 0  9994
2 31307   0.9422  90.1395 0002802 292.9408 286.1906  1.00271451 62063
0 NIGCOMSAT 1
1 31395U 07018A   24099.88429532 -.00000074  00000-0  00000-0 0  9994
2 31395  10.5511  38.7517 0007264  68.6102 121.9942  1.00198789 62042
0 CHINASAT 6B
1 31800U 07031A   24099.96432623 -.00000369  00000-0  00000-0 0  9991
2 31800   0.3368  92.1365 0003731 273.7156 294.5192  1.00271816 61415
0 DIRECTV 10
1 31862U 07032A   24100.30396411 -.00000111  00000-0  00000+0 0  9999
2 31862   0.0110 175.3023 0000302 100.6997 288.6595  1.00270082 61394
0 SPACEWAY 3
1 32018U 07036A   24100.15203385 -.00000242  00000-0  00000+0 0  9992
2 32018   1.9198  86.3370 0002005 351.2250 154.7595  0.99141105 61100
0 BSAT-3A
1 32019U 07036B   24099.67678670 -.00000359  00000-0  00000-0 0  9990
2 32019   0.0365 152.0388 0001270 123.8164 275.0414  1.00272181 42510
0 INSAT 4CR
1 32050U 07037A   24100.52479581  .00000093  00000-0  00000+0 0  9996
2 32050   4.8242  79.1182 0003251 345.1731  16.9562  0.99215754 60710
0 OPTUS D2
1 32252U 07044A   24100.51228209 -.00000118  00000-0  00000+0 0  9995
2 32252   0.0185 194.9778 0003255 183.9200 163.7474  1.00271256 60432
0 INTELSAT 11 (PAS 11)
1 32253U 07044B   24100.14750868 -.00000275  00000-0  00000+0 0  9990
2 32253   1.4854  88.5597 0003191 298.1932 181.2209  1.00270763 60510
0 WGS F1 (USA 195)
1 32258U 07046A   24100.00188256  .00000033  00000-0  00000-0 0  9990
2 32258   0.0149 191.3194 0000465 108.8861 264.2305  1.00270448 44333
0 STAR ONE C1
1 32293U 07056A   24099.69915150  .00000057  00000-0  00000-0 0  9999
2 32293   2.6770  85.5019 0010135 194.1435  31.8369  0.99063790  7468
0 SKYNET 5B
1 32294U 07056B   24100.16108421  .00000136  00000-0  00000+0 0  9997
2 32294   3.2097  82.8323 0003776 292.2281 265.9019  1.00273119 60178
0 ASTRA 4A (SIRIUS 4)
1 32299U 07057A   24100.37185289  .00000025  00000-0  00000+0 0  9990
2 32299   0.0594  70.4807 0002754 302.8855 323.4145  1.00272729 59950
0 RADUGA 1M-1
1 32373U 07058A   24099.92293333 -.00000058  00000-0  00000-0 0  9990
2 32373   8.3788  59.0051 0002993 344.7932  33.9109  0.99393759 44455
0 RASCOM 1
1 32387U 07063A   24099.68392255 -.00000299  00000-0  00000-0 0  9994
2 32387   9.8209  47.9902 0009607  51.2684 129.8813  0.99067155 59550
0 HORIZONS 2
1 32388U 07063B   24100.52618817 -.00000261  00000-0  00000+0 0  9994
2 32388   0.5337  90.1689 0001895 281.9328 301.7807  1.00272187 59865
0 THURAYA 3
1 32404U 08001A   24100.57878764 -.00000308  00000-0  00000+0 0  9992
2 32404   4.7627  31.8075 0002080 354.7599 118.5925  1.00270679 59521
0 EXPRESS AM-33
1 32478U 08003A   24099.95588503 -.00000102  00000-0  00000-0 0  9996
2 32478   2.7477  85.1174 0004517 171.8598 357.8997  1.00394256 45721
0 THOR 5
1 32487U 08006A   24100.15594987 -.00000017  00000-0  00000+0 0  9991
2 32487   0.0129 163.3268 0002575 224.1401 225.7968  1.00271245 59162
0 WINDS (KIZUNA)
1 32500U 08007A   24099.95910487 -.00000205  00000-0  00000-0 0  9990
2 32500   6.5826  70.7752 0003612   6.2204 195.4182  1.00154347 58887
0 AMC-14
1 32708U 08011A   24099.98243992  .00000065  00000-0  00000+0 0  9990
2 32708  22.9963  30.1318 0044110 358.4773 180.2006  1.00271976 60080
0 DIRECTV 11
1 32729U 08013A   24099.63527692 -.00000133  00000-0  00000-0 0  9997
2 32729   0.0484  47.0118 0000797 101.8019 178.0558  1.00269118 37579
0 ICO G-1
1 32763U 08016A   24099.92284632 -.00000178  00000-0  00000-0 0  9993
2 32763   4.8092  34.4417 0002442 335.7012  66.8865  1.00272140 58539
0 VINASAT 1
1 32767U 08018A   24099.68122484 -.00000337  00000-0  00000-0 0  9995
2 32767   0.0430  48.3203 0001977 358.9462 167.3360  1.00271236 58578
0 STAR ONE C2
1 32768U 08018B   24100.14315755 -.00000285  00000-0  00000+0 0  9996
2 32768   0.0611  45.3830 0002579 334.2824 164.7407  1.00272246 58532
0 AMOS 3
1 32794U 08022A   24099.94048205 -.00000043  00000-0  00000-0 0  9991
2 32794   0.0089 169.4637 0001806 179.5833 183.0937  1.00272490 74323
0 GALAXY 18
1 32951U 08024A   24099.91684177  .00000017  00000-0  00000-0 0  9999
2 32951   0.0177 196.1974 0003003 193.5935  14.9185  1.00271469 37834
0 CHINASAT 9
1 33051U 08028A   24100.10286612 -.00000255  00000-0  00000+0 0  9995
2 33051   0.0214 131.7061 0004590 265.3297 290.0321  1.00270257 57993
0 SKYNET 5C
1 33055U 08030A   24100.15255601 -.00000146  00000-0  00000+0 0  9990
2 33055   1.7298  85.9828 0004849 304.5773 204.4350  1.00271657 57884
0 TURKSAT 3A
1 33056U 08030B   24100.16447809  .00000139  00000-0  00000+0 0  9996
2 33056   0.0553 104.9290 0004382 297.8069 256.3371  1.00270166 58064
0 COSMOS 2440
1 33108U 08033A   24099.90552172 -.00000153  00000-0  00000-0 0  9994
2 33108   9.0478  46.9739 0003580 157.7404  39.2122  1.00272136 57774
0 INTELSAT 25(PROTOSTAR 1)
1 33153U 08034A   24100.43620683 -.00000231  00000-0  00000+0 0  9995
2 33153   0.6411  89.8209 0002967 293.1028 300.7681  1.00270481 57622
0 BADR 6
1 33154U 08034B   24100.51922637  .00000138  00000-0  00000+0 0  9994
2 33154   0.7825  87.0579 0009673 325.8488 358.4647  1.00272890 57748
0 ECHOSTAR 11
1 33207U 08035A   24099.91945243 -.00000064  00000-0  00000-0 0  9993
2 33207   0.0554  80.2135 0002583 290.5848  47.8463  1.00271111 43197
0 SUPERBIRD 7
1 33274U 08038A   24099.89839299 -.00000259  00000-0  00000-0 0  9996
2 33274   0.2913  91.5608 0001067 259.0480 314.3698  1.00270086 97719
0 AMC-21
1 33275U 08038B   24099.91649367  .00000029  00000-0  00000-0 0  9999
2 33275   0.8781  90.8846 0002789 285.9011  25.8942  1.00269918 57472
0 INMARSAT 4-F3
1 33278U 08039A   24100.53432509 -.00000147  00000-0  00000+0 0  9993
2 33278   3.6619  26.7741 0002798 357.8862 267.9513  1.00272575 57253
0 NIMIQ 4
1 33373U 08044A   24100.49773178 -.00000228  00000-0  00000+0 0  9990
2 33373   0.0141 150.9188 0002353 225.7976 278.6992  1.00272080 56824
0 GALAXY 19
1 33376U 08045A   24100.53538947 -.00000147  00000-0  00000+0 0  9992
2 33376   0.0266 265.1184 0002632 110.1604 278.7099  1.00270155 56825
0 VENESAT-1
1 33414U 08055A   24099.93378132 -.00000257  00000-0  00000-0 0  9998
2 33414   3.6957  82.4875 0061144 174.2500 239.7031  0.99273127 56529
0 ASTRA 1M
1 33436U 08057A   24100.51783400  .00000112  00000-0  00000+0 0  9997
2 33436   0.0612  80.6355 0002434 347.9158 335.3112  1.00273601 56477
0 CIEL-2
1 33453U 08063A   24100.13776215 -.00000019  00000-0  00000+0 0  9994
2 33453   0.6942  91.8762 0010442 313.9822 109.1733  0.99020918 56107
0 EUTE HOT BIRD 13C (HB 9)
1 33459U 08065A   24100.15360028 -.00000107  00000-0  00000+0 0  9993
2 33459   0.8858  87.6152 0004643 288.7852 224.2863  1.00271163 56235
0 FENGYUN 2E
1 33463U 08066A   24100.15203385 -.00000245  00000-0  00000+0 0  9994
2 33463   7.4271  61.1006 0001973  17.8763 153.1743  0.99122968 56076
0 EXPRESS AM-44
1 33595U 09007A   24100.15386133 -.00000096  00000-0  00000+0 0  9993
2 33595   1.3990  87.9822 0003846 287.1650 227.1190  1.00271337 55528
0 EXPRESS MD1
1 33596U 09007B   24099.95466672  .00000071  00000-0  00000-0 0  9999
2 33596   8.3603  60.0505 0011869 337.7547 210.7231  0.99109000 55155
0 NSS 9
1 33749U 09008A   24100.12087979  .00000064  00000-0  00000+0 0  9992
2 33749   0.0587  65.2741 0002093 299.9873  59.1581  1.00271634 55557
0 EUTE 33E
1 33750U 09008B   24099.92232418 -.00000156  00000-0  00000-0 0  9993
2 33750   0.6052  89.0932 0003940 296.5394  48.6406  1.00269679 55422
0 TELSTAR 11N
1 34111U 09009A   24100.14863998 -.00000259  00000-0  00000+0 0  9999
2 34111   0.0014 187.0573 0002527 174.5815 212.1905  1.00270972 55407
0 RADUGA 1-8
1 34264U 09010A   24099.86364237 -.00000339  00000-0  00000-0 0  9998
2 34264  10.5365  48.8243 0012672  13.0142 204.4154  0.99823706 55193
0 WGS F2 (USA 204)
1 34713U 09017A   24100.16743687  .00000061  00000-0  00000+0 0  9990
2 34713   0.0113 226.5479 0000330  52.3238  36.7870  1.00269870 54923
0 SES 7 (PROTOSTAR 2)
1 34941U 09027A   24100.48567800 -.00000353  00000-0  00000+0 0  9995
2 34941   0.1001  82.8110 0001963 236.8827 161.6005  1.00272201 54609
0 GOES 14
1 35491U 09033A   24099.91971351 -.00000073  00000-0  00000-0 0  9998
2 35491   0.1772 114.1292 0002903 301.1389   4.9337  1.00270162 54093
0 SIRIUS FM-5
1 35493U 09034A   24100.49694859 -.00000208  00000-0  00000+0 0  9991
2 35493   0.0223 264.7664 0001080 304.2229  81.9817  1.00270039 53968
0 TERRESTAR 1
1 35496U 09035A   24100.52205154 -.00000062  00000-0  00000+0 0  9993
2 35496   4.3002  47.0849 0002978 332.2630 255.8353  1.00272219 54144
0 ASIASAT 5
1 35696U 09042A   24099.93591245 -.00000316  00000-0  00000-0 0  9997
2 35696   0.0439 105.2309 0001906 264.8252 265.0479  1.00273174 53693
0 JCSAT 12
1 35755U 09044A   24100.50904660 -.00000335  00000-0  00000+0 0  9990
2 35755   0.1057  91.7278 0001740 318.9220 102.8886  1.00270614 53654
0 OPTUS D3
1 35756U 09044B   24100.38610024 -.00000155  00000-0  00000+0 0  9995
2 35756   0.0566  23.8882 0004283 348.4293 120.7817  1.00272077 53624
0 NIMIQ 5
1 35873U 09050A   24100.52618817 -.00000264  00000-0  00000+0 0  9997
2 35873   0.0148 236.0717 0002631 140.9618 297.9395  1.00270354 53338
0 AMAZONAS 2
1 35942U 09054A   24099.92916912 -.00000290  00000-0  00000+0 0  9995
2 35942   0.0594 109.1512 0003495 311.6898  50.3281  1.00269570 53163
0 COMSATBW-1
1 35943U 09054B   24100.16865517  .00000017  00000-0  00000+0 0  9995
2 35943   0.0719  88.0312 0002901 297.6812 295.8795  1.00271177 53258
0 NSS 12
1 36032U 09058A   24099.92410598  .00000063  00000-0  00000-0 0  9997
2 36032   0.0644  75.1878 0002933 311.8802 200.2548  1.00272097 52901
0 THOR 6
1 36033U 09058B   24100.29910201 -.00000018  00000-0  00000+0 0  9997
2 36033   0.0121 158.0465 0001832 235.7052 271.0870  1.00270738 52952
0 INTELSAT 14
1 36097U 09064A   24100.14716059 -.00000281  00000-0  00000+0 0  9995
2 36097   0.0180 273.2366 0002621 117.4086 175.2192  1.00270865 52680
0 EUTE 36B (EUTE W7)
1 36101U 09065A   24099.94848811  .00000147  00000-0  00000-0 0  9990
2 36101   0.0714  20.8214 0005782   5.0565 189.1533  1.00274192 52705
0 INTELSAT 15
1 36106U 09067A   24099.95823465 -.00000193  00000-0  00000-0 0  9998
2 36106   0.0196 206.9939 0002312 180.2724 240.5366  1.00269943 33213
0 WGS F3 (USA 211)
1 36108U 09068A   24100.14759571 -.00000276  00000-0  00000+0 0  9992
2 36108   0.0328 106.2927 0000361 156.8276 305.0963  1.00268677 52498
0 DIRECTV 12
1 36131U 09075A   24100.53014133 -.00000111  00000-0  00000+0 0  9995
2 36131   0.0429  49.1355 0000552 283.3925 313.8485  1.00270131 52376
0 BEIDOU 3
1 36287U 10001A   24099.89769679 -.00000287  00000-0  00000-0 0  9991
2 36287   2.4649  57.1275 0005035 301.6131 302.4531  1.00273513 52151
0 RADUGA 1M-2
1 36358U 10002A   24100.53097441 -.00000190  00000-0  00000+0 0  9998
2 36358   1.5439  89.2773 0002136 265.8053 119.3828  1.00270593 52002
0 SDO
1 36395U 10005A   24099.99286962 -.00000049  00000-0  00000+0 0  9990
2 36395  32.9830  98.6631 0001852 221.5157 133.2694  1.00273831 52016
0 INTELSAT 16
1 36397U 10006A   24100.14098199 -.00000252  00000-0  00000+0 0  9992
2 36397   0.0121 203.7500 0001775 182.6786 145.9915  1.00270266 51700
0 GOES 15
1 36411U 10008A   24100.16830708  .00000031  00000-0  00000+0 0  9997
2 36411   0.1224  92.1894 0002324 196.4980  31.0165  1.00272896 51611
0 ECHOSTAR 14
1 36499U 10010A   24099.91762495 -.00000007  00000-0  00000-0 0  9998
2 36499   0.0140 239.7353 0002009 136.5079  32.8494  1.00270314 38539
0 SES 1
1 36516U 10016A   24099.92127991 -.00000122  00000-0  00000-0 0  9999
2 36516   0.0454  46.9334 0002668 338.4673  42.8993  1.00269791 51018
0 ASTRA 3B
1 36581U 10021A   24100.03313535  .00000129  00000-0  00000+0 0  9998
2 36581   0.0468  45.8065 0003608 345.0084 202.3913  1.00269750 50636
0 COMSATBW-2
1 36582U 10021B   24100.30188674  .00000080  00000-0  00000+0 0  9999
2 36582   0.0503  74.9966 0002219 307.5496 297.3359  1.00268996 50973
0 BEIDOU 4
1 36590U 10024A   24100.10034247 -.00000142  00000-0  00000+0 0  9995
2 36590   2.6527  60.3028 0009530 335.2416 278.0562  1.00271424 50766
0 BADR-5 (ARABSAT 5B)
1 36592U 10025A   24100.30449742  .00000137  00000-0  00000+0 0  9993
2 36592   0.0334 139.1726 0002468 288.2685 266.1780  1.00269728 50892
0 COMS 1
1 36744U 10032A   24099.96684990 -.00000350  00000-0  00000+0 0  9993
2 36744   2.7506  85.3835 0000801 291.6416 296.8693  1.00270308 50645
0 ARABSAT 5A
1 36745U 10032B   24100.16212848  .00000146  00000-0  00000+0 0  9990
2 36745   0.0810  56.7399 0004375 339.4045 250.6053  1.00273940 50611
0 ECHOSTAR 15
1 36792U 10034A   24099.85749238 -.00000290  00000-0  00000+0 0  9996
2 36792   0.0546  81.7796 0002316 266.3677  96.4982  1.00272248 50381
0 BEIDOU 5
1 36828U 10036A   24100.59043334 -.00000092  00000-0  00000+0 0  9996
2 36828  54.2448 170.1913 0040199 199.2506 145.5771  1.00246542 50215
0 NILESAT 201
1 36830U 10037A   24100.15473155 -.00000065  00000-0  00000+0 0  9990
2 36830   0.0159 185.1418 0002900 214.8993 206.5511  1.00271596 50277
0 RASCOM QAF 1R
1 36831U 10037B   24099.91229951  .00000011  00000-0  00000-0 0  9996
2 36831   0.0591  96.8996 0005619 274.6321 157.4284  1.00272756 50287
0 AEHF 1 (USA 214)
1 36868U 10039A   24099.79771677  .00000020  00000-0  00000-0 0  9991
2 36868   6.2054  78.1919 0002490   2.5683  47.9726  1.00268276 45640
0 CHINASAT 6A
1 37150U 10042A   24099.92014861 -.00000006  00000-0  00000-0 0  9995
2 37150   2.1522  87.2879 0003473 282.3446  52.8440  0.99693640 49908
0 XM-5
1 37185U 10053A   24099.91840816 -.00000031  00000-0  00000-0 0  9999
2 37185   0.0265 266.6369 0000648 350.5009 155.9414  1.00270586 49468
0 BSAT-3B
1 37207U 10056B   24099.67678670 -.00000359  00000-0  00000-0 0  9990
2 37207   0.0089 357.4841 0003828 356.1773 197.2359  1.00270890 44093
0 BEIDOU 6
1 37210U 10057A   24100.51217736 -.00000119  00000-0  00000+0 0  9996
2 37210   1.6816  67.7008 0004344 206.1501 268.7393  1.00272644 49271
0 SKYTERRA 1
1 37218U 10061A   24100.53164829 -.00000124  00000-0  00000+0 0  9991
2 37218   3.4879  47.4775 0002781   1.2606 239.6316  1.00272115 49216
0 HYLAS 1
1 37237U 10065A   24099.96467432 -.00000368  00000-0  00000+0 0  9996
2 37237   3.5997  82.6836 0000714 266.5445 312.9481  1.00270193 48995
0 INTELSAT 17
1 37238U 10065B   24099.92410598 -.00000010  00000-0  00000-0 0  9999
2 37238   0.0240 248.4534 0002636 143.3890 204.4928  1.00273579 45222
0 BEIDOU 7
1 37256U 10068A   24099.86577810 -.00000209  00000-0  00000-0 0  9996
2 37256  49.0071 281.6578 0032511 208.7682 127.1106  1.00271958 48775
0 EUTE KA SAT 9A (KA SAT)
1 37258U 10069A   24100.30101650  .00000053  00000-0  00000+0 0  9991
2 37258   0.0611 288.2653 0001252 184.0777 202.9889  1.00267626 48801
0 HISPASAT 30W-5
1 37264U 10070A   24099.73313215 -.00000224  00000-0  00000-0 0  9998
2 37264   0.0467 107.6146 0003765 298.1244  25.6483  1.00271668 45162
0 KOREASAT 6
1 37265U 10070B   24099.86785720 -.00000370  00000-0  00000+0 0  9998
2 37265   0.0114 170.0130 0001269 219.6066 236.4124  1.00270144 48677
0 ELEKTRO-L
1 37344U 11001A   24099.59047817  .00000143  00000-0  00000-0 0  9997
2 37344   7.8532  62.1806 0006003  13.0275  10.3744  1.00178957 48478
0 BEIDOU 8
1 37384U 11013A   24099.79135009 -.00000201  00000-0  00000-0 0  9993
2 37384  61.2582  47.7791 0025762 181.0958   2.4625  1.00275651 47647
0 INTELSAT NEW DAWN
1 37392U 11016A   24100.16221550  .00000148  00000-0  00000+0 0  9994
2 37392   0.0112 218.9236 0001446 179.1206 251.0273  1.00271205 47506
0 YAHSAT 1A
1 37393U 11016B   24099.92966015  .00000093  00000-0  00000-0 0  9998
2 37393   0.0168 218.9677 0002272 172.9132 192.9334  1.00269584 34395
0 SBIRS GEO 1 (USA 230)
1 37481U 11019A   24099.97848764 -.00000011  00000-0  00000-0 0  9995
2 37481   2.8651  47.3932 0002196 325.8961 242.3738  1.00272311 47385
0 TELSTAR 14R
1 37602U 11021A   24099.92882103 -.00000287  00000-0  00000-0 0  9998
2 37602   0.0160 204.5236 0003105 170.3658  94.1401  1.00270765 47826
0 GSAT 8
1 37605U 11022A   24100.23862132  .00000077  00000-0  00000+0 0  9994
2 37605   0.0648  88.6409 0005413 214.8145  35.4942  1.00271517 47231
0 ST 2
1 37606U 11022B   24100.53158358 -.00000219  00000-0  00000+0 0  9995
2 37606   0.0159 223.1612 0001592 155.1355  99.3470  1.00271446 47242
0 CHINASAT 10
1 37677U 11026A   24100.05535321 -.00000360  00000-0  00000+0 0  9990
2 37677   0.0374 278.4605 0000456 189.7531 220.0038  1.00273503 46957
0 SES 3
1 37748U 11035A   24100.52868512 -.00000109  00000-0  00000+0 0  9993
2 37748   0.0592  62.1694 0003193 327.9694 255.4275  1.00270997 46680
0 KAZSAT 2
1 37749U 11035B   24100.10173483 -.00000206  00000-0  00000+0 0  9992
2 37749   0.0389 297.8834 0000450 208.8364 174.2456  1.00272886 46556
0 ASTRA 1N
1 37775U 11041A   24100.51774699  .00000112  00000-0  00000+0 0  9994
2 37775   0.0758  44.7077 0005384 347.7120  11.4053  1.00273514 46393
0 BSAT-3C
1 37776U 11041B   24100.52665556 -.00000358  00000-0  00000+0 0  9990
2 37776   0.0483  46.5363 0000217  79.4803  11.7698  1.00271664 46510
0 PAKSAT 1R
1 37779U 11042A   24100.16369490  .00000146  00000-0  00000+0 0  9996
2 37779   0.0112 242.0786 0003253 142.6585 270.1106  1.00265568 46260
0 CHINASAT 1A
1 37804U 11047A   24099.93591245 -.00000242  00000-0  00000-0 0  9996
2 37804   1.6478  88.5915 0003753 269.1125 273.7424  1.00023953 19338
0 COSMOS 2473
1 37806U 11048A   24100.50086461 -.00000284  00000-0  00000+0 0  9999
2 37806   3.5320  83.2649 0001629 100.9365 128.2643  1.00361773 46052
0 SES 2
1 37809U 11049A   24100.49677455 -.00000204  00000-0  00000+0 0  9992
2 37809   0.0464  53.9270 0002404 323.7290 272.4048  1.00271218 45980
0 ARABSAT 5C
1 37810U 11049B   24099.98243992  .00000115  00000-0  00000-0 0  9991
2 37810   0.0557  17.2500 0001130  65.1452 128.9757  1.00271650 46046
0 EUTE 7 WEST A (AB 7)
1 37816U 11051A   24100.15464455 -.00000068  00000-0  00000+0 0  9998
2 37816   0.0968  49.9246 0001882 134.0860  62.2148  1.00271624 45954
0 QUETZSAT 1
1 37826U 11054A   24100.52618817 -.00000249  00000-0  00000+0 0  9997
2 37826   0.0645  73.5487 0002580 325.8933 271.2393  1.00270721 46091
0 INTELSAT 18
1 37834U 11056A   24099.83399630  .00000045  00000-0  00000+0 0  9991
2 37834   0.0133 253.2184 0001781 141.1438 283.4380  1.00272346 45820
0 EUTE 16A (EUTE W3C)
1 37836U 11057A   24099.98243992  .00000096  00000-0  00000-0 0  9996
2 37836   0.0703   7.0852 0005216  13.5866 186.7005  1.00271458 37953
0 VIASAT 1
1 37843U 11059A   24100.51825437 -.00000031  00000-0  00000+0 0  9997
2 37843   0.0186 174.8849 0002016 201.5150 253.2927  1.00271036 46264
0 ASIASAT 7
1 37933U 11069A   24100.05535321 -.00000342  00000-0  00000+0 0  9992
2 37933   0.0129 223.1299 0001446 107.7240 352.3282  1.00274669 45440
0 BEIDOU 10
1 37948U 11073A   24099.90240078 -.00000157  00000-0  00000+0 0  9990
2 37948  49.1380 281.3397 0094680 219.1037 116.9818  1.00266577 45389
0 AMOS 5
1 37950U 11074A   24099.51537767  .00000102  00000-0  00000-0 0  9995
2 37950   6.8928  69.2248 0003519 359.6201 331.7059  1.00254620 44976
0 LUCH 5A
1 37951U 11074B   24099.75971922 -.00000052  00000-0  00000-0 0  9992
2 37951   6.7059  86.0679 0002987 304.3066 247.5014  1.00272087 45030
0 NIGCOMSAT 1R
1 38014U 11077A   24099.92966015  .00000137  00000-0  00000-0 0  9995
2 38014   0.0357 358.7152 0004393 353.8756 222.2344  1.00269036 44947
0 FENGYUN 2F
1 38049U 12002A   24099.86118279 -.00000362  00000-0  00000-0 0  9997
2 38049   5.4228  73.7966 0002731  57.7521 128.1387  1.00272851 44863
0 WGS F4 (USA 233)
1 38070U 12003A   24100.10208292 -.00000222  00000-0  00000+0 0  9991
2 38070   0.0163 142.9000 0000524  98.7063  81.3719  1.00270663 44709
0 SES 4
1 38087U 12007A   24099.68799413 -.00000176  00000-0  00000-0 0  9992
2 38087   0.0198  80.2621 0002284 299.1674  43.6706  1.00271022 44200
0 BEIDOU 11
1 38091U 12008A   24100.16778495  .00000051  00000-0  00000+0 0  9991
2 38091   1.9086  70.2193 0009642 259.3650 347.3716  1.00272061 44468
0 MUOS 1
1 38093U 12009A   24100.53175788 -.00000133  00000-0  00000+0 0  9993
2 38093   3.9218  44.0391 0058611 359.0102 246.6349  1.00270641 44441
0 INTELSAT 22
1 38098U 12011A   24099.90049492 -.00000067  00000-0  00000-0 0  9998
2 38098   0.0173 235.3627 0001802 151.6581 206.8947  1.00272845 43795
0 COSMOS 2479
1 38101U 12012A   24099.69140649  .00000069  00000-0  00000-0 0  9997
2 38101   6.9944  63.7126 0003619 256.1737 309.8434  1.00318893 44060
0 APSTAR 7
1 38107U 12013A   24099.95658122 -.00000110  00000-0  00000-0 0  9991
2 38107   0.0158 146.8268 0002286 232.0151 239.7157  1.00271026 43798
0 YAHSAT 1B
1 38245U 12016A   24100.45187090  .00000119  00000-0  00000+0 0  9993
2 38245   0.0244 118.5679 0003039 271.5495  18.3203  1.00269216 43910
0 AEHF 2 (USA 235)
1 38254U 12019A   24099.99417497 -.00000165  00000-0  00000+0 0  9991
2 38254   4.8035  61.8232 0003044 274.9328 124.6354  1.00269927 43662
0 JCSAT 13
1 38331U 12023A   24100.03868684 -.00000364  00000-0  00000+0 0  9999
2 38331   0.0391  93.6756 0002296 301.3896 300.6443  1.00271273 43569
0 VINASAT 2
1 38332U 12023B   24100.50904654 -.00000336  00000-0  00000+0 0  9999
2 38332   0.0024 265.6289 0002550 129.7283 117.9723  1.00270675 43600
0 NIMIQ 6
1 38342U 12026A   24099.92319440 -.00000181  00000-0  00000-0 0  9998
2 38342   0.0563  78.0864 0002712 293.9667  66.8457  1.00271284 42404
0 CHINASAT 2A
1 38352U 12028A   24099.93591245 -.00000301  00000-0  00000-0 0  9991
2 38352   0.0119  30.2252 0002105 349.0583 253.5277  1.00272626 43361
0 INTELSAT 19
1 38356U 12030A   24100.51755323 -.00000064  00000-0  00000+0 0  9999
2 38356   0.0181 224.1532 0003011 160.6726 165.7462  1.00272727 43500
0 ECHOSTAR 17
1 38551U 12035A   24100.52608380 -.00000083  00000-0  00000+0 0  9993
2 38551   0.0190 194.5887 0001823 176.0350 269.9075  1.00270234 43118
0 METEOSAT 10 (MSG 3)
1 38552U 12035B   24100.15612392 -.00000009  00000-0  00000+0 0  9999
2 38552   2.8113  60.3783 0001687 300.0972 253.9602  1.00277186 43208
0 SES 5
1 38652U 12036A   24100.12965948  .00000026  00000-0  00000+0 0  9994
2 38652   0.0405 338.3599 0002109  34.8317 236.3119  1.00271673 43057
0 INTELSAT 20
1 38740U 12043A   24099.98175697 -.00000033  00000-0  00000-0 0  9995
2 38740   0.0125 168.8348 0001522  27.0428  63.7660  1.00271931 42731
0 HYLAS 2
1 38741U 12043B   24099.94753087  .00000146  00000-0  00000-0 0  9991
2 38741   0.3594  90.5347 0001589 315.0483 164.2448  1.00272708 42755
0 INTELSAT 21
1 38749U 12045A   24100.20466043 -.00000293  00000-0  00000+0 0  9999
2 38749   0.0186 175.0827 0001737 203.8593 194.6685  1.00271434 42621
0 ASTRA 2F
1 38778U 12051A   24099.88521485  .00000142  00000-0  00000-0 0  9994
2 38778   0.0586 248.0668 0001546  33.4241 263.0062  1.00268877 33117
0 GSAT 10
1 38779U 12051B   24100.10103865 -.00000172  00000-0  00000+0 0  9994
2 38779   0.0360  97.0211 0001814 195.9201  24.2503  1.00269380 42287
0 INTELSAT 23
1 38867U 12057A   24100.43194272 -.00000292  00000-0  00000+0 0  9999
2 38867   0.0148 222.6118 0000504 188.1213 249.9129  1.00282558 42094
0 BEIDOU 16
1 38953U 12059A   24100.53080038 -.00000180  00000-0  00000+0 0  9998
2 38953   2.3851  80.9254 0011000 305.5039  86.8818  1.00268272 42041
0 LUCH 5B
1 38977U 12061A   24099.68243963 -.00000140  00000-0  00000-0 0  9995
2 38977   8.7376  57.2974 0000902  73.6684 295.7899  1.00271276 41590
0 YAMAL 300K
1 38978U 12061B   24100.12087979  .00000063  00000-0  00000+0 0  9993
2 38978   0.0149 248.9845 0000424   1.9337 173.3737  1.00271077 41869
0 STARONE C3
1 38991U 12062A   24100.52618817 -.00000256  00000-0  00000+0 0  9999
2 38991   0.0437  73.0778 0001510 315.5844 284.0054  1.00269793 41842
0 EUTE 21B
1 38992U 12062B   24099.98243992  .00000122  00000-0  00000-0 0  9997
2 38992   0.0703   7.3617 0002719   9.9204 195.6890  1.00271059 41817
0 ECHOSTAR 16
1 39008U 12065A   24100.00070164 -.00000290  00000-0  00000+0 0  9990
2 39008   0.0150 199.2995 0001851 165.3164 131.8692  1.00267970 41796
0 CHINASAT 12
1 39017U 12067A   24100.10190889 -.00000214  00000-0  00000+0 0  9996
2 39017   0.0199 133.9611 0003237 246.6862 301.3524  1.00272137 41695
0 EUTE 70B
1 39020U 12069A   24099.88359914 -.00000052  00000-0  00000-0 0  9991
2 39020   0.0664   8.4756 0001090 345.0731 232.7099  1.00271614 41497
0 YAMAL 402
1 39022U 12070A   24100.16700175  .00000078  00000-0  00000+0 0  9994
2 39022   0.0344 103.6121 0003938 271.8973 297.3991  1.00272383 41587
0 SKYNET 5D
1 39034U 12075A   24100.16656663  .00000092  00000-0  00000+0 0  9990
2 39034   0.0703   3.5153 0003630  13.7535 293.3174  1.00270778 41505
0 MEXSAT 3
1 39035U 12075B   24100.54079671 -.00000033  00000-0  00000+0 0  9997
2 39035   0.0208 114.6245 0001904 245.4540 278.0613  1.00271866 41466
0 TDRS 11
1 39070U 13004A   24100.57699244  .00000072  00000-0  00000+0 0  9998
2 39070   2.7206   9.1903 0005391  37.3347 185.1368  1.00276207 40972
0 AMAZONAS 3
1 39078U 13006A   24100.50190889 -.00000290  00000-0  00000+0 0  9995
2 39078   0.0213 209.8414 0001502 174.0160 294.0584  1.00272895 40965
0 AZERSPACE 1
1 39079U 13006B   24099.92966015  .00000125  00000-0  00000-0 0  9998
2 39079   0.0227 107.9386 0001485 287.6694 182.7071  1.00269676 40798
0 SBIRS GEO 2 (USA 241)
1 39120U 13011A   24099.86219160 -.00000263  00000-0  00000-0 0  9990
2 39120   2.8550  45.8898 0002183 327.5303  96.5603  1.00271794 40527
0 EUTE 117 WEST A(SATMEX 8
1 39122U 13012A   24099.91814709 -.00000021  00000-0  00000-0 0  9991
2 39122   0.0624  73.2061 0002802 306.6227  31.5451  1.00271919 40416
0 ANIK G1
1 39127U 13014A   24100.52506892 -.00000081  00000-0  00000+0 0  9994
2 39127   0.0604  87.2031 0001989 261.5911 291.1594  1.00271245 40239
0 CHINASAT 11
1 39157U 13020A   24100.09077035 -.00000298  00000-0  00000+0 0  9998
2 39157   0.0212 234.0986 0003225  96.6233 357.6696  1.00269416 40134
0 EUTE 7B (EUTE 3D)
1 39163U 13022A   24100.00188256  .00000040  00000-0  00000-0 0  9991
2 39163   0.0709   7.0333 0005820   2.7501 195.6302  1.00272638 39957
0 WGS 5 (USA 243)
1 39168U 13024A   24100.13035650 -.00000292  00000-0  00000+0 0  9990
2 39168   0.0194 218.6534 0000820 124.3358 209.2723  1.00266682 39815
0 SES 6
1 39172U 13026A   24100.43437935 -.00000269  00000-0  00000+0 0  9996
2 39172   0.0355 304.4004 0001990  64.2479 305.3765  1.00270128 39713
0 IRNSS R1A
1 39199U 13034A   24099.52260053  .00000091  00000-0  00000-0 0  9999
2 39199  33.9227  75.4695 0018383 188.4523 176.4575  1.00275145 39359
0 MUOS 2
1 39206U 13036A   24099.76302609  .00000059  00000-0  00000+0 0  9993
2 39206   3.5129  39.4999 0048534 182.2043  73.4267  1.00271787 39283
0 ALPHASAT (I-4A F4)
1 39215U 13038A   24100.51896531  .00000127  00000-0  00000+0 0  9999
2 39215   2.8823   8.0602 0001870  37.0951   4.7990  1.00272628 39269
0 INSAT 3D
1 39216U 13038B   24100.53036525 -.00000163  00000-0  00000+0 0  9992
2 39216   0.0799  85.2792 0001853 224.4432 161.5523  1.00270712 39295
0 WGS 6 (USA 244)
1 39222U 13041A   24099.77966311  .00000081  00000-0  00000+0 0  9999
2 39222   0.0164 224.3152 0000155  29.6288  89.0358  1.00270779 39048
0 ES'HAIL 1
1 39233U 13044A   24100.51913935  .00000136  00000-0  00000+0 0  9994
2 39233   0.0568   7.4737 0002434  22.4484  20.7362  1.00273288 38895
0 GSAT 7
1 39234U 13044B   24099.72965900 -.00000085  00000-0  00000-0 0  9999
2 39234   0.0888  87.0752 0007392 201.4380 245.6272  1.00271064 37318
0 AMOS 4
1 39237U 13045A   24099.92410598 -.00000001  00000-0  00000-0 0  9991
2 39237   0.0169 140.4861 0003288 226.9816 227.8607  1.00272342 38948
0 AEHF 3 (USA 246)
1 39256U 13050A   24100.12523091  .00000128  00000-0  00000+0 0  9994
2 39256   3.3069  73.1683 0047412 183.2615 191.4261  1.00271314 38677
0 ASTRA 2E
1 39285U 13056A   24099.94700874  .00000142  00000-0  00000-0 0  9992
2 39285   0.0707 329.6210 0002888  45.7223 191.7163  1.00271611 38429
0 SIRIUS FM-6
1 39360U 13058A   24099.91823412 -.00000024  00000-0  00000-0 0  9999
2 39360   0.0206 256.6061 0000891 358.4397 157.0108  1.00270210 38044
0 RADUGA 1M-3
1 39375U 13062A   24099.94831407  .00000149  00000-0  00000-0 0  9993
2 39375   3.9177  82.1237 0002302 173.3786 318.5836  1.00272047 38098
0 SES 8
1 39460U 13071A   24100.10338826 -.00000277  00000-0  00000+0 0  9998
2 39460   0.0221 251.8279 0002215 153.9310 284.3013  1.00269000 37868
0 INMARSAT 5-F1
1 39476U 13073A   24100.45491668  .00000021  00000-0  00000+0 0  9995
2 39476   0.0169 196.4714 0000456  51.1315 176.9230  1.00268535 37929
0 TKSAT 1
1 39481U 13075A   24100.49668751 -.00000202  00000-0  00000+0 0  9998
2 39481   0.0138 258.2369 0003101  93.2505 298.3196  1.00270345 37834
0 EXPRESS AM-5
1 39487U 13077A   24100.18521494 -.00000288  00000-0  00000+0 0  9998
2 39487   0.0217 234.4183 0000059  17.7189 152.4824  1.00271659 38328
0 GSAT 14
1 39498U 14001A   24099.72965900 -.00000085  00000-0  00000-0 0  9995
2 39498   0.0414  82.4942 0003546 221.2238 230.4164  1.00271530 37370
0 THAICOM 6
1 39500U 14002A   24100.10016843 -.00000129  00000-0  00000+0 0  9996
2 39500   0.0322 143.6897 0001638 292.2041 236.5290  1.00268852 37506
0 TDRS 12
1 39504U 14004A   24100.19997895 -.00000277  00000-0  00000+0 0  9990
2 39504   3.5529   1.7279 0001818 212.7524  14.5224  1.00274874 37432
0 ABS 2
1 39508U 14006A   24099.95623312 -.00000094  00000-0  00000-0 0  9995
2 39508   0.0626  80.9597 0002409 299.0040 236.8791  1.00271279 37261
0 ATHENA-FIDUS
1 39509U 14006B   24100.16360786  .00000146  00000-0  00000+0 0  9991
2 39509   0.0165 204.3849 0001534 183.4980 266.6956  1.00273108 37480
0 TURKSAT 4A
1 39522U 14007A   24100.45073959  .00000138  00000-0  00000+0 0  9993
2 39522   0.0502 260.8591 0002941  63.1885  78.3938  1.00272524 37201
0 EXPRESS AT1
1 39612U 14010A   24100.16726281  .00000071  00000-0  00000+0 0  9994
2 39612   0.0157 239.2091 0000210  88.1956 346.6943  1.00271453 37033
0 EXPRESS AT2
1 39613U 14010B   24099.91507788 -.00000289  00000-0  00000-0 0  9997
2 39613   0.0162 255.8829 0000562  55.8192 355.2116  1.00272775 36480
0 AMAZONAS 4A
1 39616U 14011A   24100.14141709 -.00000260  00000-0  00000+0 0  9994
2 39616   0.0616 128.2359 0002639 272.2563 134.2570  1.00272949 36855
0 ASTRA 5B
1 39617U 14011B   24099.98382601  .00000129  00000-0  00000+0 0  9995
2 39617   0.0667 311.0702 0003372  42.5545 221.7560  1.00272817 36854
0 IRNSS 1B
1 39635U 14017A   24099.52260053  .00000078  00000-0  00000+0 0  9990
2 39635  29.1199 253.9979 0021367 179.5293   6.8713  1.00272072 36766
0 LUCH 5V
1 39727U 14023A   24100.53288892 -.00000274  00000-0  00000+0 0  9993
2 39727   3.1277  76.4184 0001834 308.7464  99.6587  1.00273620554502
0 KAZSAT 3
1 39728U 14023B   24100.52566604  .00000053  00000-0  00000+0 0  9997
2 39728   0.0127 155.2281 0000412 347.2295 303.5130  1.00272085 36430
0 EUTE 3B
1 39773U 14030A   24099.91229951  .00000012  00000-0  00000-0 0  9992
2 39773   0.0664  12.1890 0001464 190.6455 326.3184  1.00270983 36054
0 USA 254
1 40100U 14043B   24099.69514847 -.00000012  00000-0  00000+0 0  9998
2 40100   2.6344  86.0237 0002681 289.9870 273.9568  0.99130604 19644
0 ASIASAT 8
1 40107U 14046A   24100.15534072 -.00000041  00000-0  00000+0 0  9996
2 40107   0.0173 153.2600 0002078 226.9696 229.6787  1.00271736 35487
0 ASIASAT 6
1 40141U 14052A   24099.96519646 -.00000370  00000-0  00000-0 0  9993
2 40141   0.0409  78.5513 0000788 287.4371 299.0839  1.00271294 35058
0 OPTUS 10
1 40146U 14054A   24100.38609997 -.00000155  00000-0  00000+0 0  9998
2 40146   0.0282 178.6308 0002067 206.3678 108.1209  1.00270732 35112
0 MEASAT 3B
1 40147U 14054B   24100.46066016 -.00000249  00000-0  00000+0 0  9991
2 40147   0.0430  49.3095 0003429 359.3833  46.8299  1.00269526 35161
0 LUCH (OLYMP)
1 40258U 14058A   24099.94214718 -.00000149  00000-0  00000-0 0  9992
2 40258   0.0643  79.5015 0001763 264.5689 174.6696  1.00271545 32984
0 HIMAWARI 8
1 40267U 14060A   24099.91473066 -.00000284  00000-0  00000-0 0  9995
2 40267   0.0256 169.9688 0000858 238.3809 259.2469  1.00271457 34747
0 IRNSS 1C
1 40269U 14061A   24100.53053931 -.00000164  00000-0  00000+0 0  9990
2 40269   4.4988 106.4921 0019694 359.3582   6.3623  1.00268994 34829
0 INTELSAT 30
1 40271U 14062A   24099.92241119 -.00000158  00000-0  00000-0 0  9995
2 40271   0.0126 235.9948 0001062  89.0521 109.6297  1.00270653 34576
0 ARSAT 1
1 40272U 14062B   24100.52618817 -.00000267  00000-0  00000+0 0  9998
2 40272   0.0282 300.2055 0001528  66.3859 309.2709  1.00271317 34882
0 EXPRESS AM-6
1 40277U 14064A   24099.92966015  .00000090  00000-0  00000-0 0  9996
2 40277   0.0146 256.6483 0000227 202.9459 125.7371  1.00271168 33568
0 GSAT 16
1 40332U 14078A   24100.16700175  .00000077  00000-0  00000+0 0  9999
2 40332   0.0138 203.7399 0005671 230.0386 239.3063  1.00270328 34242
0 DIRECTV 14
1 40333U 14078B   24100.53286451 -.00000133  00000-0  00000+0 0  9991
2 40333   0.0323 281.9109 0000269 279.4997  89.4305  1.00271745 34235
0 YAMAL 401
1 40345U 14082A   24099.95919191 -.00000236  00000-0  00000-0 0  9991
2 40345   0.0154 216.3138 0000564 176.9799 239.6592  1.00270316 33906
0 ASTRA 2G
1 40364U 14089A   24100.16169337  .00000142  00000-0  00000+0 0  9998
2 40364   0.0528  27.0993 0004137  14.6288 242.5345  1.00270501 34006
0 FENGYUN 2G
1 40367U 14090A   24100.03272595 -.00000308  00000-0  00000+0 0  9995
2 40367   3.6137  82.2958 0002269  84.4464 142.4714  1.00274999 34028
0 MUOS 3
1 40374U 15002A   24099.86654272 -.00000134  00000-0  00000-0 0  9999
2 40374   3.1448  33.7130 0052641 180.9906 279.4270  1.00272111 33572
0 INMARSAT 5-F2
1 40384U 15005A   24100.07345249 -.00000293  00000-0  00000+0 0  9995
2 40384   0.0180 203.4866 0000438  74.0360 251.7079  1.00266704 33669
0 ABS 3A
1 40424U 15010A   24099.91229951 -.00000034  00000-0  00000-0 0  9991
2 40424   0.0132 242.4245 0001597  43.3556 237.3433  1.00272332 33363
0 EUTE 115 WEST B(SATMEX 7
1 40425U 15010B   24100.54079755 -.00000033  00000-0  00000+0 0  9997
2 40425   0.0119  37.7081 0000394  57.4681 182.8777  1.00271117 33423
0 EXPRESS AM-7
1 40505U 15012A   24100.52201109  .00000143  00000-0  00000+0 0  9997
2 40505   0.0148 202.3983 0003346 162.7856  60.9628  1.00270526 33355
0 IRNSS 1D
1 40547U 15018A   24099.86099817 -.00000310  00000-0  00000+0 0  9991
2 40547  27.6318 253.9018 0021833 182.5432 183.4420  1.00266915 33232
0 BD-17
1 40549U 15019A   24100.53288892 -.00000168  00000-0  00000+0 0  9995
2 40549  50.6983 304.9954 0036192 191.6385 348.2428  1.00263726 33052
0 THOR 7
1 40613U 15022A   24100.29910201 -.00000016  00000-0  00000+0 0  9992
2 40613   0.0116 209.8346 0002472 165.5374 289.6664  1.00272796 32798
0 TURKMENALEM 52E/MONACOSA
1 40617U 15023A   24100.16621854  .00000096  00000-0  00000+0 0  9993
2 40617   0.0170 131.5542 0002255 237.7661 300.4125  1.00270537 32819
0 DIRECTV 15
1 40663U 15026A   24100.53014118 -.00000110  00000-0  00000+0 0  9995
2 40663   0.0411  84.7849 0001030 127.0533  74.4031  1.00269667 32520
0 SKY MEXICO-1
1 40664U 15026B   24100.49834095 -.00000242  00000-0  00000+0 0  9991
2 40664   0.0267 261.7366 0002095 131.5124 265.6047  1.00272131 32520
0 METEOSAT 11 (MSG 4)
1 40732U 15034A   24100.15795139  .00000056  00000-0  00000+0 0  9996
2 40732   1.2215  67.0988 0000963 237.1839 319.7557  1.00276809 32003
0 STARONE C4
1 40733U 15034B   24099.92742868 -.00000273  00000-0  00000-0 0  9993
2 40733   0.0466  17.1029 0002488   4.0395  80.3898  1.00272182 32013
0 WGS 7 (USA 263)
1 40746U 15036A   24099.76137265  .00000009  00000-0  00000-0 0  9996
2 40746   0.0178 236.3342 0000275 247.0537 163.1809  1.00270555 31854
0 INTELSAT 34
1 40874U 15039A   24099.93030042 -.00000294  00000-0  00000+0 0  9991
2 40874   0.0369 311.0587 0000447 331.7496 194.2504  1.00271135 31601
0 EUTE 8 WEST B
1 40875U 15039B   24099.92549441 -.00000074  00000-0  00000-0 0  9996
2 40875   0.0684   8.2473 0003984  10.1758 144.4294  1.00271027 31635
0 GSAT 6
1 40880U 15041A   24100.10095163 -.00000167  00000-0  00000+0 0  9997
2 40880   0.0397 111.3550 0002224 267.1019 298.1771  1.00268867 31572
0 INMARSAT 5-F3
1 40882U 15042A   24100.12341015  .00000042  00000-0  00000+0 0  9993
2 40882   0.0183 201.3192 0000283  39.8027 180.7573  1.00270754 31546
0 MUOS 4
1 40887U 15044A   24100.52905993 -.00000099  00000-0  00000+0 0  9998
2 40887   3.0193  30.4620 0058861 358.7188  74.4938  1.00271280 31547
0 TJS-1
1 40892U 15046A   24100.50720890 -.00000164  00000-0  00000+0 0  9998
2 40892   0.0670  36.4310 0003413  14.5150 124.8930  1.00269651 31542
0 EXPRESS AM-8
1 40895U 15048A   24100.00743990 -.00000120  00000-0  00000-0 0  9997
2 40895   0.0152 249.7173 0000140 123.9557 172.7424  1.00270106 31330
0 BD-20
1 40938U 15053A   24099.86286955 -.00000155  00000-0  00000-0 0  9993
2 40938  50.4683 268.2076 0050001 187.4442 149.0396  1.00278163 31339
0 SKY MUSTER
1 40940U 15054A   24099.96919949 -.00000287  00000-0  00000-0 0  9990
2 40940   0.0139 250.1134 0001738 125.5102 311.2493  1.00271166 31240
0 ARSAT 2
1 40941U 15054B   24100.49790583 -.00000232  00000-0  00000+0 0  9992
2 40941   0.0319 290.7574 0001803  82.4457 283.2547  1.00271452 31297
0 MORELOS 3
1 40946U 15056A   24100.51941208 -.00000054  00000-0  00000+0 0  9996
2 40946   3.0594 350.1789 0002980   1.8166 280.1190  1.00272578 31276
0 APSTAR 9
1 40982U 15059A   24100.18521494 -.00000273  00000-0  00000+0 0  9996
2 40982   0.0559  88.3037 0000785 181.9879 136.2925  1.00272470 31146
0 TURKSAT 4B
1 40984U 15060A   24100.52401262  .00000107  00000-0  00000+0 0  9993
2 40984   0.0261 104.6789 0002135 267.1151  65.0818  1.00273718 31084
0 CHINASAT 2C
1 41021U 15063A   24100.48371066 -.00000331  00000-0  00000+0 0  9992
2 41021   0.0023  25.9975 0001704   9.6762  80.0875  1.00268523 31003
0 GSAT 15
1 41028U 15065A   24099.93591245 -.00000266  00000-0  00000-0 0  9999
2 41028   0.0549  83.6149 0002620 312.2696 232.2146  1.00271695 30784
0 BADR 7
1 41029U 15065B   24100.51922637  .00000137  00000-0  00000+0 0  9995
2 41029   0.0466 242.3983 0002854   6.1947 162.5722  1.00273825 30906
0 LAOSAT 1
1 41034U 15067A   24099.96684990 -.00000351  00000-0  00000-0 0  9996
2 41034   0.0278 316.8997 0002277  24.7582 332.6005  1.00269174 30844
0 TELSTAR 12V
1 41036U 15068A   24100.00743990 -.00000127  00000-0  00000-0 0  9994
2 41036   0.0192 211.6059 0002467 159.2573 174.5600  1.00269917 30611
0 CHINASAT 1C
1 41103U 15073A   24099.93064851 -.00000213  00000-0  00000-0 0  9992
2 41103   5.0653  78.1041 0003402 158.9869 242.0879  0.99167643 30624
0 ELEKTRO-L 2
1 41105U 15074A   24099.79524628 -.00000121  00000-0  00000+0 0  9999
2 41105   4.6653  80.2729 0002045 216.6760 172.3419  1.00270023 30504
0 COSMOS 2513
1 41121U 15075A   24100.53001716 -.00000146  00000-0  00000+0 0  9998
2 41121   0.0594  99.7497 0002883 288.4111  81.1501  1.00271769 30596
0 EXPRESS AMU1
1 41191U 15082A   24100.16325977  .00000148  00000-0  00000+0 0  9991
2 41191   0.0318  92.4987 0002939 267.0676 293.1866  1.00273286 30496
0 GAOFEN 4
1 41194U 15083A   24099.96232471 -.00000343  00000-0  00000-0 0  9995
2 41194   0.0523  60.0673 0002298  93.4147 136.3435  1.00269364 30517
0 BELINTERSAT 1
1 41238U 16001A   24100.52427368  .00000099  00000-0  00000+0 0  9997
2 41238   0.0090 324.1149 0001166 260.2074 214.1068  1.00272599 30222
0 IRNSS 1E
1 41241U 16003A   24099.86050633 -.00000286  00000-0  00000-0 0  9994
2 41241  31.4774  74.5108 0018944 185.6417 358.9480  1.00277795 30067
0 INTELSAT 29E
1 41308U 16004A   24099.95605910 -.00000162  00000-0  00000-0 0  9996
2 41308   4.4818  79.8453 0007073 356.8797 178.9669  1.00633252 30237
0 EUTE 9B
1 41310U 16005A   24100.30110354  .00000054  00000-0  00000+0 0  9993
2 41310   0.0616  63.6330 0003094 275.7278 336.0405  1.00271059 30109
0 SES 9
1 41380U 16013A   24100.05535321 -.00000352  00000-0  00000+0 0  9998
2 41380   0.0195 279.9183 0001940 120.2554 285.8243  1.00265448 29749
0 EUTE 65W
1 41382U 16014A   24100.14315755 -.00000284  00000-0  00000+0 0  9992
2 41382   0.0565  12.8224 0001993   5.2168 166.1501  1.00268694 29635
0 IRNSS 1F
1 41384U 16015A   24100.16256359  .00000154  00000-0  00000+0 0  9996
2 41384   3.4060 120.3945 0018445 191.6725 336.7848  1.00272163 29644
0 BEIDOU IGSO-6
1 41434U 16021A   24100.31807295 -.00000175  00000-0  00000+0 0  9991
2 41434  59.0827  46.0434 0049028 225.1955 136.8105  1.00283561 29463
0 IRNSS 1G
1 41469U 16027A   24099.96711095 -.00000339  00000-0  00000+0 0  9997
2 41469   3.2854 121.6534 0005481 312.0702 241.7022  1.00273240 29157
0 JCSAT 2B
1 41471U 16028A   24100.50720858 -.00000173  00000-0  00000+0 0  9997
2 41471   0.0163 161.7786 0000507 197.8398 175.2002  1.00273136 29081
0 THAICOM 8
1 41552U 16031A   24099.74632660 -.00000130  00000-0  00000-0 0  9991
2 41552   0.0591  31.1889 0003944 353.0989 160.3940  1.00271945 28700
0 INTELSAT 31
1 41581U 16035A   24100.49512111 -.00000159  00000-0  00000+0 0  9990
2 41581   0.0096 226.0662 0002005 331.5011  83.8425  1.00272343 28687
0 BD-2-G7
1 41586U 16037A   24099.96328196 -.00000361  00000-0  00000-0 0  9990
2 41586   1.7409  68.7008 0009065 322.6499 263.6370  1.00274307 28776
0 ABS 2A
1 41588U 16038A   24100.09632769 -.00000091  00000-0  00000+0 0  9993
2 41588   0.0232 220.2107 0001739 100.2314 346.7736  1.00267350 28673
0 EUTE 117W B
1 41589U 16038B   24100.51573868 -.00000019  00000-0  00000+0 0  9995
2 41589   0.0121  56.0476 0000218  17.1347 193.7269  1.00271187 28694
0 BRISAT
1 41591U 16039A   24099.89841097 -.00000205  00000-0  00000-0 0  9997
2 41591   0.0173 240.3894 0002071 133.0731 298.0922  1.00270570 28599
0 ECHOSTAR 18
1 41592U 16039B   24100.14394075 -.00000290  00000-0  00000+0 0  9991
2 41592   0.0175 198.5531 0001443 169.0501 180.7388  1.00267957 28600
0 TIANTONG-1 1
1 41725U 16048A   24100.58047875 -.00000325  00000-0  00000+0 0  9999
2 41725   2.5549  27.3125 0006512 337.7799 143.5770  1.00273538 28258
0 JCSAT 16
1 41729U 16050A   24099.89839299 -.00000259  00000-0  00000-0 0  9993
2 41729   0.0200 224.8273 0001797 212.2621 227.9047  1.00270269 28058
0 USA 270
1 41744U 16052A   24090.96716365 -.00000334  00000-0  00000+0 0  9992
2 41744   2.1149  87.4714 0002649 219.7213 356.7010  1.00349673 10631
0 USA 271
1 41745U 16052B   24097.28208110 -.00000286  00000-0  00000+0 0  9995
2 41745   2.1055  87.3740 0010189  13.6528 142.9434  0.99953200 19889
0 INTELSAT 36
1 41747U 16053A   24100.52766757 -.00000032  00000-0  00000+0 0  9995
2 41747   0.0118 245.5392 0000929  99.3658 111.8094  1.00269300 27931
0 INTELSAT 33E
1 41748U 16053B   24100.52601413  .00000041  00000-0  00000+0 0  9992
2 41748   0.0150 244.7462 0001455 164.5263  38.3319  1.00272917 28212
0 INSAT 3DR
1 41752U 16054A   24099.74127043 -.00000085  00000-0  00000-0 0  9991
2 41752   0.1152  87.6356 0011313 189.6923 261.0015  1.00271161 27803
0 GSAT 18
1 41793U 16060A   24099.74127043 -.00000085  00000-0  00000-0 0  9993
2 41793   0.0638  84.4107 0001229  46.6453  47.2697  1.00271475 27516
0 SKY MUSTER 2
1 41794U 16060B   24100.37619889 -.00000252  00000-0  00000+0 0  9995
2 41794   0.0135 249.1766 0001416 127.8899 101.2808  1.00271710 27566
0 HIMAWARI 9
1 41836U 16064A   24100.55323168 -.00000283  00000-0  00000+0 0  9992
2 41836   0.0123 194.7945 0000923 186.0867 157.3195  1.00269068 27292
0 SJ-17
1 41838U 16065A   24100.02054648 -.00000061  00000-0  00000-0 0  9996
2 41838   3.7112  82.1808 0006432 327.2019 323.0645  1.00221890 27191
0 GOES 16
1 41866U 16071A   24100.14115604 -.00000256  00000-0  00000+0 0  9992
2 41866   0.0130 272.2014 0000535 119.1721 142.0995  1.00271881 27089
0 TIANLIAN 1-04
1 41869U 16072A   24100.56840417  .00000025  00000-0  00000+0 0  9992
2 41869   0.1739 272.7881 0028388 237.4121  69.6029  1.00276241 27299
0 WGS 8 (USA 272)
1 41879U 16075A   24100.38141012 -.00000210  00000-0  00000+0 0  9998
2 41879   0.0141 149.5698 0000212 288.6972  46.9441  1.00270507 26914
0 FENGYUN 4A
1 41882U 16077A   24100.10182185 -.00000209  00000-0  00000+0 0  9998
2 41882   0.2835  82.0822 0008507 345.5597 253.8908  1.00264864 26962
0 ECHOSTAR 19
1 41893U 16079A   24100.53539031 -.00000146  00000-0  00000+0 0  9995
2 41893   0.0175 179.1143 0001163 185.3297 289.4272  1.00270601 26720
0 JCSAT 15
1 41903U 16082A   24100.52665553 -.00000358  00000-0  00000+0 0  9994
2 41903   0.0168 202.8127 0000914 144.3550 150.7384  1.00268122 26774
0 STARONE D1
1 41904U 16082B   24100.49729668 -.00000218  00000-0  00000+0 0  9994
2 41904   0.0200 262.1125 0002385 118.7786 272.3510  1.00269624 26779
0 TJS-2
1 41911U 17001A   24100.52723244 -.00000014  00000-0  00000+0 0  9996
2 41911   0.3980  83.8000 0002310 240.7088 130.0238  1.00273170 26716
0 SBIRS GEO 4 (USA 273)
1 41937U 17004A   24099.92206310 -.00000153  00000-0  00000-0 0  9993
2 41937   1.4390 352.8497 0002221  18.2700  61.6632  1.00270942 26597
0 HISPASAT 36W-1
1 41942U 17006A   24100.43533660 -.00000252  00000-0  00000+0 0  9990
2 41942   0.0160  36.9568 0000972 347.0562 294.8731  1.00272392 26526
0 TELKOM 3S
1 41944U 17007A   24100.04980135 -.00000370  00000-0  00000+0 0  9994
2 41944   0.0121 220.6414 0002253 154.2623 318.8019  1.00270436 26205
0 INTELSAT 32E
1 41945U 17007B   24100.14750868 -.00000277  00000-0  00000+0 0  9990
2 41945   0.0184 164.4894 0000407 110.1209 293.1907  1.00270427 26236
0 ECHOSTAR 23
1 42070U 17014A   24100.33730003 -.00000065  00000-0  00000+0 0  9993
2 42070   0.0188 204.1032 0001948 165.7051 199.7842  1.00270511 25937
0 WGS 9 (USA 275)
1 42075U 17016A   24100.15368728 -.00000104  00000-0  00000+0 0  9995
2 42075   0.0227 206.9820 0000512  71.9622 322.2858  1.00270450 25919
0 SES 10
1 42432U 17017A   24099.99965737 -.00000281  00000-0  00000+0 0  9991
2 42432   0.0155  40.0062 0002549 341.7744 108.9230  1.00270284 25712
0 CHINASAT 16
1 42662U 17018A   24099.91367941 -.00000360  00000-0  00000-0 0  9991
2 42662   0.0747  86.9759 0003817 207.8477 342.2508  1.00267977 25716
0 KOREASAT 7
1 42691U 17023A   24099.96441325 -.00000370  00000-0  00000-0 0  9994
2 42691   0.0396  21.4854 0001280 356.2160 283.0821  1.00271878 25364
0 SGDC
1 42692U 17023B   24099.65961617 -.00000258  00000-0  00000+0 0  9996
2 42692   0.0517  74.8089 0002406 313.4491 331.7484  1.00271518 25430
0 GSAT 9
1 42695U 17024A   24100.09077035 -.00000295  00000-0  00000+0 0  9994
2 42695   0.0231 139.5770 0002495 233.8326 314.4153  1.00275132 25417
0 INMARSAT 5-F4
1 42698U 17025A   24100.09573028  .00000068  00000-0  00000+0 0  9998
2 42698   0.0197 214.8130 0000231  74.0814 359.8942  1.00269281 25249
0 SES 15
1 42709U 17026A   24100.50516399  .00000052  00000-0  00000+0 0  9999
2 42709   0.0120  79.2894 0000345 244.7434 286.8939  1.00269864 26173
0 VIASAT 2
1 42740U 17029A   24100.17827023 -.00000273  00000-0  00000+0 0  9995
2 42740   0.0191 121.2422 0000838 257.0755 173.8316  1.00266869 25165
0 EUTE 172B
1 42741U 17029B   24099.83234287 -.00000014  00000-0  00000+0 0  9993
2 42741   0.0463 165.1691 0000258  37.0504 106.9854  1.00270373 25694
0 GSAT 19
1 42747U 17031A   24100.16560939  .00000117  00000-0  00000+0 0  9990
2 42747   0.0823  84.1277 0005710 321.6128 259.6491  1.00272525 25113
0 ECHOSTAR 21
1 42749U 17032A   24100.12965948  .00000054  00000-0  00000+0 0  9998
2 42749   2.6568 318.4403 0002229  61.4327 234.9286  1.00273689 25104
0 CHINASAT 9A
1 42763U 17035A   24099.91971351 -.00000003  00000-0  00000-0 0  9992
2 42763   2.9203  85.1054 0006069 299.1254  35.9184  0.99777822 25677
0 BULGARIASAT-1
1 42801U 17038A   24100.15647201  .00000003  00000-0  00000+0 0  9999
2 42801   0.0796  90.7214 0002677 289.2183 236.1674  1.00270934 24895
0 HELLAS-SAT 3
1 42814U 17040A   24100.56646753  .00000145  00000-0  00000+0 0  9999
2 42814   0.0621  20.9522 0005086 358.3345  61.9047  1.00270836 24853
0 GSAT 17
1 42815U 17040B   24100.31781186 -.00000265  00000-0  00000+0 0  9997
2 42815   0.0824  87.1084 0003954 215.5783 103.2610  1.00271593 24867
0 INTELSAT 35E
1 42818U 17041A   24099.99355152 -.00000246  00000-0  00000-0 0  9992
2 42818   0.0181 183.9457 0002055 189.9860 146.9659  1.00271833 24804
0 COSMOS 2520
1 42907U 17046A   24100.16508725  .00000129  00000-0  00000+0 0  9992
2 42907   0.0288 115.8368 0000170 101.5847  84.9166  1.00275760 24376
0 TDRS 13
1 42915U 17047A   24100.36854603 -.00000112  00000-0  00000+0 0  9996
2 42915   3.6983 343.2114 0015477 106.7103 229.3299  1.00271550 24375
0 QZS-3
1 42917U 17048A   24100.50658677 -.00000355  00000-0  00000+0 0  9997
2 42917   0.0561 206.4964 0001776 176.8429 124.2729  1.00274253 24362
0 AMAZONAS 5
1 42934U 17053A   24095.59720425 -.00000290  00000-0  00000-0 0  9994
2 42934   0.0424 323.3051 0004621  31.9666 352.1050  1.00271950 23988
0 ASIASAT 9
1 42942U 17057A   24100.39517494 -.00000367  00000-0  00000+0 0  9996
2 42942   0.0205 223.1450 0002284 151.9378  87.3988  1.00268483 23939
0 INTELSAT 37E
1 42950U 17059A   24100.15246897 -.00000149  00000-0  00000+0 0  9995
2 42950   0.0017 191.9743 0002014 192.5747 210.2101  1.00270912 24034
0 BSAT-4A
1 42951U 17059B   24100.05535321 -.00000358  00000-0  00000+0 0  9990
2 42951   0.0769  62.9785 0004641 333.3310 291.2396  1.00272211 23949
0 ECHOSTAR 105/SES 11
1 42967U 17063A   24099.65164529 -.00000097  00000-0  00000-0 0  9994
2 42967   0.0380  54.9634 0002362 290.5892 341.4608  1.00271081 23708
0 KOREASAT 5A
1 42984U 17067A   24099.96380410 -.00000366  00000-0  00000-0 0  9999
2 42984   0.0151 272.9409 0001360 104.1647 280.7057  1.00270987 23516
0 ALCOMSAT 1
1 43039U 17078A   24100.15116363 -.00000194  00000-0  00000+0 0  9999
2 43039   0.0393 279.9482 0001676  86.3931 221.1645  1.00273210 23267
0 ANGOSAT 1
1 43087U 17086A   24099.56123858  .00000044  00000-0  00000-0 0  9994
2 43087   5.5596  76.3616 0015936 154.3832  57.3996  0.99376464 22789
0 SBIRS GEO 3 (USA 282)
1 43162U 18009A   24100.12444771  .00000117  00000-0  00000+0 0  9999
2 43162   2.4437 333.1456 0001758  43.9357  66.1600  1.00272083 22821
0 AL YAH 3
1 43174U 18012A   24099.98799444 -.00000163  00000-0  00000-0 0  9990
2 43174   0.0781 271.0959 0001936 104.4552 157.7659  1.00271254 22837
0 SES 14
1 43175U 18012B   24100.43298699 -.00000286  00000-0  00000+0 0  9995
2 43175   0.0175 300.9247 0000725  71.1146 294.4900  1.00270541 23030
0 GOES 17
1 43226U 18022A   24100.13532552 -.00000099  00000-0  00000+0 0  9991
2 43226   0.0160  35.7487 0000946 304.9295 161.2199  1.00270669 22421
0 HISPASAT 30W-6
1 43228U 18023A   24100.15011935 -.00000224  00000-0  00000+0 0  9993
2 43228   0.0218  52.1771 0005743 333.5596 196.1740  1.00269172 22480
0 SUPERBIRD 8
1 43271U 18033A   24099.83034135 -.00000100  00000-0  00000+0 0  9994
2 43271   0.0181 222.3095 0001067 142.4404 293.7106  1.00271866 22020
0 HYLAS 4
1 43272U 18033B   24100.43577171 -.00000241  00000-0  00000+0 0  9992
2 43272   0.0704 268.1917 0000853 108.0043 305.3432  1.00269404 22052
0 IRNSS 1I
1 43286U 18035A   24098.03243834  .00000106  00000-0  00000-0 0  9997
2 43286  28.9464  86.7475 0019447 187.2220 348.6686  1.00277380 22049
0 USA 283
1 43339U 18036A   24097.46241206 -.00000327  00000-0  00000+0 0  9999
2 43339   0.0243 189.7137 0000180 306.3289 359.7427  1.00269556 10708
0 COSMOS 2526
1 43432U 18037A   24099.96676286 -.00000353  00000-0  00000-0 0  9991
2 43432   0.0287 122.4222 0000207 256.5048 294.8589  1.00269950 21953
0 APSTAR 6C
1 43450U 18041A   24100.20188072 -.00000326  00000-0  00000+0 0  9996
2 43450   0.0204 287.1108 0001413  31.6305  85.8587  1.00269257 21835
0 BANGABANDHUSAT-1
1 43463U 18044A   24099.96502241 -.00000371  00000-0  00000-0 0  9992
2 43463   0.0025 172.3543 0002336 205.6896 286.1547  1.00271120 21699
0 SES 12
1 43488U 18049A   24100.53297594 -.00000277  00000-0  00000+0 0  9996
2 43488   0.0453  49.3539 0001676 253.6959 182.0694  1.00270260 21408
0 FENGYUN 2H
1 43491U 18050A   24100.52975611 -.00000135  00000-0  00000+0 0  9997
2 43491   0.9653  87.6927 0000872  12.2860   8.0815  1.00274673 21501
0 BEIDOU IGSO-7
1 43539U 18057A   24099.59392338 -.00000114  00000-0  00000-0 0  9998
2 43539  55.1341 169.9764 0073528 232.4087 121.8913  1.00273830 21112
0 TELSTAR 19V
1 43562U 18059A   24099.49927847 -.00000288  00000-0  00000-0 0  9996
2 43562   0.0189 195.9581 0001364 249.1709 228.8432  1.00272083 21122
0 TELKOM-4
1 43587U 18064A   24099.96284684 -.00000352  00000-0  00000-0 0  9998
2 43587   0.0084 227.5250 0001361 173.3639 251.4288  1.00270385 20869
0 TELSTAR 18V
1 43611U 18069A   24099.91507788 -.00000302  00000-0  00000-0 0  9994
2 43611   0.0176 208.4719 0001704 166.0417 290.5621  1.00268474 20615
0 AZERSPACE 2/INTELSAT 38
1 43632U 18074A   24100.52305536  .00000129  00000-0  00000+0 0  9995
2 43632   0.0190 179.5613 0001284 198.9122  53.1714  1.00271321 20436
0 HORIZONS 3E
1 43633U 18074B   24099.76015434 -.00000039  00000-0  00000-0 0  9996
2 43633   0.0152 227.8567 0001019  91.6845 320.5972  1.00268988 20327
0 AEHF 4 (USA 288)
1 43651U 18079A   24099.92795081 -.00000285  00000-0  00000-0 0  9995
2 43651   1.7662 329.4808 0003479  35.0013  99.7253  1.00270600 20285
0 BEIDOU 3G1
1 43683U 18085A   24100.18397115 -.00000283  00000-0  00000+0 0  9991
2 43683   2.0231 112.3790 0003381 340.0743 311.6955  1.00274963 20041
0 GSAT 29
1 43698U 18089A   24099.64460477  .00000077  00000-0  00000-0 0  9992
2 43698   0.0388 257.2573 0000445  76.2287 150.9927  1.00267771 19775
0 ES'HAIL 2
1 43700U 18090A   24100.16125826  .00000136  00000-0  00000+0 0  9999
2 43700   0.0185 205.3572 0001111 192.4325 243.9494  1.00270224 19906
0 GEO-KOMPSAT-2A
1 43823U 18100A   24100.50658697 -.00000352  00000-0  00000+0 0  9993
2 43823   0.0519  51.3703 0001887  12.4061  85.0732  1.00270226 19650
0 GSAT 11
1 43824U 18100B   24100.17083074 -.00000084  00000-0  00000+0 0  9990
2 43824   0.0183 207.9352 0006845 210.8874 274.5745  1.00269681 19610
0 GSAT 7A
1 43864U 18105A   24100.52653626  .00000018  00000-0  00000+0 0  9990
2 43864   0.0394 104.1838 0002159 280.8478  65.5740  1.00273353 19441
0 COSMOS 2533
1 43867U 18107A   24099.95527588 -.00000047  00000-0  00000-0 0  9990
2 43867   0.0348  92.3417 0000501 149.6375   9.5982  1.00273207 19412
0 TJS-3
1 43874U 18110A   24100.02425366 -.00000006  00000-0  00000-0 0  9999
2 43874   0.4752  88.9478 0000955 226.1929  64.4144  1.00266787 19404
0 CHINASAT 2D
1 43920U 19001A   24099.96719799 -.00000346  00000-0  00000-0 0  9994
2 43920   0.0669  76.0758 0001482 158.3652  81.4383  1.00268519 46064
0 SAUDIGEOSAT1/HELLASSAT4
1 44034U 19007A   24100.29771628  .00000145  00000-0  00000+0 0  9990
2 44034   0.0710 124.2637 0003577  15.2741 204.6946  1.00273401 19348
0 GSAT 31
1 44035U 19007B   24100.16560939  .00000117  00000-0  00000+0 0  9997
2 44035   0.0388  78.2942 0003698 236.4357 350.6392  1.00271800 18944
0 NUSANTARA SATU
1 44048U 19009A   24099.82720854 -.00000243  00000-0  00000+0 0  9998
2 44048   0.0352 130.5172 0002615 320.9910 189.8428  1.00269368 18744
0 S5
1 44065U 19009D   24099.58534385 -.00000091  00000-0  00000-0 0  9994
2 44065   4.5131  80.2542 0001179  93.0258 244.7774  0.99341011 18496
0 CHINASAT 6C
1 44067U 19012A   24099.93797133 -.00000344  00000-0  00000-0 0  9992
2 44067   0.0883  92.2161 0000483 248.3256 325.2203  1.00269326 18722
0 WGS 10 (USA 291)
1 44071U 19014A   24100.16813304  .00000039  00000-0  00000+0 0  9995
2 44071   0.0170 215.3255 0000273  94.5752   8.7984  1.00270712 18592
0 TIANLIAN 2-01
1 44076U 19017A   24100.17143990 -.00000114  00000-0  00000+0 0  9996
2 44076   0.0943 263.3758 0003945 105.4543 327.8416  1.00270482 18594
0 ARABSAT 6A
1 44186U 19021A   24099.94744385  .00000146  00000-0  00000-0 0  9995
2 44186   0.0534 347.3834 0001833  28.4062 193.4872  1.00279104 18148
0 BEIDOU 3 IGSO-1
1 44204U 19023A   24100.46822942 -.00000232  00000-0  00000+0 0  9990
2 44204  57.5465  45.9414 0019483 215.1631 225.0384  1.00269367 18354
0 BEIDOU 2 G8
1 44231U 19027A   24100.18484139 -.00000254  00000-0  00000+0 0  9999
2 44231   1.7784  73.4098 0003104 181.9488 153.5072  1.00276035 18063
0 YAMAL 601
1 44307U 19031A   24100.16587045  .00000112  00000-0  00000+0 0  9998
2 44307   0.0174 186.4072 0002340 192.3005 287.8964  1.00271988 17906
0 AT&T T-16
1 44333U 19034A   24100.13602170 -.00000123  00000-0  00000+0 0  9997
2 44333   0.0182 212.9242 0000984  14.1669 278.8815  1.00268637 17617
0 EUTE 7C
1 44334U 19034B   24100.00188256  .00000040  00000-0  00000-0 0  9996
2 44334   0.0703 193.4966 0003144 169.7317 202.1736  1.00270637 18099
0 BEIDOU 3 IGSO-2
1 44337U 19035A   24100.54986310 -.00000107  00000-0  00000+0 0  9990
2 44337  55.1680 165.8703 0032960 207.2289 142.1667  1.00262194 17682
0 COSMOS 2539
1 44457U 19048A   24100.12965948  .00000074  00000-0  00000+0 0  9997
2 44457   0.0243 122.3330 0000180  36.0006  98.2237  1.00276311 17205
0 EDRS-C
1 44475U 19049A   24099.36586409  .00000146  00000-0  00000-0 0  9993
2 44475   0.0794 255.3044 0000834 121.5713 343.1202  1.00270367 17182
0 INTELSAT 39
1 44476U 19049B   24099.95465840  .00000025  00000-0  00000-0 0  9990
2 44476   0.0016 183.4375 0000953 329.3912  90.4768  1.00270987 17120
0 AMOS 17
1 44479U 19050A   24099.98243992  .00000101  00000-0  00000-0 0  9995
2 44479   0.0274 105.4230 0001490 296.7924 166.1637  1.00272944 17176
0 AEHF 5 (USA 292)
1 44481U 19051A   24100.52496986  .00000073  00000-0  00000+0 0  9992
2 44481   2.8580 306.9543 0048364   3.7761 130.9967  1.00269619 17248
0 EUTE 5W B
1 44624U 19067A   24100.15507965 -.00000050  00000-0  00000+0 0  9993
2 44624   0.0685  21.9153 0002849 359.9854 226.8046  1.00271964 16429
0 MEV-1
1 44625U 19067B   24099.43477772 -.00000210  00000-0  00000-0 0  9993
2 44625   0.0147 141.6076 0000764 247.4271 297.1518  1.00273098 14967
0 TJS-4
1 44637U 19070A   24100.53062632 -.00000176  00000-0  00000+0 0  9999
2 44637   0.0410 351.5633 0004004  58.8389  62.3444  1.00267565 16525
0 BEIDOU 3 IGSO-3
1 44709U 19073A   24099.92252236 -.00000182  00000-0  00000+0 0  9990
2 44709  56.1678 289.4529 0036468 179.5161 180.4434  1.00270664 16428
0 TIBA-1
1 44800U 19080A   24100.16317275  .00000148  00000-0  00000+0 0  9995
2 44800   0.0584  17.7998 0001845 359.3186 274.9926  1.00271053 16060
0 INMARSAT GX5
1 44801U 19080B   24100.12965948  .00000067  00000-0  00000+0 0  9998
2 44801   0.0586  17.9142 0002157   1.9079 235.7024  1.00274120 16077
0 JCSAT 18
1 44868U 19091A   24100.50359703 -.00000209  00000-0  00000+0 0  9990
2 44868   0.0413   5.4422 0000362 249.3479 274.7181  1.00270605 16082
0 ELEKTRO-L 3
1 44903U 19095A   24100.52914694 -.00000104  00000-0  00000+0 0  9996
2 44903   0.0535 313.1623 0000992  69.5483  81.9744  1.00272069 15689
0 SJ-20
1 44910U 19097A   24100.00981316 -.00000026  00000-0  00000-0 0  9998
2 44910   2.8627  84.6754 0003187 327.5498 217.0768  1.00270323 15685
0 TJS-5
1 44978U 20002A   24099.91855203 -.00000350  00000-0  00000-0 0  9992
2 44978   0.4103  84.2377 0003980 187.7609   3.8665  1.00272384 15686
0 GSAT 30
1 45026U 20005A   24099.78154403 -.00000173  00000-0  00000-0 0  9992
2 45026   0.0584  86.9894 0002257 350.8629 123.9968  1.00272744 15493
0 EUTE KONNECT
1 45027U 20005B   24100.37220098  .00000042  00000-0  00000+0 0  9998
2 45027   0.0674  86.9852 0000568 175.8984  76.4013  1.00271178 16304
0 JCSAT 17
1 45245U 20013A   24099.96841628 -.00000331  00000-0  00000-0 0  9992
2 45245   5.6316 349.3798 0003207  11.9632 320.9678  1.00271014 15210
0 GEO-KOMPSAT-2B
1 45246U 20013B   24099.96684990 -.00000352  00000-0  00000-0 0  9993
2 45246   0.0313 332.9429 0001325  13.6366 327.4373  1.00270694 15260
0 BEIDOU 3 G2
1 45344U 20017A   24100.17178799 -.00000148  00000-0  00000+0 0  9999
2 45344   2.2363  26.7936 0001186 318.0922 354.9087  1.00271248 15267
0 AEHF 6 (USA 298)
1 45465U 20022B   24099.96867736 -.00000314  00000-0  00000+0 0  9994
2 45465   4.3911 298.7729 0046135   7.8598  17.4492  1.00266557 14974
0 BEIDOU 3 G3
1 45807U 20040A   24099.96328196 -.00000361  00000-0  00000-0 0  9999
2 45807   0.6733  31.9265 0006837 300.5668 322.4600  1.00266322 14052
0 APSTAR 6D
1 45863U 20045A   24100.20188072 -.00000326  00000-0  00000+0 0  9993
2 45863   0.0592  81.9231 0002610 323.0925 359.5743  1.00271336 13846
0 KOREASAT 116
1 45920U 20048A   24099.96441325 -.00000370  00000-0  00000-0 0  9993
2 45920   0.0181 252.7655 0000835 119.3096 289.0012  1.00273266 13620
0 EXPRESS 103
1 45985U 20053A   24099.93591245 -.00000289  00000-0  00000-0 0  9993
2 45985   0.0126 248.4383 0000195 123.1835 259.4753  1.00271199 13630
0 EXPRESS 80
1 45986U 20053B   24100.22786281 -.00000144  00000-0  00000+0 0  9994
2 45986   0.0316 263.3186 0001158 301.4795 155.1899  1.00272744 13635
0 BSAT-4B
1 46112U 20056A   24099.67036693 -.00000358  00000-0  00000-0 0  9998
2 46112   0.0759 112.7963 0002766 336.1622  99.6021  1.00269378 13377
0 MEV-2
1 46113U 20056B   24099.58316828 -.00000019  00000-0  00000-0 0  9994
2 46113   0.0017 184.8663 0000911 190.1500  31.2752  1.00271846 72534
0 GALAXY 30
1 46114U 20056C   24100.50976236  .00000029  00000-0  00000+0 0  9994
2 46114   0.0181 165.7864 0001943 214.7543 236.2004  1.00272174 13395
0 GAOFEN 13
1 46610U 20071A   24099.96476135 -.00000371  00000-0  00000-0 0  9994
2 46610   0.4396  86.9744 0012854 273.7178 302.0098  1.00268491 12909
0 TIANTONG-1 2
1 46916U 20082A   24100.03777326 -.00000367  00000-0  00000+0 0  9996
2 46916   2.5176 293.3190 0000983  68.5134 334.4273  1.00275433 12636
0 LUCAS (JDRS-1)
1 47202U 20089A   24099.98652192 -.00000243  00000-0  00000+0 0  9999
2 47202   0.0174 129.5435 0003149 239.5217 274.5814  1.00270909 12387
0 XM-7
1 47240U 20096A   24099.56645993 -.00000213  00000-0  00000-0 0  9992
2 47240   0.0342 354.5220 0001639 187.2382 134.2292  1.00271473 12291
0 CMS-01
1 47256U 20099A   24099.90613637 -.00000173  00000-0  00000-0 0  9995
2 47256   0.0832  88.4371 0006180 198.6215 319.7774  1.00270085 12180
0 TURKSAT 5A
1 47306U 21001A   24099.94744385  .00000146  00000-0  00000-0 0  9993
2 47306   0.0325  95.4725 0000074 170.1127 304.0799  1.00270853 11958
0 TIANTONG-1 3
1 47321U 21003A   24099.95753847 -.00000166  00000-0  00000-0 0  9999
2 47321   3.0155 311.4091 0003248  52.3366 260.1492  1.00269857 11941
0 TJS-6
1 47613U 21010A   24099.90537414  .00000034  00000-0  00000+0 0  9998
2 47613   0.2016  83.8974 0001293   7.4841 250.6510  1.00269097 11780
0 SHIYAN 9 (SY-9)
1 47851U 21019A   24100.42639828 -.00000248  00000-0  00000+0 0  9991
2 47851  18.2440 312.6961 0012606  15.4404 170.3107  1.00267344 16123
0 SBIRS GEO 5 (USA 315)
1 48618U 21042A   24099.89077192  .00000100  00000-0  00000-0 0  9994
2 48618   6.2571 325.5832 0001934  40.1131 173.4834  1.00272153 10748
0 FENGYUN 4B
1 48808U 21047A   24100.05535321 -.00000338  00000-0  00000+0 0  9997
2 48808   0.1977 284.3975 0005767 203.6674 194.3956  1.00271249 10548
0 XM-8
1 48838U 21049A   24099.56645993 -.00000213  00000-0  00000+0 0  9994
2 48838   0.0130 235.6441 0001558 106.1270 334.2218  1.00270495 10606
0 TIANLIAN 1-O5
1 49011U 21063A   24100.15943078  .00000100  00000-0  00000+0 0  9991
2 49011   0.4809 280.7236 0002066  88.6981 262.6771  1.00271804 10061
0 STARONE D2
1 49055U 21069A   24099.92742868 -.00000272  00000-0  00000-0 0  9993
2 49055   0.0222 175.4313 0000981 221.7864  64.2438  1.00270118  9919
0 EUTELSAT QUANTUM
1 49056U 21069B   24099.92966015  .00000117  00000-0  00000-0 0  9993
2 49056   0.0451  48.7443 0002398 324.2340 207.0679  1.00273809  9916
0 CHINASAT 2E
1 49062U 21071A   24099.93591245 -.00000300  00000-0  00000-0 0  9998
2 49062   0.0596  50.2859 0001981  79.4183 142.9080  1.00274464 10028
0 TJS-7
1 49115U 21077A   24099.92154097 -.00000131  00000-0  00000-0 0  9994
2 49115   0.1347  78.0149 0000684 289.3805  62.4912  1.00273010  1558
0 CHINASAT 9B
1 49125U 21080A   24100.57898728 -.00000319  00000-0  00000+0 0  9996
2 49125   0.0158 204.9010 0001424 157.4431 145.7798  1.00266681  9627
0 SJ-21
1 49330U 21094A   24100.01205459 -.00000363  00000-0  00000-0 0  9991
2 49330   9.6838  45.7507 0079812  30.6867 250.3268  1.00345675  9242
0 SES 17
1 49332U 21095A   24099.99957035 -.00000280  00000-0  00000+0 0  9999
2 49332   0.0141 157.5903 0000953 205.1322 127.7465  1.00269365  9824
0 SYRACUSE 4A
1 49333U 21095B   24100.16517427  .00000127  00000-0  00000+0 0  9998
2 49333   0.0176 235.6998 0001563 134.6833 292.4890  1.00270729  9970
0 CHINASAT 1D
1 49505U 21114A   24099.96719799 -.00000345  00000-0  00000-0 0  9996
2 49505   0.0361 317.1789 0001945 223.2527 135.5989  1.00270141  8816
0 STPSAT-6
1 49817U 21118A   24099.91910434 -.00000053  00000-0  00000-0 0  9996
2 49817   0.0295 111.8398 0000926 252.1869  52.8396  1.00271158  8543
0 LDPE-1
1 49818U 21118B   24100.00743990 -.00000188  00000-0  00000-0 0  9990
2 49818   2.0529  88.5919 0015426 172.7360 261.4090  1.00818116  8628
0 EXPRESS AMU-7
1 50001U 21123A   24100.04778807 -.00000250  00000-0  00000+0 0  9990
2 50001   0.0228 247.4763 0000421  44.7935  67.7193  1.00271578  8521
0 EXPRESS AMU-3
1 50002U 21123B   24099.16690003 -.00000331  00000-0  00000-0 0  9993
2 50002   0.0237 249.1098 0000421 142.2923 328.5935  1.00274284  8500
0 TIANLIAN 2-02
1 50005U 21124A   24100.49840899 -.00000023  00000-0  00000+0 0  9993
2 50005   0.8500 274.7111 0002417  76.3861 197.4942  1.00271267  8609
0 TURKSAT 5B
1 50212U 21126A   24099.66329819  .00000137  00000-0  00000-0 0  9991
2 50212   0.0715 351.1939 0005018 350.2375 136.7669  1.00271623  8283
0 INMARSAT 6-F1
1 50319U 21128A   24099.67165235 -.00000180  00000-0  00000+0 0  9990
2 50319   0.0159 258.7477 0000116 299.6615 324.5366  1.00271117  8222
0 SHIYAN 12 01 (SY-12 01)
1 50321U 21129A   24099.95640719 -.00000144  00000-0  00000-0 0  9991
2 50321   1.6054  92.0907 0000768  33.1272 132.4002  1.00464919  8481
0 SHIYAN 12 02 (SY-12 02)
1 50322U 21129B   24100.02216917 -.00000034  00000-0  00000-0 0  9990
2 50322   1.6040  92.0901 0002156 211.2016  75.7342  1.00106926  8486
0 TJS-9
1 50574U 21135A   24099.96867736 -.00000307  00000-0  00000-0 0  9990
2 50574   0.0668  90.4475 0001622  10.9906 222.2389  1.00272471  8424
0 USA 324
1 51280U 22006A   24098.33104895 -.00000122  00000-0  00000+0 0  9997
2 51280   1.6486  89.4257 0000482 311.5584  70.5879  1.00411158  8105
0 USA 325
1 51281U 22006B   24080.42394150 -.00000373  00000-0  00000+0 0  9992
2 51281   0.1054  74.3757 0000619  81.6114 298.1585  1.00165384  7901
0 GOES 18
1 51850U 22021A   24100.12888587  .00000089  00000-0  00000+0 0  9996
2 51850   0.0104  76.4004 0000428 241.2777 149.5688  1.00270660  7797
0 CHINASAT 6D
1 52255U 22038A   24100.10298647 -.00000361  00000-0  00000+0 0  9998
2 52255   0.0288 250.0296 0001467  82.2013  27.7548  1.00272871  7432
0 NILESAT 301
1 52817U 22061A   24099.87966034 -.00000066  00000-0  00000-0 0  9994
2 52817   0.0288   6.2615 0005931   4.1825 136.8285  1.00268694  6725
0 CMS-02
1 52903U 22067A   24100.09632769 -.00000173  00000-0  00000+0 0  9996
2 52903   0.0042 214.9272 0006627 229.2691 231.3305  1.00271619  6589
0 MEASAT 3D
1 52904U 22067B   24099.95954000 -.00000250  00000-0  00000-0 0  9991
2 52904   0.0425 346.7697 0002683 358.6989 289.1549  1.00272145  6637
0 SES 22
1 52933U 22071A   24099.62814105  .00000080  00000-0  00000-0 0  9993
2 52933   0.0140 258.6915 0001128 125.6728 264.1337  1.00273321  6526
0 WFOV (USA 332)
1 52940U 22073A   24100.04980135 -.00000369  00000-0  00000+0 0  9999
2 52940   1.6468  89.0696 0002228 245.0610   0.8569  1.00268953  6342
0 TIANLIAN-2 03
1 53100U 22078A   24099.19601773  .00000061  00000-0  00000-0 0  9993
2 53100   1.4078 279.0837 0002563 106.0892 252.8848  1.00272932  6536
0 SBIRS GEO 6 (USA 336)
1 53355U 22092A   24099.96893844 -.00000307  00000-0  00000-0 0  9992
2 53355   4.7182 311.0454 0003872  54.8225 319.6322  1.00271169  6222
0 EUTE KONNECT VHTS
1 53765U 22110A   24100.12965948  .00000009  00000-0  00000+0 0  9991
2 53765   0.0573  90.1908 0002605 275.2965 241.7316  1.00268790  5775
0 CHINASAT 1E
1 53813U 22112A   24100.10069056 -.00000157  00000-0  00000+0 0  9995
2 53813   0.0590 300.5263 0001878  70.1144 304.8237  1.00270363  5820
0 SES 20
1 53960U 22123A   24099.63449372 -.00000109  00000-0  00000-0 0  9996
2 53960   0.0213 145.8321 0001578 281.1274 255.7311  1.00270280  5527
0 SES 21
1 53961U 22123B   24099.98686507  .00000061  00000-0  00000+0 0  9993
2 53961   0.0147  91.5227 0001037 281.7267  48.7303  1.00271532  5537
0 GALAXY 33
1 54026U 22128A   24100.50273500  .00000071  00000-0  00000+0 0  9998
2 54026   0.0173 231.6721 0001785 142.3451 232.1889  1.00271355  5735
0 GALAXY 34
1 54027U 22128B   24099.70089196  .00000051  00000-0  00000-0 0  9993
2 54027   0.0189 242.6381 0001542 132.6588 305.4600  1.00273438  5897
0 ANGOSAT 2
1 54033U 22131A   24099.94587744  .00000127  00000-0  00000-0 0  9996
2 54033   0.0114 206.9668 0001200 253.6520 100.5654  1.00272432  5452
0 HOTBIRD 13F
1 54048U 22134A   24099.90674442  .00000079  00000-0  00000-0 0  9999
2 54048   0.0650 288.3554 0002528  49.1176 199.5854  1.00271502  5435
0 LDPE-2
1 54219U 22144A   24100.49851499 -.00000272  00000-0  00000+0 0  9991
2 54219   3.6608  65.1475 0000804  19.0774 215.1635  1.00631893  3786
0 HOTBIRD 13G
1 54225U 22146A   24099.65755470  .00000079  00000-0  00000-0 0  9992
2 54225   0.0472 243.5444 0001905 333.5909 229.9798  1.00271809  5207
0 CHINASAT 19
1 54230U 22148A   24099.68749049 -.00000088  00000-0  00000-0 0  9999
2 54230   0.0353  91.0388 0004686 179.7419 337.4956  1.00270863  5355
0 GALAXY 31
1 54243U 22153A   24100.51251250  .00000005  00000-0  00000+0 0  9993
2 54243   0.0199 184.1546 0001343 190.5718 247.0135  1.00272197  5141
0 GALAXY 32
1 54244U 22153B   24099.92328142 -.00000182  00000-0  00000-0 0  9999
2 54244   0.0179 279.6823 0002841  87.6807  71.6649  1.00271313  5125
0 EUTELSAT 10B
1 54259U 22157A   24100.00188256  .00000061  00000-0  00000-0 0  9996
2 54259   0.0583  90.3212 0002688 216.7668 261.3133  1.00272933  5012
0 GALAXY 35
1 54741U 22170A   24099.92284632 -.00000170  00000-0  00000-0 0  9992
2 54741   0.0192 175.5495 0001221 199.8637  61.3638  1.00271539  4874
0 GALAXY 36
1 54742U 22170B   24099.99521924 -.00000193  00000-0  00000+0 0  9992
2 54742   0.0129 241.4807 0002881 183.9949  41.5258  1.00271775  4902
0 METEOSAT 12 (MTG I1)
1 54743U 22170C   24100.15603691 -.00000016  00000-0  00000+0 0  9990
2 54743   0.4199  23.3197 0000907  10.4766 219.7951  1.00270578  4966
0 SJ-23
1 55131U 23002A   24100.00979997 -.00000198  00000-0  00000-0 0  9998
2 55131   1.6534  88.5797 0001412 319.8073 243.6062  1.00066582  4497
0 USA 340
1 55138U 22144E   24099.53830446 -.00000241  00000-0  00000-0 0  9990
2 55138   3.6244  65.9626 0001185  29.7352  96.0167  0.99560653  3768
0 LINUSS1
1 55246U 22144G   24099.73868823 -.00000294  00000-0  00000-0 0  9990
2 55246   3.4212  67.8166 0010601 298.4429 241.4711  0.99056503  3747
0 LINUSS2
1 55247U 22144H   24099.74424538 -.00000293  00000-0  00000-0 0  9992
2 55247   3.4207  67.7953 0009649 307.6677 231.8728  0.99054004  3793
0 USA 342
1 55263U 23008A   24097.29722304  .00000130  00000-0  00000+0 0  9998
2 55263   0.0115 133.6346 0000347 157.9466  34.5216  1.00270879  4489
0 LDPE-3A
1 55264U 23008B   24100.13967664 -.00000258  00000-0  00000+0 0  9999
2 55264   1.0687  88.1231 0000851  21.2469  56.3565  1.00637167  3797
0 ELEKTRO-L 4
1 55506U 23016A   24098.99295282 -.00000067  00000-0  00000-0 0  9996
2 55506   0.0789  91.8678 0001635 216.7325  51.3781  1.00271492  4305
0 AMAZONAS NEXUS
1 55508U 23017A   24096.49075733 -.00000291  00000-0  00000+0 0  9997
2 55508   0.0184  75.0712 0004791 312.5620 282.3115  1.00266468  4201
0 CHINASAT-26
1 55686U 23023A   24099.96624073 -.00000362  00000-0  00000+0 0  9992
2 55686   0.0705  72.6451 0003031 348.4011 249.6210  1.00271270  4229
0 LUCH (OLYMP) 2
1 55841U 23031A   24100.00188256  .00000024  00000-0  00000-0 0  9995
2 55841   0.0518 315.4079 0000773  80.1779 167.5710  1.00272500  3936
0 GAOFEN 13 02
1 55912U 23036A   24100.18527649 -.00000243  00000-0  00000+0 0  9994
2 55912   2.1733 304.3844 0003852 319.2893 147.6546  1.00269086  4075
0 SES 18
1 55970U 23038A   24099.92084479 -.00000109  00000-0  00000-0 0  9990
2 55970   0.0249 253.4151 0001784  65.4773 107.1596  1.00272506  4080
0 SES 19
1 55971U 23038B   24099.69976067  .00000080  00000-0  00000-0 0  9995
2 55971   0.0165 228.7786 0001352 151.5654 294.0817  1.00271774  4058
0 INTELSAT 40E
1 56174U 23052A   24099.92319440 -.00000182  00000-0  00000-0 0  9993
2 56174   0.0052 200.4354 0000787 218.5590  20.0062  1.00271259  3800
0 VIASAT 3
1 56370U 23060A   24099.92362951 -.00000193  00000-0  00000-0 0  9997
2 56370   0.0186 134.5636 0000458 208.2261  98.4769  1.00271936  3425
0 ARCTURUS
1 56371U 23060B   24100.52958205 -.00000123  00000-0  00000+0 0  9992
2 56371   0.7076  89.1236 0000132  74.1662 303.5837  1.00271561  3448
0 GS-1
1 56372U 23060C   24095.70000000 -.00000286  00000-0  00000+0 0  9997
2 56372   0.6440  88.3180 0003732 252.1820 255.1730  0.99762009  3381
0 BEIDOU 3 G4
1 56564U 23066A   24100.11635462 -.00000122  00000-0  00000+0 0  9993
2 56564   2.2963 288.2112 0003595 190.5082 281.0331  1.00272236  3397
0 ARABSAT 7B (BADR 8)
1 56757U 23075A   24099.85691571  .00000137  00000-0  00000+0 0  9993
2 56757   0.0758  28.9963 0005568 334.4963 168.5688  1.00272521  3112
0 NVS-01
1 56759U 23076A   24099.96711095 -.00000350  00000-0  00000+0 0  9990
2 56759   4.1760 267.6922 0005473  20.0837  27.5983  1.00267664  3194
0 NUSANTARA TIGA
1 57045U 23086A   24099.89874106 -.00000243  00000-0  00000-0 0  9997
2 57045   0.0478 314.8222 0005358  19.9393 332.4083  1.00268988  1746
0 H2SAT
1 57213U 23093A   24100.15621094 -.00000008  00000-0  00000+0 0  9998
2 57213   0.0165 208.3119 0001024 316.1232  90.1776  1.00270194  2861
0 SYRACUSE 4B
1 57214U 23093B   24099.67688556 -.00000062  00000-0  00000-0 0  9990
2 57214   0.0100 214.8143 0000050 348.6338 231.1436  1.00269522  1496
0 JUPITER 3 (ECHOSTAR 24)
1 57479U 23108A   24099.71604804 -.00000158  00000-0  00000+0 0  9994
2 57479   0.0191 203.5162 0001239 175.6018 340.8867  1.00270142  2631
0 GALAXY 37
1 57493U 23112A   24100.13088737  .00000040  00000-0  00000+0 0  9994
2 57493   0.0164 297.3381 0001956  80.2367 100.4040  1.00273474  2505
0 LUDI TANCE-4 01A
1 57624U 23120A   24100.31702868 -.00000201  00000-0  00000+0 0  9997
2 57624  15.8207 213.3762 0003446 157.8509  30.4685  1.00274582  2529
0 TJS-10
1 58204U 23169A   24100.02667727 -.00000004  00000-0  00000-0 0  9999
2 58204   0.4711  88.6421 0007391 270.9898  20.9925  1.00268522  1527
0 CHINASAT-6E
1 58253U 23172A   24099.96432623 -.00000370  00000-0  00000-0 0  9998
2 58253   0.0276 267.4562 0003610 173.2120 219.7038  1.00271730  1640
0 YAOGAN-41
1 58582U 23197A   24100.04389640 -.00000377  00000-0  00000+0 0  9998
2 58582   4.7502 307.7205 0017047 166.5253 222.5758  1.00274910  1308
0 INSAT 3DS
1 58990U 24033A   24100.09632769 -.00000172  00000-0  00000+0 0  9992
2 58990   0.1574  87.0334 0011090 359.5101 228.9594  1.00271162   818
0 TELKOMSAT 113BT
1 58995U 24035A   24100.48979937 -.00000365  00000-0  00000+0 0  9997
2 58995   0.0217 108.1536 0001126 278.1751 101.1533  1.00269562   626
0 TJS-11
1 59020U 24037A   24100.01455112 -.00000384  00000-0  00000-0 0  9994
2 59020   5.4512 321.1610 0002929 142.5782 219.4400  1.00276043   575
0 HG-01
1 59069U 24040A   24100.56646753  .00000148  00000-0  00000+0 0  9990
2 59069   0.0201  57.1837 0003653 339.3766  39.3087  1.00274000   564


Writing TLE_GEO.txt


In [ ]:
#@title Starlink Group 7

%%file TLE_STARLINK.txt
1 57663U 23124A   24100.00002315  .00048970  00000-0  17715-2 0  9998
2 57663  53.1171 218.0419 0001345  81.3491 349.6081 15.27971769  5821
1 57840U 23141A   24100.58334491  .02538192  00000-0  99040-1 0  9992
2 57840  53.1093 291.4848 0001340  80.8626 207.7701 15.21674457 33306
1 57935U 23148A   24100.10033608  .00018279  00000-0  67057-3 0  9990
2 57935  53.1071  33.1143 0001271  94.1432 265.9713 15.28038012 31329
1 58028U 23156A   24100.27734911  .00070268  00000-0  24529-2 0  9998
2 58028  53.1009  93.4983 0002641 103.7060 256.4233 15.28972992 29144
1 58097U 23161A   24100.48543353 -.00004540  00000-0 -15147-3 0  9990
2 58097  53.1071 171.6830 0001131  84.1923 275.9205 15.27992870 27398
1 58150U 23166A   24100.58334491  .00031527  00000-0  11477-2 0  9991
2 58150  53.1152 225.6976 0000993  76.9636 182.7393 15.27971484  5863
1 58374U 23178A   24100.45533783  .00001280  00000-0  58829-4 0  9990
2 58374  53.1089  11.3045 0001108  76.7921 283.3201 15.28044119 22807
1 58531U 23192A   24100.18467691 -.00000100  00000-0  90607-5 0  9997
2 58531  53.1604  71.4782 0001443 105.5166 254.5994 15.28028526 18891
1 58705U 24002A   24100.46209456 -.00017999  00000-0 -14421-3 0  9998
2 58705  53.1554 128.4526 0001194  84.1584 275.9570 15.69679663 16519
1 58765U 24011A   24099.53851345  .00000252  00000-0  36957-4 0  9990
2 58765  53.1576 304.6979 0001450  94.4835 265.6320 15.04912660 14239
1 58826U 24017A   24100.62822405  .00019113  00000-0  70009-3 0  9993
2 58826  53.1574 220.1299 0003411  88.3513 271.7878 15.28056216 13020
1 58876U 24020A   24100.33772480  .00000867  00000-0  43986-4 0  9998
2 58876  53.1568 331.1037 0000769  73.1232 286.9852 15.28000050 12174
1 58931U 24027A   24100.29242358  .00029473  00000-0  10720-2 0  9990
2 58931  53.1545 317.1188 0000938  75.1535 284.9569 15.28035008 10103
1 58965U 24031A   24100.29415884 -.00000791  00000-0 -15866-4 0  9990
2 58965  53.1589 305.2532 0002128  85.1045 275.0198 15.28015405  9453
1 58998U 24036A   24100.21770524  .00003050  00000-0  12155-3 0  9994
2 58998  53.1602  86.2148 0001413  99.2722 260.8437 15.28352747  8256
1 59254U 24050A   24100.27875535  .00065508  00000-0  14674-2 0  9993
2 59254  53.1603 199.7831 0001472  76.1357 283.9812 15.42855877  4476
1 59201U 24046A   24100.50457127 -.00004171  00000-0 -84482-4 0  9996
2 59201  53.1592 177.0257 0001375 100.6870 259.4291 15.42821758  5573
1 59374U 24062A   24100.58334491 -.01154883  00000-0 -66087-2 0  9997
2 59374  53.1666 280.6562 0000962 109.0497 346.4104 15.80679753  2402


Writing TLE_STARLINK.txt


### *The function graph is in charge of calculating the positions or orbits of the objects specified on the txt file (TLE or 3LE). Also it can create the graph direcly. Use the parameters for choosing the output that you desire.*

In [ ]:
def graph(nameFile, reference = "orbit", visual = True, color_line = 'rgb(0,0,255)', data = False):

  data = open(nameFile, "r")

  lines = data.readlines()

  for i in range(len(lines)):

    if lines[i][0] == "1" :

      linea1 = lines[i]

      linea2 = lines[i+1]

      satelite = twoline2rv(linea1,linea2,wgs84)

      a = satelite.a*satelite.radiusearthkm
      e = satelite.ecco
      I = satelite.inclo
      Omega = satelite.Om
      omega = satelite.argpo
      p = a*(1-e**2)
      M_inversa = spy.eul2m(-Omega,-I,-omega,3,1,3)

      if reference == "orbit":

        fs  = np.linspace(0,2*np.pi,100)
        rs = p/(1+e*np.cos(fs))
        xppps = rs*np.cos(fs)
        yppps = rs*np.sin(fs)
        zppps = np.zeros_like(xppps)

        rs = np.zeros((100,3))
        for i in range(100):
          rs[i] = spy.mxv(M_inversa,[xppps[i],yppps[i],zppps[i]])

        fig.add_trace(
            go.Scatter3d(
                x=rs[:,0], y=rs[:,1], z=rs[:,2],
                mode='lines', line=dict(color="{}".format(color_line)),
                showlegend=False )
        )

      elif reference == "point":

        fs  = 0
        rs = p/(1+e*np.cos(fs))
        xppps = rs*np.cos(fs)
        yppps = rs*np.sin(fs)
        zppps = np.zeros_like(xppps)

        rs = spy.mxv(M_inversa,[xppps,yppps,zppps])

        fig.add_trace(
            go.Scatter3d(
                x=[rs[0]], y=[rs[1]], z=[rs[2]],
                mode='markers',
                marker=dict(size=1),
                showlegend=False )
        )

  if visual == True:
    fig.show()

  return rs


#*Making the Starlink Group 7 orbits*

In [ ]:
graph("TLE_STARLINK.txt")

#*Making the plot for all the Geosynchronous satellites that have reported data in the last 30 days*

In [ ]:
graph("TLE_GEO.txt","point")

# *Creating  .txt  with the TLE information*

In [ ]:
#@title ISS
%%file TLE_ISS.txt
1 25544U 98067A   24100.57837436  .00012133  00000-0  21993-3 0  9997
2 25544  51.6397 295.5767 0004852  51.9778   3.8711 15.50034484447908


Writing TLE_ISS.txt


In [ ]:
#@title Hubble Telescope
%%file TLE_HST.txt
1 20580U 90037B   24100.38214272  .00004903  00000-0  23433-3 0  9994
2 20580  28.4684  85.7582 0002388   6.3795 353.6828 15.16642789666579

Writing TLE_HST.txt


In [ ]:
#@title Tiangong
%%file TLE_CSS.txt
1 48274U 21035A   24100.56220095  .00017862  00000-0  17907-3 0  9993
2 48274  41.4673 142.4221 0008563 355.1573   4.9183 15.65059665168355

Writing TLE_CSS.txt


In [ ]:
#@title Code for Graph the earth
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import sys
import pandas as pd
import numpy as np
import requests
import plotly.graph_objects as go
import numpy as np

def dibuja_tierra():
  def plot_back(fig):
      """back half of sphere"""
      clor=f'rgb(220, 220, 220)'
      R = np.sqrt(6368.134)
      u_angle = np.linspace(0, np.pi, 25)
      v_angle = np.linspace(0, np.pi, 25)
      x_dir = np.outer(R*np.cos(u_angle), R*np.sin(v_angle))
      y_dir = np.outer(R*np.sin(u_angle), R*np.sin(v_angle))
      z_dir = np.outer(R*np.ones(u_angle.shape[0]), R*np.cos(v_angle))
      fig.add_surface(z=z_dir, x=x_dir, y=y_dir, colorscale=[[0, clor], [1, clor]], opacity=1.0, showlegend=False, lighting=dict(diffuse=0.1)) # opacity=fig.sphere_alpha, colorscale=[[0, fig.sphere_color], [1, fig.sphere_color]])

  def plot_front(fig):
      """front half of sphere"""
      clor=f'rgb(220, 220, 220)'
      R = np.sqrt(6368.134)
      u_angle = np.linspace(-np.pi, 0, 25)
      v_angle = np.linspace(0, np.pi, 25)
      x_dir = np.outer(R*np.cos(u_angle), R*np.sin(v_angle))
      y_dir = np.outer(R*np.sin(u_angle), R*np.sin(v_angle))
      z_dir = np.outer(R*np.ones(u_angle.shape[0]), R*np.cos(v_angle))
      fig.add_surface(z=z_dir, x=x_dir, y=y_dir, colorscale=[[0, clor], [1, clor]], opacity=1.0, showlegend=False, lighting=dict(diffuse=0.1)) # opacity=fig.sphere_alpha, colorscale=[[0, fig.sphere_color], [1, fig.sphere_color]])

  def plot_polygon(poly):
      xy_coords = poly.exterior.coords.xy
      lon = np.array(xy_coords[0])
      lat = np.array(xy_coords[1])

      lon = lon * np.pi/180
      lat = lat * np.pi/180

      R = 6378.134
      x = R * np.cos(lat) * np.cos(lon)
      y = R * np.cos(lat) * np.sin(lon)
      z = R * np.sin(lat)
      return x, y, z

  # Read the shapefile.  Creates a DataFrame object
  gdf = gpd.read_file("ne_110m_admin_0_countries.shp")
  fig = go.Figure()
  plot_front(fig)
  plot_back(fig)

  marker = dict(color=[f'rgb({np.random.randint(0,256)}, {np.random.randint(0,256)}, {np.random.randint(0,256)})' for _ in range(25)],
            size=10)

  for i in gdf.index :
      # print(gdf.loc[i].NAME)            # Call a specific attribute

      polys = gdf.loc[i].geometry         # Polygons or MultiPolygons

      if polys.geom_type == 'Polygon':
          x, y, z = plot_polygon(polys)
          fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color=f'rgb(0, 0,0)'), showlegend=False) )

      elif polys.geom_type == 'MultiPolygon':

          for poly in polys.geoms:
              x, y, z = plot_polygon(poly)
              fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color=f'rgb(0, 0,0)'), showlegend=False) )
  fig.show()
  return fig

In [ ]:
#@title Reset graph for plotting new lines


fig = dibuja_tierra()

#*Orbits for the ISS, HST and CSS*

In [ ]:
graph("TLE_CSS.txt", visual= False, color_line="black")
graph("TLE_ISS.txt", visual= False, color_line ="rgb(255,0,0)")
graph("TLE_HST.txt", color_line= "yellow")

###*Getting the positions of each object through all the orbit*

In [ ]:
css_rs = graph("TLE_CSS.txt", visual= False, color_line="black", data=True)
iss_rs = graph("TLE_ISS.txt", visual= False, color_line ="rgb(255,0,0)",data=True)
hst_rs = graph("TLE_HST.txt", visual = False ,color_line= "yellow",data=True)

#*For calculate the minimum distance of the ISS in respect to an object, the distance between two point was calculated (Euclidean function). After calculating all the distances between all the possible positions of the two objects, we look for the smallest distance.*

#*Minimal distance between **ISS** and **CSS**.*

In [ ]:
mins = np.zeros((100))

for a in range(len(iss_rs)):

  distances = [euclidean((iss_rs[a,0],iss_rs[a,1],iss_rs[a,2]),(css_rs[i,0],css_rs[i, 1],css_rs[i,2])) for i in range(len(css_rs)) ]

  mins[a] = np.min(distances)

iss_css = np.min(mins)

iss_css

167.31573056033514

#*The minimal distance between the ISS and CSS could be 167km*

#*Minimal distance between **ISS** and **HST**.*

In [ ]:
mins = np.zeros((100))

for a in range(len(iss_rs)):

  distances = [euclidean((iss_rs[a,0],iss_rs[a,1],iss_rs[a,2]),(hst_rs[i,0],hst_rs[i, 1],hst_rs[i,2])) for i in range(len(hst_rs)) ]

  mins[a] = np.min(distances)

iss_hst = np.min(mins)

iss_hst

170.22150860257932

#*The minimal distance between the ISS and HST could be 170km*